# For Reading Parameter Files to just see the fit parameter values

In [1]:
%reset -f
import ROOT
import traceback
from MyCommonAnalysisFunction_richcap import *
from array import array
from datetime import datetime


ROOT.TH1.AddDirectory(0)
# ROOT.gStyle.SetTitleOffset(1.3,'y')
ROOT.gStyle.SetTitleOffset(1.75,'y')

ROOT.gStyle.SetGridColor(17)
ROOT.gStyle.SetPadGridX(1)
ROOT.gStyle.SetPadGridY(1)

print(f"{color.BOLD}\nStarting RG-A SIDIS Analysis\n{color.END}")


Starting RG-A SIDIS Analysis



In [66]:
import re
import numpy as np
import pandas as pd

# Regex for ANSI escape sequences
ansi_escape = re.compile(r'\x1B\[[0-?]*[ -/]*[@-~]')

def parse_fit_file(path, nrows=6, ncols=6):
    results = []

    current_q2y = None
    current_zpt = None
    buffer_lines = []
    parB = None
    parC = None
    chi2 = None

    # Regex for header after stripping ANSI
    header_re = re.compile(r"For SMEARED Q2-y Bin (\d+) - z-PT Bin (\d+):")

    with open(path, "r", encoding="utf-8") as f:
        for raw_line in f:
            # Strip ANSI codes
            line = ansi_escape.sub("", raw_line).strip()

            # Match header
            m = header_re.match(line)
            if(m):
                current_q2y = int(m.group(1))
                current_zpt = int(m.group(2))
                buffer_lines = []
                parB = None
                parC = None
                chi2 = None

            elif(line.startswith("Par B")):
                buffer_lines.append(line)
                try:
                    val = line.split("=")[1].split("±")[0].strip()
                    # parB = float(val)
                    # print(f"current_zpt = {current_zpt}\n\tparB = {parB}")
                except Exception:
                    parB = None

            elif(line.startswith("Par C")):
                buffer_lines.append(line)
                try:
                    val = line.split("=")[1].split("±")[0].strip()
                    parC = float(val)
                except Exception:
                    parC = None

            elif(line.startswith("chi2/NDF")):
                buffer_lines.append(line)
                try:
                    val = line.split("=")[1].strip()
                    chi2 = float(val)
                except Exception:
                    chi2 = None

                if((current_q2y is not None) and (current_zpt is not None)):
                    results.append({
                        "Q2y": current_q2y,
                        "zPT": current_zpt,
                        "ParB": parB,
                        "ParC": parC,
                        "Chi2NDF": chi2,
                        "Details": "\n".join(buffer_lines)
                    })
                buffer_lines = []
                parB = None
                parC = None
                chi2 = None

            else:
                if(current_q2y is not None):
                    buffer_lines.append(line)

    df = pd.DataFrame(results)
    if(not df.empty):
        df = df.set_index("zPT").sort_index()

    grid_B = np.full((nrows, ncols), np.nan)
    grid_C = np.full((nrows, ncols), np.nan)
    grid_N = np.full((nrows, ncols), np.nan)

    if(not df.empty):
        for zpt, row in df.iterrows():
            r = (zpt - 1) // ncols
            c = (zpt - 1) % ncols
            if((0 <= r < nrows) and (0 <= c < ncols)):
                grid_B[r, c] = row["ParB"]
                grid_C[r, c] = row["ParC"]
                grid_N[r, c] = row["Chi2NDF"]

    df_B = pd.DataFrame(grid_B, index=[f"Bin {(i*ncols)+1}:" for i in range(nrows)],
                        columns=[f"Bin {j+1}:" for j in range(ncols)])
    df_C = pd.DataFrame(grid_C, index=[f"Bin {(i*ncols)+1}:" for i in range(nrows)],
                        columns=[f"Bin {j+1}:" for j in range(ncols)])
    df_N = pd.DataFrame(grid_N, index=[f"Bin {(i*ncols)+1}:" for i in range(nrows)],
                        columns=[f"Bin {j+1}:" for j in range(ncols)])

    return df, df_B, df_C, df_N


# def plot_grids(df_B, df_C, df_N):
#     plt.imshow(df_B.values, cmap="coolwarm", interpolation="none")
#     plt.colorbar(label="Par B")
#     plt.title("Parameter B across bins (6x6 grid)")
#     plt.show()

#     plt.imshow(df_C.values, cmap="viridis", interpolation="none")
#     plt.colorbar(label="Par C")
#     plt.title("Parameter C across bins (6x6 grid)")
#     plt.show()

#     plt.imshow(df_N.values, cmap="viridis", interpolation="none")
#     plt.colorbar(label="Chi2NDF")
#     plt.title("Chi2NDF across bins (6x6 grid)")
#     plt.show()


import ROOT

def plot_grids(df_B, df_C, df_N):
    nrows, ncols = df_B.shape

    # Histograms
    hB = ROOT.TH2D("ParB", "Parameter B;Column;Row", ncols, 0.5, ncols+0.5, nrows, 0.5, nrows+0.5)
    hC = ROOT.TH2D("ParC", "Parameter C;Column;Row", ncols, 0.5, ncols+0.5, nrows, 0.5, nrows+0.5)
    hN = ROOT.TH2D("Chi2NDF", "Chi2NDF;Column;Row",  ncols, 0.5, ncols+0.5, nrows, 0.5, nrows+0.5)

    # Fill contents (note: DataFrame index is 0-based, ROOT bins are 1-based)
    for r in range(nrows):
        for c in range(ncols):
            b_val = df_B.iat[r, c]
            c_val = df_C.iat[r, c]
            n_val = df_N.iat[r, c]
            if(not pd.isna(b_val)):
                hB.SetBinContent(c+1, r+1, b_val)
            if(not pd.isna(c_val)):
                hC.SetBinContent(c+1, r+1, c_val)
            if(not pd.isna(n_val)):
                hN.SetBinContent(c+1, r+1, n_val)

    c1 = ROOT.TCanvas("c1", "ParB", 1600, 1200)
    c1.Divide(3, 1, 0.001, 0.001)
    c1.cd(1)
    hB.Draw("COLZ TEXT")
    c1.Update()
    c1.cd(2)
    hC.Draw("COLZ TEXT")
    c1.Update()
    c1.cd(3)
    hN.Draw("COLZ TEXT")
    c1.Update()

    return hB, hC, hN, c1


# ----------------------------
# Example usage
# ----------------------------

# 1) From a file:
# df, df_B, df_C = parse_fit_file("fit_parameters.txt", nrows=6, ncols=6, q2y_bin=5)
# print(df.head())
# print("\nGrid of Par B:\n", df_B)
# print("\nGrid of Par C:\n", df_C)

# 2) From an in-memory string (handy for quick tests):
# sample_text = \"\"\"<paste your TXT here>\"\"\"
# df, df_B, df_C = parse_fit_text(sample_text, q2y_bin=5)
# df_B, df_C

# Path to your file
filename = "Parameters_Pass_2_Sector_Tests_FC_14_V1_EvGen_Ran_on_9_22_2025_Q2_y_Bins_5_Smeared.txt"

df, df_B, df_C, df_N = parse_fit_file(filename, nrows=6, ncols=6)
# print(df.head())
print(f"\n{color.BOLD}Grid of Par B:{color.END}\n{df_B}")
print(f"\n{color.BOLD}Grid of Par C:{color.END}\n{df_C}")
# print(f"\n{color.BOLD}Grid of Chi2NDF:{color.END}\n{df_N}")

hB, hC, hN, c1 = plot_grids(df_B, df_C, df_N)

c1.Draw()


Grid of Par B:
         Bin 1:  Bin 2:  Bin 3:  Bin 4:  Bin 5:  Bin 6:
Bin 1:      NaN     NaN     NaN     NaN     NaN     NaN
Bin 7:      NaN     NaN     NaN     NaN     NaN     NaN
Bin 13:     NaN     NaN     NaN     NaN     NaN     NaN
Bin 19:     NaN     NaN     NaN     NaN     NaN     NaN
Bin 25:     NaN     NaN     NaN     NaN     NaN     NaN
Bin 31:     NaN     NaN     NaN     NaN     NaN     NaN

Grid of Par C:
               Bin 1:    Bin 2:    Bin 3:    Bin 4:        Bin 5:    Bin 6:
Bin 1:  -1.592534e-02  0.034865  0.025981  0.027086  1.820384e-13  0.032386
Bin 7:  -2.245680e-09  0.032636 -0.006955 -0.029571 -1.113481e-02  0.061881
Bin 13: -4.626566e-09 -0.001186 -0.018218 -0.001950  1.149595e-03  0.114373
Bin 19: -3.881049e-03 -0.025000 -0.038710 -0.015614  2.613844e-02       NaN
Bin 25: -5.894557e-09 -0.032035 -0.013636 -0.017105  5.819263e-02       NaN
Bin 31: -1.071867e-10 -0.023650 -0.013587  0.026046           NaN       NaN


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


In [64]:
testB = """Bin_1:  -5.766642e-02 -0.099021 -0.136051 -0.130691 -0.162074 -0.299121
Bin_7:  -2.769346e-02 -0.036273 -0.072000 -0.098819 -0.169508 -0.276181
Bin_13: -1.069385e-02 -0.059146 -0.054865 -0.071292 -0.135149 -0.321248
Bin_19: -7.058285e-03 -0.040592 -0.050000 -0.091596 -0.191425       NaN
Bin_25: -6.608292e-09 -0.051372 -0.050000 -0.129273 -0.191433       NaN
Bin_31: -8.101156e-10 -0.046543 -0.059977 -0.138144       NaN       NaN"""
testB = testB.split("\n")

testC = """Bin 1:  -1.592534e-02  0.034865  0.025981  0.027086  1.820384e-13  0.032386
Bin 7:  -2.245680e-09  0.032636 -0.006955 -0.029571 -1.113481e-02  0.061881
Bin 13: -4.626566e-09 -0.001186 -0.018218 -0.001950  1.149595e-03  0.114373
Bin 19: -3.881049e-03 -0.025000 -0.038710 -0.015614  2.613844e-02       NaN
Bin 25: -5.894557e-09 -0.032035 -0.013636 -0.017105  5.819263e-02       NaN
Bin 31: -1.071867e-10 -0.023650 -0.013587  0.026046           NaN       NaN"""
testC = testC.split("\n")

test = testB
table_rows = {}
for row, ii in enumerate(test):
    while("  " in ii):
        ii = str(ii.replace("  ", " "))
    # print(ii.split(" "))
    table_rows[f"Row_{row+1}"] = ii.split(" ")

row_check = 2
col_check = 1
print(f"\nCurrent for Row {row_check} - Column {col_check}: {color.BOLD}{table_rows[f'Row_{row_check}'][col_check]}{color.END}")
ideal = (float(table_rows[f"Row_{row_check-1}"][col_check]) + float(table_rows[f"Row_{row_check+1}"][col_check]))/2
print(f"\t    Calculated Ideal: {color.BBLUE}{ideal}{color.END}\n")


Current for Row 2 - Column 1: -2.769346e-02
	    Calculated Ideal: -0.034180135



In [80]:
import re
import numpy as np
import pandas as pd

ansi_escape = re.compile(r'\x1B\[[0-?]*[ -/]*[@-~]')

def parse_fit_file(path, nrows=6, ncols=6):
    results = []
    current_q2y = None
    current_zpt = None
    parB = None
    parC = None
    chi2 = None
    header_re = re.compile(r"For SMEARED Q2-y Bin (\d+) - z-PT Bin (\d+):")

    with open(path, "r", encoding="utf-8") as f:
        for raw_line in f:
            line = ansi_escape.sub("", raw_line).strip()

            m = header_re.match(line)
            if(m):
                current_q2y = int(m.group(1))
                current_zpt = int(m.group(2))
                parB = None
                parC = None
                chi2 = None

            elif(line.startswith("Par B")):
                try:
                    val = line.split("=")[1].split("±")[0].strip()
                    parB = float(val)
                except Exception:
                    parB = None

            elif(line.startswith("Par C")):
                try:
                    val = line.split("=")[1].split("±")[0].strip()
                    parC = float(val)
                except Exception:
                    parC = None

            elif(line.startswith("chi2/NDF")):
                try:
                    val = line.split("=")[1].strip()
                    chi2 = float(val)
                except Exception:
                    chi2 = None

                if((current_q2y is not None) and (current_zpt is not None)):
                    results.append((current_zpt, parB, parC, chi2))
                parB = None
                parC = None
                chi2 = None

    df = pd.DataFrame(results, columns=["zPT", "ParB", "ParC", "Chi2"])
    if(not df.empty):
        df = df.set_index("zPT").sort_index()

    grid_B = np.full((nrows, ncols), np.nan)
    grid_C = np.full((nrows, ncols), np.nan)

    if(not df.empty):
        for zpt, row in df.iterrows():
            r = (zpt - 1) // ncols
            c = (zpt - 1) % ncols
            grid_B[r, c] = row["ParB"]
            grid_C[r, c] = row["ParC"]

    df_B = pd.DataFrame(grid_B, index=[f"Row {i+1}" for i in range(nrows)],
                        columns=[f"Col {j+1}" for j in range(ncols)])
    df_C = pd.DataFrame(grid_C, index=[f"Row {i+1}" for i in range(nrows)],
                        columns=[f"Col {j+1}" for j in range(ncols)])

    return df, df_B, df_C

# def neighbor_differences(grid):
#     nrows, ncols = grid.shape
#     diff_grid = np.full_like(grid, np.nan, dtype=float)
#     for r in range(nrows):
#         for c in range(ncols):
#             val = grid[r, c]
#             if(np.isnan(val)):
#                 continue
#             neighbors = []
#             # if(r > 0 and not np.isnan(grid[r-1, c])): neighbors.append(grid[r-1, c])
#             # if(r < nrows-1 and not np.isnan(grid[r+1, c])): neighbors.append(grid[r+1, c])
#             if(c > 0 and not np.isnan(grid[r, c-1])):
#                 neighbors.append(grid[r, c-1])
#             else:
#                 neighbors = [val]
#             if(c < ncols-1 and not np.isnan(grid[r, c+1])):
#                 neighbors.append(grid[r, c+1])
#             else:
#                 neighbors = [val]
#             if(len(neighbors) > 0):
#                 avg = np.mean(neighbors)
#                 diff_grid[r, c] = avg# val - avg
#     return pd.DataFrame(diff_grid,
#                         index=[f"Row{i+1}" for i in range(nrows)],
#                         columns=[f"Col{j+1}" for j in range(ncols)])

def neighbor_differences(grid):
    nrows, ncols = grid.shape
    diff_grid = np.full_like(grid, np.nan, dtype=float)

    for r in range(nrows):
        for c in range(ncols):
            val = grid[r, c]
            if(np.isnan(val)):
                continue

            neighbors = []
            if(c > 0 and not np.isnan(grid[r, c-1])):
                neighbors.append(grid[r, c-1])
            if(c < ncols-1 and not np.isnan(grid[r, c+1])):
                neighbors.append(grid[r, c+1])

            if(len(neighbors) == 2):
                avg = np.mean(neighbors)
                diff_grid[r, c] = avg# val - avg
            else:
                # no valid left/right neighbors → keep original value
                diff_grid[r, c] = val

    return pd.DataFrame(diff_grid,
                        index=[f"Row{i+1}" for i in range(nrows)],
                        columns=[f"Col{j+1}" for j in range(ncols)])



df, df_B, df_C = parse_fit_file(filename, nrows=6, ncols=6)



diff_B = neighbor_differences(df_B.values)
diff_C = neighbor_differences(df_C.values)

# a) Fixed number of decimals (example: 4 decimal places)
pd.set_option("display.precision", 5)
# # b) Significant figures (example: 5 sig figs)
# pd.options.display.float_format = "{:.5g}".format
# 2. Add spacing between columns (example: min 12 spaces per column)
pd.set_option("display.colheader_justify", "center")  # optional centering
pd.set_option("display.width", None)  # avoid line wrapping
# When you want to print, use to_string with col_space:
df_B = df_B.to_string(col_space=12)#, float_format="{:.4f}".format)
df_C = df_C.to_string(col_space=12)#, float_format="{:.4f}".format)
diff_B = diff_B.to_string(col_space=12)#, float_format="{:.4f}".format)
diff_C = diff_C.to_string(col_space=12)#, float_format="{:.4f}".format)


print(f"\n{color.BOLD}Grid of Par B:{color.END}\n{df_B}")
print(f"\n\n{color.BOLD}Par B - Neighbor Avg:{color.END}\n{diff_B}")
# print(f"\n{color.BOLD}Grid of Par C:{color.END}\n{df_C}")
# print(f"\n\n{color.BOLD}Par C - Neighbor Avg:{color.END}\n{diff_C}")




Grid of Par B:
                 Col 1        Col 2        Col 3        Col 4        Col 5        Col 6   
Row 1          -0.057666   -0.099021     -0.13605     -0.13069     -0.16207     -0.29912  
Row 2          -0.027693   -0.036273       -0.072    -0.098819     -0.16951     -0.27618  
Row 3          -0.010694   -0.059146    -0.054865    -0.071292     -0.13515     -0.32125  
Row 4         -0.0070583   -0.040592        -0.05    -0.091596     -0.19143          NaN  
Row 5        -6.6083e-09   -0.051372        -0.05     -0.12927     -0.19143          NaN  
Row 6        -8.1012e-10   -0.046543    -0.059977     -0.13814          NaN          NaN  


Par B - Neighbor Avg:
                 Col1         Col2         Col3         Col4         Col5         Col6    
Row1           -0.057666   -0.096859     -0.11486     -0.14906     -0.21491     -0.29912  
Row2           -0.027693   -0.049847    -0.067546     -0.12075      -0.1875     -0.27618  
Row3           -0.010694   -0.032779    -0.065219 

In [ ]:
#splitline{#splitline{Cos(#phi_{h}) Moments vs z}{CLAS12 Pass 2 #color[632]{Preliminary} - (Bayesian) Unfolded with RC}}{#splitline{2.4 < Q^{2} < 2.9}{0.65 < y < 0.75}}

In [82]:
ROOT.kRed

(EColor::kRed) : (unsigned int) 632

In [ ]:
// --- open file and fetch the only object (assumed TCanvas) ---
TFile *f = TFile::Open("Fit_Par_B_RC_Bayesian_Smear_Q2_y_Bin_5_MultiDim_z_pT_Bin_Y_bin_phi_t_VS_Z.root");

if (!f || f->IsZombie()) { Error("macro","Could not open file."); }
TKey *k = (TKey*)f->GetListOfKeys()->At(0);
if (!k) { Error("macro","No keys in file."); }
TCanvas *c = (TCanvas*)k->ReadObj();
if (!c || !c->InheritsFrom(TCanvas::Class())) { Error("macro","First object is not a TCanvas."); }
c->Draw();
c->cd();

// --- enter first sub-pad if one exists ---
TPad *p = nullptr;
{ TIter it(c->GetListOfPrimitives()); TObject *o;
  while ((o = it())) { if (o->InheritsFrom(TPad::Class())) { p = (TPad*)o; break; } }
}
if (p) p->cd();

gPad->Update(); // ensure frame/axes exist

// --- find the multigraph in the current pad ---
TMultiGraph *mg = nullptr;
{ TIter it(gPad->GetListOfPrimitives()); TObject *o;
  while ((o = it())) { if (o->InheritsFrom(TMultiGraph::Class())) { mg = (TMultiGraph*)o; break; } }
}
if (!mg) { Warning("macro","No TMultiGraph found in this pad."); }

// --- set Y range to [-0.45, 0.1] (prefer frame axis if available) ---
if (mg) {
  gPad->Update(); // make sure mg built its frame
  TH1 *frame = mg->GetHistogram(); // created after Draw/Update
  if (frame && frame->GetYaxis()) {
    frame->GetYaxis()->SetRangeUser(-0.45, 0.1);
  } else {
    mg->SetMinimum(-0.45);
    mg->SetMaximum(0.1);
  }
}

// --- set all TGraph* marker sizes inside the multigraph to 2.0 ---
if (mg && mg->GetListOfGraphs()) {
  TIter gIt(mg->GetListOfGraphs()); TObject *obj = nullptr;
  while ((obj = gIt())) {
    if (obj->InheritsFrom(TGraph::Class())) {
      ((TGraph*)obj)->SetMarkerSize(2.0);
    }
  }
}

// --- find a legend, adjust columns, and move to bottom 15% ---
TLegend *leg = nullptr;
{ TIter it2(gPad->GetListOfPrimitives()); TObject *o2;
  while ((o2 = it2())) { if (o2->InheritsFrom(TLegend::Class())) { leg = (TLegend*)o2; break; } }
}
if (leg) {
  leg->SetNColumns(3);
  leg->SetColumnSeparation(0.1); // optional
  leg->SetMargin(0.2);           // optional

  // --- reposition legend to occupy bottom 15% of pad (your chosen numbers) ---
  double x1 = 0.20;     // left margin fraction
  double x2 = 0.90;     // right margin fraction
  double y1 = 0.125;    // small gap from bottom
  double y2 = y1 + 0.1; // height (currently ~10%)

  leg->SetX1NDC(x1);
  leg->SetX2NDC(x2);
  leg->SetY1NDC(y1);
  leg->SetY2NDC(y2);
}

// --- set the title: remove any old TPaveText-based titles, then draw a centered-justified TLatex ---
{
  const char *newTitle =
    "#splitline{#splitline{Cos(#phi_{h}) Moments vs z}"
               "{#font[22]{CLAS12 Pass 2 #color[632]{Preliminary}} - (Bayesian) Unfolded with RC}}"
    "{#splitline{2.4 < Q^{2} < 2.9}{0.65 < y < 0.75}}";

  // Keep internal titles consistent for future redraws
  if (mg) mg->SetTitle(newTitle);
  TH1 *frame = (mg ? mg->GetHistogram() : nullptr);
  if (frame) frame->SetTitle(newTitle);

  // Remove any pre-existing "title" TPaveText boxes
  if (TPaveText *old = (TPaveText*)gPad->GetPrimitive("title")) {
    gPad->GetListOfPrimitives()->Remove(old);
    delete old;
  }
  // (Optional) sweep any other title-like TPaveText objects
  {
    TList *plist = gPad->GetListOfPrimitives();
    if (plist) {
      std::vector<TObject*> toDelete;
      TIter it3(plist); TObject *o3 = nullptr;
      while ((o3 = it3())) {
        if (o3->InheritsFrom(TPaveText::Class())) {
          TString nm = o3->GetName();
          if (nm.BeginsWith("title")) toDelete.push_back(o3);
        }
      }
      for (auto *o : toDelete) { plist->Remove(o); delete o; }
    }
  }

  // Draw the new title at the same position, but with centered justification
  double L = gPad->GetLeftMargin();
  double R = gPad->GetRightMargin();
  double T = gPad->GetTopMargin();
  double x = L + 0.5*(1.0 - L - R);
  double y = 1.0 - 0.5*T;  // keep your previous vertical position

  TLatex *lt = new TLatex();
  lt->SetNDC(true);
  lt->SetTextAlign(23);   // 2 = horizontal center, 3 = vertical top
  lt->SetTextFont(42);
  lt->SetTextSize(0.040);
  lt->DrawLatex(x, y, newTitle);
}


// --- redraw/update ---
gPad->Modified();
gPad->Update();


In [83]:
root_color.Bold

'#font[22]'

In [84]:
ROOT.kBlue

(EColor::kBlue) : (unsigned int) 600

In [ ]:
// --- open file and fetch the only object (assumed TCanvas) ---
TFile *f = TFile::Open("Fit_Par_B_RC_Bayesian_Smear_Q2_y_Bin_5_MultiDim_z_pT_Bin_Y_bin_phi_t_VS_Z.root");



if (!f || f->IsZombie()) { Error("macro","Could not open file."); }
TKey *k = (TKey*)f->GetListOfKeys()->At(0);
if (!k) { Error("macro","No keys in file."); }
TCanvas *c = (TCanvas*)k->ReadObj();
if (!c || !c->InheritsFrom(TCanvas::Class())) { Error("macro","First object is not a TCanvas."); }

// Turn off ROOT's automatic title drawing (prevents TPaveText titles from reappearing)
gStyle->SetOptTitle(0);

c->Draw();
c->cd();

// --- enter first sub-pad if one exists ---
TPad *p = nullptr;
{ TIter it(c->GetListOfPrimitives()); TObject *o;
  while ((o = it())) { if (o->InheritsFrom(TPad::Class())) { p = (TPad*)o; break; } }
}
if (p) p->cd();

gPad->Update(); // ensure frame/axes exist

// --- find the multigraph in the current pad ---
TMultiGraph *mg = nullptr;
{ TIter it(gPad->GetListOfPrimitives()); TObject *o;
  while ((o = it())) { if (o->InheritsFrom(TMultiGraph::Class())) { mg = (TMultiGraph*)o; break; } }
}
if (!mg) { Warning("macro","No TMultiGraph found in this pad."); }

// --- set Y range to [-0.45, 0.1] (prefer frame axis if available) ---
if (mg) {
  gPad->Update(); // make sure mg built its frame
  TH1 *frame = mg->GetHistogram(); // created after Draw/Update
  if (frame && frame->GetYaxis()) {
    frame->GetYaxis()->SetRangeUser(-0.45, 0.1);
  } else {
    mg->SetMinimum(-0.45);
    mg->SetMaximum(0.1);
  }
}



// --- set all TGraph* marker sizes inside the multigraph to 2.0 ---
if (mg && mg->GetListOfGraphs()) {
  TIter gIt(mg->GetListOfGraphs()); TObject *obj = nullptr;
  while ((obj = gIt())) {
    if (obj->InheritsFrom(TGraph::Class())) {
      ((TGraph*)obj)->SetMarkerSize(2.0);
    }
  }
}

// --- find a legend, adjust columns, and move to bottom 15% ---
TLegend *leg = nullptr;
{ TIter it2(gPad->GetListOfPrimitives()); TObject *o2;
  while ((o2 = it2())) { if (o2->InheritsFrom(TLegend::Class())) { leg = (TLegend*)o2; break; } }
}
if (leg) {
  leg->SetNColumns(3);
  leg->SetColumnSeparation(0.1); // optional
  leg->SetMargin(0.2);           // optional

  // --- reposition legend to occupy bottom ~15% of the pad ---
  double x1 = 0.20;     // left margin fraction
  double x2 = 0.90;     // right margin fraction
  double y1 = 0.125;    // small gap from bottom
  double y2 = y1 + 0.1;

  leg->SetX1NDC(x1);
  leg->SetX2NDC(x2);
  leg->SetY1NDC(y1);
  leg->SetY2NDC(y2);
}

// --- remove any pre-existing TPaveText title(s) so we don't get duplicates ---
{
  // common one named exactly "title"
  if (TPaveText *old = (TPaveText*)gPad->GetPrimitive("title")) {
    gPad->GetListOfPrimitives()->Remove(old);
    delete old;
  }
  // also sweep any TPaveText whose name starts with "title"
  if (TList *plist = gPad->GetListOfPrimitives()) {
    std::vector<TObject*> toDelete;
    TIter it3(plist); TObject *o3 = nullptr;
    while ((o3 = it3())) {
      if (o3->InheritsFrom(TPaveText::Class())) {
        TString nm = o3->GetName();
        if (nm.BeginsWith("title")) toDelete.push_back(o3);
      }
    }
    for (auto *o : toDelete) { plist->Remove(o); delete o; }
  }
}

// --- draw the title as 4 centered-justified TLatex lines at the same position ---
{
  // Position: keep your original "top of pad" location
  double Lm = gPad->GetLeftMargin();
  double Rm = gPad->GetRightMargin();
  double Tm = gPad->GetTopMargin();

  const double x = Lm + 0.5*(1.0 - Lm - Rm); // centered in drawable width
  double y = 1.0 - 0.5*Tm;                    // original vertical anchor (near top)

  // Text sizes and spacing (NDC units)
  const double szMain = 0.040;   // main and mid lines
  const double szSub  = 0.036;   // bottom two lines slightly smaller
  const double dyMain = 1.15*szMain; // line spacing factors
  const double dySub  = 1.10*szSub;

  // Create/draw centered lines (SetTextAlign(23) => centered horizontally, top-aligned vertically)
  auto drawLine = [&](const char* name, double yline, double size, const char* txt) {
    if (TObject *prev = gPad->GetPrimitive(name)) { gPad->GetListOfPrimitives()->Remove(prev); delete prev; }
    TLatex *lt = new TLatex();
    lt->SetName(name);
    lt->SetNDC(true);
    lt->SetTextAlign(23);
    lt->SetTextFont(42);
    lt->SetTextSize(size);
    lt->DrawLatex(x, yline, txt);
  };

  // Lines (equivalent to your nested #splitline, but with explicit control + centered justification)
  drawLine("user_title_l1", y,           szMain, "#font[22]{#scale[1.5]{Cos(#phi_{h}) Moments vs z}}");
  y -= 1.5*dyMain;
  drawLine("user_title_l2", y,           szMain, "#font[22]{CLAS12 Pass 2 #color[632]{Preliminary}} - #color[872]{(Bayesian) Unfolded with RC}");
  y -= dyMain;
  drawLine("user_title_l3", y,           szSub,  "#font[22]{2.4 < Q^{2} < 2.9}");
  y -= dySub;
  drawLine("user_title_l4", y,           szSub,  "#font[22]{0.65 < y < 0.75}");
}

List_of_Files.append(["Par_B", "Q2_y_Bin_5", "VS_Z", -0.45, 0.1, "2.4 < Q^{2} < 2.9", "0.65 < y < 0.75"])

// --- final redraw/update ---
gPad->Modified();
gPad->Update();


In [85]:
ROOT.kViolet -8

872

In [58]:
# # ROOT_COMMANDS = """
# # // --- open file and fetch the only object (assumed TCanvas) ---
# # TFile *f = TFile::Open("FILE_NAME_INPUT.root");
# # if (!f || f->IsZombie()) { Error("macro","Could not open file."); }
# # TKey *k = (TKey*)f->GetListOfKeys()->At(0);
# # if (!k) { Error("macro","No keys in file."); }
# # TCanvas *c = (TCanvas*)k->ReadObj();
# # if (!c || !c->InheritsFrom(TCanvas::Class())) { Error("macro","First object is not a TCanvas."); }
# # // Turn off ROOT's automatic title drawing (prevents TPaveText titles from reappearing)
# # gStyle->SetOptTitle(0);
# # c->Draw();
# # c->cd();
# # // --- enter first sub-pad if one exists ---
# # TPad *p = nullptr;
# # { TIter it(c->GetListOfPrimitives()); TObject *o;
# #   while ((o = it())) { if (o->InheritsFrom(TPad::Class())) { p = (TPad*)o; break; } }
# # }
# # if (p) p->cd();
# # gPad->Update(); // ensure frame/axes exist
# # // --- find the multigraph in the current pad ---
# # TMultiGraph *mg = nullptr;
# # { TIter it(gPad->GetListOfPrimitives()); TObject *o;
# #   while ((o = it())) { if (o->InheritsFrom(TMultiGraph::Class())) { mg = (TMultiGraph*)o; break; } }
# # }
# # if (!mg) { Warning("macro","No TMultiGraph found in this pad."); }
# # if (mg) {
# #   gPad->Update(); // make sure mg built its frame
# #   TH1 *frame = mg->GetHistogram(); // created after Draw/Update
# #   if (frame && frame->GetYaxis()) {
# #     frame->GetYaxis()->SetRangeUser(MIN_RANGE, MAX_RANGE);
# #   } else {
# #     mg->SetMinimum(MIN_RANGE);
# #     mg->SetMaximum(MAX_RANGE);
# #   }
# # }
# # // --- set all TGraph* marker sizes inside the multigraph to 2.0 ---
# # if (mg && mg->GetListOfGraphs()) {
# #   TIter gIt(mg->GetListOfGraphs()); TObject *obj = nullptr;
# #   while ((obj = gIt())) {
# #     if (obj->InheritsFrom(TGraph::Class())) {
# #       ((TGraph*)obj)->SetMarkerSize(2.0);
# #     }
# #   }
# # }
# # // --- find a legend, adjust columns, and move to bottom 15% ---
# # TLegend *leg = nullptr;
# # { TIter it2(gPad->GetListOfPrimitives()); TObject *o2;
# #   while ((o2 = it2())) { if (o2->InheritsFrom(TLegend::Class())) { leg = (TLegend*)o2; break; } }
# # }
# # if (leg) {
# #   leg->SetNColumns(3);
# #   leg->SetColumnSeparation(0.1); // optional
# #   leg->SetMargin(0.2);           // optional
# #   // --- reposition legend to occupy bottom ~15% of the pad ---
# #   double x1 = 0.20;     // left margin fraction
# #   double x2 = 0.90;     // right margin fraction
# #   double y1 = 0.125;    // small gap from bottom
# #   double y2 = y1 + 0.1;
# #   leg->SetX1NDC(x1);
# #   leg->SetX2NDC(x2);
# #   leg->SetY1NDC(y1);
# #   leg->SetY2NDC(y2);
# # }
# # // --- remove any pre-existing TPaveText title(s) so we don't get duplicates ---
# # {
# #   // common one named exactly "title"
# #   if (TPaveText *old = (TPaveText*)gPad->GetPrimitive("title")) {
# #     gPad->GetListOfPrimitives()->Remove(old);
# #     delete old;
# #   }
# #   // also sweep any TPaveText whose name starts with "title"
# #   if (TList *plist = gPad->GetListOfPrimitives()) {
# #     std::vector<TObject*> toDelete;
# #     TIter it3(plist); TObject *o3 = nullptr;
# #     while ((o3 = it3())) {
# #       if (o3->InheritsFrom(TPaveText::Class())) {
# #         TString nm = o3->GetName();
# #         if (nm.BeginsWith("title")) toDelete.push_back(o3);
# #       }
# #     }
# #     for (auto *o : toDelete) { plist->Remove(o); delete o; }
# #   }
# # }
# # // --- draw the title as 4 centered-justified TLatex lines at the same position ---
# # {
# #   // Position: keep your original "top of pad" location
# #   double Lm = gPad->GetLeftMargin();
# #   double Rm = gPad->GetRightMargin();
# #   double Tm = gPad->GetTopMargin();
# #   const double x = Lm + 0.5*(1.0 - Lm - Rm); // centered in drawable width
# #   double y = 1.0 - 0.5*Tm;                    // original vertical anchor (near top)
# #   // Text sizes and spacing (NDC units)
# #   const double szMain = 0.040;   // main and mid lines
# #   const double szSub  = 0.036;   // bottom two lines slightly smaller
# #   const double dyMain = 1.15*szMain; // line spacing factors
# #   const double dySub  = 1.10*szSub;
# #   // Create/draw centered lines (SetTextAlign(23) => centered horizontally, top-aligned vertically)
# #   auto drawLine = [&](const char* name, double yline, double size, const char* txt) {
# #     if (TObject *prev = gPad->GetPrimitive(name)) { gPad->GetListOfPrimitives()->Remove(prev); delete prev; }
# #     TLatex *lt = new TLatex();
# #     lt->SetName(name);
# #     lt->SetNDC(true);
# #     lt->SetTextAlign(23);
# #     lt->SetTextFont(42);
# #     lt->SetTextSize(size);
# #     lt->DrawLatex(x, yline, txt);
# #   };
# #   // Lines (equivalent to your nested #splitline, but with explicit control + centered justification)
# #   drawLine("user_title_l1", y,           szMain, "#font[22]{#scale[1.5]{MOMEMENT_TITLE}}");
# #   y -= 1.5*dyMain;
# #   drawLine("user_title_l2", y,           szMain, "#font[22]{CLAS12 Pass 2 #color[632]{Preliminary}} - METHOD_TITLE");
# #   y -= dyMain;
# #   drawLine("user_title_l3", y,           szSub,  "#font[22]{Q2_RANGE}");
# #   y -= dySub;
# #   drawLine("user_title_l4", y,           szSub,  "#font[22]{Y_RANGE}");
# # }
# # // --- final redraw/update ---
# # gPad->Modified();
# # gPad->Update();
# # """

# ROOT_COMMANDS = """// --- open file and fetch the only object (assumed TCanvas) ---
# TFile *f = TFile::Open("FILE_NAME_INPUT.root");

# if (!f || f->IsZombie()) { Error("macro","Could not open file."); }
# TKey *k = (TKey*)f->GetListOfKeys()->At(0);
# if (!k) { Error("macro","No keys in file."); }
# TCanvas *c = (TCanvas*)k->ReadObj();
# if (!c || !c->InheritsFrom(TCanvas::Class())) { Error("macro","First object is not a TCanvas."); }

# // Turn off ROOT's automatic title drawing (prevents TPaveText titles from reappearing)
# gStyle->SetOptTitle(0);

# c->Draw();
# c->cd();

# // --- enter first sub-pad if one exists ---
# TPad *p = nullptr;
# { TIter it(c->GetListOfPrimitives()); TObject *o;
#   while ((o = it())) { if (o->InheritsFrom(TPad::Class())) { p = (TPad*)o; break; } }
# }
# if (p) p->cd();

# gPad->Update(); // ensure frame/axes exist

# // --- find the multigraph in the current pad ---
# TMultiGraph *mg = nullptr;
# { TIter it(gPad->GetListOfPrimitives()); TObject *o;
#   while ((o = it())) { if (o->InheritsFrom(TMultiGraph::Class())) { mg = (TMultiGraph*)o; break; } }
# }
# if (!mg) { Warning("macro","No TMultiGraph found in this pad."); }

# if (mg) {
#   gPad->Update(); // make sure mg built its frame
#   TH1 *frame = mg->GetHistogram(); // created after Draw/Update
#   if (frame && frame->GetYaxis()) {
#     frame->GetYaxis()->SetRangeUser(MIN_RANGE, MAX_RANGE);
#   } else {
#     mg->SetMinimum(MIN_RANGE);
#     mg->SetMaximum(MAX_RANGE);
#   }
# }

# // --- set all TGraph* marker sizes inside the multigraph to 2.5 ---
# if (mg && mg->GetListOfGraphs()) {
#   TIter gIt(mg->GetListOfGraphs()); TObject *obj = nullptr;
#   while ((obj = gIt())) {
#     if (obj->InheritsFrom(TGraph::Class())) {
#       // ((TGraph*)obj)->SetMarkerSize(2.0);
#       ((TGraph*)obj)->SetMarkerSize(2.5);
#     }
#   }
# }

# // --- find a legend, adjust columns, and move to bottom 15% ---
# TLegend *leg = nullptr;
# { TIter it2(gPad->GetListOfPrimitives()); TObject *o2;
#   while ((o2 = it2())) { if (o2->InheritsFrom(TLegend::Class())) { leg = (TLegend*)o2; break; } }
# }
# if (leg) {
#   leg->SetNColumns(3);
#   leg->SetColumnSeparation(0.1); // optional
#   leg->SetMargin(0.2);           // optional

#   // --- reposition legend to occupy bottom ~15% of the pad ---
#   double x1 = 0.20;     // left margin fraction
#   double x2 = 0.90;     // right margin fraction
#   double y1 = 0.125;    // small gap from bottom
#   double y2 = y1 + TLEGEND_HEIGHT;

#   leg->SetX1NDC(x1);
#   leg->SetX2NDC(x2);
#   leg->SetY1NDC(y1);
#   leg->SetY2NDC(y2);
# }

# // --- remove any pre-existing TPaveText title(s) so we don't get duplicates ---
# {
#   // common one named exactly "title"
#   if (TPaveText *old = (TPaveText*)gPad->GetPrimitive("title")) {
#     gPad->GetListOfPrimitives()->Remove(old);
#     delete old;
#   }
#   // also sweep any TPaveText whose name starts with "title"
#   if (TList *plist = gPad->GetListOfPrimitives()) {
#     std::vector<TObject*> toDelete;
#     TIter it3(plist); TObject *o3 = nullptr;
#     while ((o3 = it3())) {
#       if (o3->InheritsFrom(TPaveText::Class())) {
#         TString nm = o3->GetName();
#         if (nm.BeginsWith("title")) toDelete.push_back(o3);
#       }
#     }
#     for (auto *o : toDelete) { plist->Remove(o); delete o; }
#   }
# }

# // --- draw the title as 4 centered-justified TLatex lines at the same position ---
# {
#   // Position: keep your original "top of pad" location
#   double Lm = gPad->GetLeftMargin();
#   double Rm = gPad->GetRightMargin();
#   double Tm = gPad->GetTopMargin();

#   const double x = Lm + 0.5*(1.0 - Lm - Rm); // centered in drawable width
#   double y = 1.0 - 0.5*Tm;                    // original vertical anchor (near top)

#   // Text sizes and spacing (NDC units)
#   const double szMain = 0.040;   // main and mid lines
#   const double szSub  = 0.036;   // bottom two lines slightly smaller
#   const double dyMain = 1.15*szMain; // line spacing factors
#   const double dySub  = 1.10*szSub;

#   // Create/draw centered lines (SetTextAlign(23) => centered horizontally, top-aligned vertically)
#   auto drawLine = [&](const char* name, double yline, double size, const char* txt) {
#     if (TObject *prev = gPad->GetPrimitive(name)) { gPad->GetListOfPrimitives()->Remove(prev); delete prev; }
#     TLatex *lt = new TLatex();
#     lt->SetName(name);
#     lt->SetNDC(true);
#     lt->SetTextAlign(23);
#     lt->SetTextFont(42);
#     lt->SetTextSize(size);
#     lt->DrawLatex(x, yline, txt);
#   };

#   // Lines (equivalent to your nested #splitline, but with explicit control + centered justification)
#   // drawLine("user_title_l1", y,           szMain, "#font[22]{#scale[1.5]{MOMEMENT_TITLE}}");
#   // y -= 1.5*dyMain;
#   drawLine("user_title_l2", y,           szMain, "#font[22]{CLAS12 Pass 2 #color[632]{Preliminary}} - METHOD_TITLE");
#   y -= dyMain;
#   drawLine("user_title_l3", y,           szSub,  "#font[22]{#scale[1.25]{Q2_RANGE}}");
#   y -= 1.25*dySub;
#   drawLine("user_title_l4", y,           szSub,  "#font[22]{#scale[1.25]{Y_RANGE}}");

#   if (mg) {
#     gPad->Update();
#     TH1 *frame = mg->GetHistogram();
#     if (frame && frame->GetXaxis()) {
#       TString xtitle = frame->GetXaxis()->GetTitle();
#       xtitle.ReplaceAll("(Smeared) ", ""); // strip only that part
#       frame->GetXaxis()->SetTitle(xtitle);
#     }
#   }
# }

# // --- final redraw/update ---
# gPad->Modified();
# gPad->Update();
# """

# if(True):
#     print("\n\x1b[91m\x1b[1m// SAVING TCANVASES...\x1b[0m")
#     ROOT_COMMANDS = f"""{ROOT_COMMANDS}
# // Save results:
# c->SaveAs("FILE_NAME_INPUT.pdf");
# // TFile out("FILE_NAME_INPUT.root", "RECREATE"); c->Write(); out.Close();"""

# TLEGEND_HEIGHT = 0.1
# ROOT_COMMANDS = ROOT_COMMANDS.replace("TLEGEND_HEIGHT", TLEGEND_HEIGHT)

# List_of_Files = []

# # # List_of_Files.append(["Par_B_RC", "Q2_y_Bin_5",  "VS_Z", -0.425, 0.05, "2.4 < Q^{2} < 2.9", "0.65 < y < 0.75"])
# # # List_of_Files.append(["Par_C_RC", "Q2_y_Bin_5",  "VS_Z", -0.125, 0.15, "2.4 < Q^{2} < 2.9", "0.65 < y < 0.75"])
# # # List_of_Files.append(["Par_B", "Q2_y_Bin_5",  "VS_Z", -0.45, 0.1, "2.4 < Q^{2} < 2.9", "0.65 < y < 0.75"])
# # # List_of_Files.append(["Par_C", "Q2_y_Bin_5",  "VS_Z", -0.15, 0.2, "2.4 < Q^{2} < 2.9", "0.65 < y < 0.75"])
# # # List_of_Files.append(["Par_B_RC", "Q2_y_Bin_14", "VS_Z", -0.45, 0.1, "3.7 < Q^{2} < 5.3", "0.55 < y < 0.65"])
# # # List_of_Files.append(["Par_C_RC", "Q2_y_Bin_14", "VS_Z", -0.15, 0.2, "3.7 < Q^{2} < 5.3", "0.55 < y < 0.65"])
# # # List_of_Files.append(["Par_B_RC", "Q2_y_Bin_5",  "VS_PT", -0.45, 0.1, "2.4 < Q^{2} < 2.9", "0.65 < y < 0.75"])
# # # List_of_Files.append(["Par_C_RC", "Q2_y_Bin_5",  "VS_PT", -0.15, 0.2, "2.4 < Q^{2} < 2.9", "0.65 < y < 0.75"])
# # # List_of_Files.append(["Par_B_RC", "Q2_y_Bin_14", "VS_PT", -0.45, 0.1, "3.7 < Q^{2} < 5.3", "0.55 < y < 0.65"])
# # # List_of_Files.append(["Par_C_RC", "Q2_y_Bin_14", "VS_PT", -0.15, 0.2, "3.7 < Q^{2} < 5.3", "0.55 < y < 0.65"])


# # List_of_Files.append(["Par_B_RC", "Q2_y_Bin_5",  "VS_Z", -0.575, 0.05, "2.4 < Q^{2} < 2.9", "0.65 < y < 0.75"])
# # List_of_Files.append(["Par_B",    "Q2_y_Bin_5",  "VS_Z", -0.575, 0.05, "2.4 < Q^{2} < 2.9", "0.65 < y < 0.75"])
# # List_of_Files.append(["Par_C_RC", "Q2_y_Bin_5",  "VS_Z", -0.1,   0.2,  "2.4 < Q^{2} < 2.9", "0.65 < y < 0.75"])
# # List_of_Files.append(["Par_C",    "Q2_y_Bin_5",  "VS_Z", -0.1,   0.2,  "2.4 < Q^{2} < 2.9", "0.65 < y < 0.75"])

# # List_of_Files.append(["Par_B_RC", "Q2_y_Bin_14", "VS_Z", -0.575, 0.075, "3.7 < Q^{2} < 5.3", "0.55 < y < 0.65"])
# # List_of_Files.append(["Par_B",    "Q2_y_Bin_14", "VS_Z", -0.575, 0.075, "3.7 < Q^{2} < 5.3", "0.55 < y < 0.65"])
# # List_of_Files.append(["Par_C_RC", "Q2_y_Bin_14", "VS_Z", -0.1,   0.15,  "3.7 < Q^{2} < 5.3", "0.55 < y < 0.65"])
# # List_of_Files.append(["Par_C",    "Q2_y_Bin_14", "VS_Z", -0.1,   0.15,  "3.7 < Q^{2} < 5.3", "0.55 < y < 0.65"])

# # List_of_Files.append(["Par_B_RC", "Q2_y_Bin_5",  "VS_PT", -0.575, 0.05, "2.4 < Q^{2} < 2.9", "0.65 < y < 0.75"])
# # List_of_Files.append(["Par_B",    "Q2_y_Bin_5",  "VS_PT", -0.575, 0.05, "2.4 < Q^{2} < 2.9", "0.65 < y < 0.75"])
# # List_of_Files.append(["Par_C_RC", "Q2_y_Bin_5",  "VS_PT", -0.1,   0.2,  "2.4 < Q^{2} < 2.9", "0.65 < y < 0.75"])
# # List_of_Files.append(["Par_C",    "Q2_y_Bin_5",  "VS_PT", -0.1,   0.2,  "2.4 < Q^{2} < 2.9", "0.65 < y < 0.75"])

# # List_of_Files.append(["Par_B_RC", "Q2_y_Bin_14", "VS_PT", -0.575, 0.075, "3.7 < Q^{2} < 5.3", "0.55 < y < 0.65"])
# # List_of_Files.append(["Par_B",    "Q2_y_Bin_14", "VS_PT", -0.575, 0.075, "3.7 < Q^{2} < 5.3", "0.55 < y < 0.65"])
# List_of_Files.append(["Par_C_RC", "Q2_y_Bin_14", "VS_PT", -0.1,   0.15,  "3.7 < Q^{2} < 5.3", "0.55 < y < 0.65"])
# List_of_Files.append(["Par_C",    "Q2_y_Bin_14", "VS_PT", -0.1,   0.15,  "3.7 < Q^{2} < 5.3", "0.55 < y < 0.65"])

# output_text = ""
# for par, Bin, var, yMin, yMax, q2_range, y_range in List_of_Files:
#     title_method = "#color[872]{(Bayesian) Unfolded with RC}" if("RC" in par) else "#color[30]{(Bayesian) Unfolded}"
#     title_method = "#color[872]{Unfolded with RC}" if("RC" in par) else "#color[30]{Unfolded without RC}"
#     # title_moment = "Cos(#phi_{h}) Moments " if("Par_B" in par) else "Cos(2#phi_{h}) Moments "
#     # title_moment = f"{title_moment}{'vs z' if('Z' in var) else 'vs P_{T}'}"
#     file_name_in = f"Fit_{par}_Bayesian_Smear_{Bin}_MultiDim_z_pT_Bin_Y_bin_phi_t_{var}"
#     output_text  = f"{output_text}\n{ROOT_COMMANDS}"
#     output_text  = output_text.replace("Q2_RANGE",  q2_range)
#     output_text  = output_text.replace("Y_RANGE",    y_range)
#     output_text  = output_text.replace("MIN_RANGE", str(yMin))
#     output_text  = output_text.replace("MAX_RANGE", str(yMax))
#     output_text  = output_text.replace("METHOD_TITLE",    title_method)
#     # output_text  = output_text.replace("MOMEMENT_TITLE",  title_moment)
#     output_text  = output_text.replace("FILE_NAME_INPUT", file_name_in)
    
# print(f"{output_text}\n\n")


# The Cell Below Can Loop through any bin
The one above is an older version

In [63]:
# start = 3

In [79]:
Save_Q = True
TLEGEND_HEIGHT = 0.1

Unfold_Q = not True # Use Bayesian unfolding if 'True', else show the RC Factors
Unfold_Q = True
# Q2_y_Bin_Set_List = [4]
Q2_y_Bin_Set_List = range(1, 18)


# # Q2_y_Bin_Set_List = range(1, 3)
# Q2_y_Bin_Set_List = range(start, start+3)
# start += 3


List_of_Files = []
ROOT_COMMANDS = """// --- open file and fetch the only object (assumed TCanvas) ---
TFile *f = TFile::Open("FILE_NAME_INPUT.root");

if (!f || f->IsZombie()) { Error("macro","Could not open file."); }
TKey *k = (TKey*)f->GetListOfKeys()->At(0);
if (!k) { Error("macro","No keys in file."); }
TCanvas *c = (TCanvas*)k->ReadObj();
if (!c || !c->InheritsFrom(TCanvas::Class())) { Error("macro","First object is not a TCanvas."); }

// Turn off ROOT's automatic title drawing (prevents TPaveText titles from reappearing)
gStyle->SetOptTitle(0);

// c->Draw();
c->cd();

// --- enter first sub-pad if one exists ---
TPad *p = nullptr;
{ TIter it(c->GetListOfPrimitives()); TObject *o;
  while ((o = it())) { if (o->InheritsFrom(TPad::Class())) { p = (TPad*)o; break; } }
}
if (p) p->cd();

gPad->Update(); // ensure frame/axes exist

// --- find the multigraph in the current pad ---
TMultiGraph *mg = nullptr;
{ TIter it(gPad->GetListOfPrimitives()); TObject *o;
  while ((o = it())) { if (o->InheritsFrom(TMultiGraph::Class())) { mg = (TMultiGraph*)o; break; } }
}
if (!mg) { Warning("macro","No TMultiGraph found in this pad."); }

if (mg) {
  gPad->Update(); // make sure mg built its frame
  TH1 *frame = mg->GetHistogram(); // created after Draw/Update
  if (frame && frame->GetYaxis()) {
    frame->GetYaxis()->SetRangeUser(MIN_RANGE, MAX_RANGE);
  } else {
    mg->SetMinimum(MIN_RANGE);
    mg->SetMaximum(MAX_RANGE);
  }
}

// --- set all TGraph* marker sizes inside the multigraph to 2.5 ---
if (mg && mg->GetListOfGraphs()) {
  TIter gIt(mg->GetListOfGraphs()); TObject *obj = nullptr;
  while ((obj = gIt())) {
    if (obj->InheritsFrom(TGraph::Class())) {
      // ((TGraph*)obj)->SetMarkerSize(2.0);
      ((TGraph*)obj)->SetMarkerSize(2.5);
    }
  }
}

// --- find a legend, adjust columns, and move to bottom 15% ---
TLegend *leg = nullptr;
{ TIter it2(gPad->GetListOfPrimitives()); TObject *o2;
  while ((o2 = it2())) { if (o2->InheritsFrom(TLegend::Class())) { leg = (TLegend*)o2; break; } }
}
if (leg) {
  leg->SetNColumns(3);
  leg->SetColumnSeparation(0.1); // optional
  leg->SetMargin(0.2);           // optional

  // --- reposition legend to occupy bottom ~15% of the pad ---
  double x1 = 0.20;     // left margin fraction
  double x2 = 0.90;     // right margin fraction
  double y1 = 0.125;    // small gap from bottom
  double y2 = y1 + TLEGEND_HEIGHT;

  leg->SetX1NDC(x1);
  leg->SetX2NDC(x2);
  leg->SetY1NDC(y1);
  leg->SetY2NDC(y2);
}

// --- remove any pre-existing TPaveText title(s) so we don't get duplicates ---
{
  // common one named exactly "title"
  if (TPaveText *old = (TPaveText*)gPad->GetPrimitive("title")) {
    gPad->GetListOfPrimitives()->Remove(old);
    delete old;
  }
  // also sweep any TPaveText whose name starts with "title"
  if (TList *plist = gPad->GetListOfPrimitives()) {
    std::vector<TObject*> toDelete;
    TIter it3(plist); TObject *o3 = nullptr;
    while ((o3 = it3())) {
      if (o3->InheritsFrom(TPaveText::Class())) {
        TString nm = o3->GetName();
        if (nm.BeginsWith("title")) toDelete.push_back(o3);
      }
    }
    for (auto *o : toDelete) { plist->Remove(o); delete o; }
  }
}

// --- draw the title as 4 centered-justified TLatex lines at the same position ---
{
  // Position: keep your original "top of pad" location
  double Lm = gPad->GetLeftMargin();
  double Rm = gPad->GetRightMargin();
  double Tm = gPad->GetTopMargin();

  const double x = Lm + 0.5*(1.0 - Lm - Rm); // centered in drawable width
  double y = 1.0 - 0.5*Tm;                    // original vertical anchor (near top)

  // Text sizes and spacing (NDC units)
  const double szMain = 0.040;   // main and mid lines
  const double szSub  = 0.036;   // bottom two lines slightly smaller
  const double dyMain = 1.15*szMain; // line spacing factors
  const double dySub  = 1.10*szSub;

  // Create/draw centered lines (SetTextAlign(23) => centered horizontally, top-aligned vertically)
  auto drawLine = [&](const char* name, double yline, double size, const char* txt) {
    if (TObject *prev = gPad->GetPrimitive(name)) { gPad->GetListOfPrimitives()->Remove(prev); delete prev; }
    TLatex *lt = new TLatex();
    lt->SetName(name);
    lt->SetNDC(true);
    lt->SetTextAlign(23);
    lt->SetTextFont(42);
    lt->SetTextSize(size);
    lt->DrawLatex(x, yline, txt);
  };

  // Lines (equivalent to your nested #splitline, but with explicit control + centered justification)
  // drawLine("user_title_l1", y,           szMain, "#font[22]{#scale[1.5]{MOMEMENT_TITLE}}");
  // y -= 1.5*dyMain;
  drawLine("user_title_l2", y,           szMain, "METHOD_TITLE");
  y -= dyMain;
  drawLine("user_title_l3", y,           szSub,  "#font[22]{#scale[1.25]{Q2_RANGE}}");
  y -= 1.25*dySub;
  drawLine("user_title_l4", y,           szSub,  "#font[22]{#scale[1.25]{Y_RANGE}}");

  if (mg) {
    gPad->Update();
    TH1 *frame = mg->GetHistogram();
    if (frame && frame->GetXaxis()) {
      TString xtitle = frame->GetXaxis()->GetTitle();
      xtitle.ReplaceAll("(Smeared) ", ""); // strip only that part
      frame->GetXaxis()->SetTitle(xtitle);
    }
  }
}

// --- final redraw/update ---
gPad->Modified();
gPad->Update();
"""

if(Save_Q):
    print("\n\x1b[91m\x1b[1m// SAVING TCANVASES...\x1b[0m")
    ROOT_COMMANDS = f"""{ROOT_COMMANDS}
// Save results:
c->SaveAs("FILE_NAME_INPUT.pdf");
// TFile out("FILE_NAME_INPUT.root", "RECREATE"); c->Write(); out.Close();"""

ROOT_COMMANDS = ROOT_COMMANDS.replace("TLEGEND_HEIGHT", str(TLEGEND_HEIGHT))


Q2_Bin_Ranges = { "1_Q2": "2.00 < Q^{2} < 2.40",
        		   "1_y":  "0.65 < y < 0.75", 
        		  "2_Q2": "2.00 < Q^{2} < 2.40",
        		   "2_y":  "0.55 < y < 0.65", 
        		  "3_Q2": "2.00 < Q^{2} < 2.40",
        		   "3_y":  "0.45 < y < 0.55", 
        		  "4_Q2": "2.00 < Q^{2} < 2.40",
        		   "4_y":  "0.35 < y < 0.45", 
        		  "5_Q2": "2.40 < Q^{2} < 2.90",
        		   "5_y":  "0.65 < y < 0.75", 
        		  "6_Q2": "2.40 < Q^{2} < 2.90",
        		   "6_y":  "0.55 < y < 0.65", 
        		  "7_Q2": "2.40 < Q^{2} < 2.90",
        		   "7_y":  "0.45 < y < 0.55", 
        		  "8_Q2": "2.40 < Q^{2} < 2.90",
        		   "8_y":  "0.35 < y < 0.45", 
        		  "9_Q2": "2.90 < Q^{2} < 3.70",
        		   "9_y":  "0.65 < y < 0.75", 
        		 "10_Q2": "2.90 < Q^{2} < 3.70",
        		  "10_y":  "0.55 < y < 0.65", 
        		 "11_Q2": "2.90 < Q^{2} < 3.70",
        		  "11_y":  "0.45 < y < 0.55", 
        		 "12_Q2": "2.90 < Q^{2} < 3.70",
        		  "12_y":  "0.35 < y < 0.45", 
        		 "13_Q2": "3.70 < Q^{2} < 5.30",
        		  "13_y":  "0.65 < y < 0.75", 
        		 "14_Q2": "3.70 < Q^{2} < 5.30",
        		  "14_y":  "0.55 < y < 0.65", 
        		 "15_Q2": "3.70 < Q^{2} < 5.30",
        		  "15_y":  "0.45 < y < 0.55", 
        		 "16_Q2": "5.30 < Q^{2} < 7.90",
        		  "16_y":  "0.65 < y < 0.75", 
        		 "17_Q2": "5.30 < Q^{2} < 7.90",
        		  "17_y":  "0.55 < y < 0.65", 
        		 }

for Q2_y_Bin_Set in Q2_y_Bin_Set_List:
    if(Unfold_Q):
        # List_of_Files.append(["Par_B_RC", "Q2_y_Bin_5",  "VS_PT", -0.575, 0.05, "2.4 < Q^{2} < 2.9", "0.65 < y < 0.75"])
        # List_of_Files.append(["Par_B",    "Q2_y_Bin_5",  "VS_PT", -0.575, 0.05, "2.4 < Q^{2} < 2.9", "0.65 < y < 0.75"])
        # List_of_Files.append(["Par_C_RC", "Q2_y_Bin_5",  "VS_PT", -0.1,   0.2,  "2.4 < Q^{2} < 2.9", "0.65 < y < 0.75"])
        # List_of_Files.append(["Par_C",    "Q2_y_Bin_5",  "VS_PT", -0.1,   0.2,  "2.4 < Q^{2} < 2.9", "0.65 < y < 0.75"])
        
        # List_of_Files.append(["Par_B_RC", "Q2_y_Bin_14", "VS_PT", -0.575, 0.075, "3.7 < Q^{2} < 5.3", "0.55 < y < 0.65"])
        # List_of_Files.append(["Par_B",    "Q2_y_Bin_14", "VS_PT", -0.575, 0.075, "3.7 < Q^{2} < 5.3", "0.55 < y < 0.65"])
        # List_of_Files.append(["Par_C_RC", "Q2_y_Bin_14", "VS_PT", -0.1,   0.15,  "3.7 < Q^{2} < 5.3", "0.55 < y < 0.65"])
        # List_of_Files.append(["Par_C",    "Q2_y_Bin_14", "VS_PT", -0.1,   0.15,  "3.7 < Q^{2} < 5.3", "0.55 < y < 0.65"])
        
        List_of_Files.append(["Par_B_RC", f"Q2_y_Bin_{Q2_y_Bin_Set}", "VS_Z", -0.6,    0.2,  Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_Q2"], Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_y"]])
        # List_of_Files.append(["Par_B",    f"Q2_y_Bin_{Q2_y_Bin_Set}", "VS_Z", -0.6,    0.2,  Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_Q2"], Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_y"]])
        List_of_Files.append(["Par_C_RC", f"Q2_y_Bin_{Q2_y_Bin_Set}", "VS_Z", -0.45, 0.25, Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_Q2"], Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_y"]])
        # List_of_Files.append(["Par_C",    f"Q2_y_Bin_{Q2_y_Bin_Set}", "VS_Z", -0.45, 0.25, Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_Q2"], Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_y"]])
    else:
        List_of_Files.append(["Par_A_RC", f"Q2_y_Bin_{Q2_y_Bin_Set}", "VS_Z",  0.825, 1.1,  Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_Q2"], Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_y"]])
        # List_of_Files.append(["Par_B_RC", f"Q2_y_Bin_{Q2_y_Bin_Set}", "VS_Z", -0.2,  0.0,  Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_Q2"], Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_y"]])
        # List_of_Files.append(["Par_C_RC", f"Q2_y_Bin_{Q2_y_Bin_Set}", "VS_Z", -0.04, 0.05, Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_Q2"], Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_y"]])
        # List_of_Files.append(["Par_C_RC", f"Q2_y_Bin_{Q2_y_Bin_Set}", "VS_Z", -0.05, 0.05, Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_Q2"], Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_y"]])


output_text = ""
for par, Bin, var, yMin, yMax, q2_range, y_range in List_of_Files:
    if(Unfold_Q):
        title_method = "#color[872]{Unfolded with RC}" if("RC" in par) else "#color[30]{Unfolded without RC}"
        title_method = f"#font[22]{{CLAS12 Pass 2 #color[632]{{Preliminary}}}} - {title_method}"
        file_name_in = f"Fit_{par}_Bayesian_Smear_{Bin}_MultiDim_z_pT_Bin_Y_bin_phi_t_{var}"
    elif("RC" in par):
        title_method = "#color[4]{EvGen RC Factors}"
        file_name_in = f"Fit_{par}_Smear_{Bin}_phi_t_{var}"
    else:
        print("// No 'RC' in 'par' --> Must skip...")
        continue
    output_text  = f"{output_text}\n{ROOT_COMMANDS}"
    output_text  = output_text.replace("Q2_RANGE",  q2_range)
    output_text  = output_text.replace("Y_RANGE",    y_range)
    output_text  = output_text.replace("MIN_RANGE", str(yMin))
    output_text  = output_text.replace("MAX_RANGE", str(yMax))
    output_text  = output_text.replace("METHOD_TITLE",    title_method)
    # output_text  = output_text.replace("MOMEMENT_TITLE",  title_moment)
    output_text  = output_text.replace("FILE_NAME_INPUT", file_name_in)
    
print(f"{output_text}\n\n.q\n")


// SAVING TCANVASES...

// --- open file and fetch the only object (assumed TCanvas) ---
TFile *f = TFile::Open("Fit_Par_A_RC_Smear_Q2_y_Bin_1_phi_t_VS_Z.root");

if (!f || f->IsZombie()) { Error("macro","Could not open file."); }
TKey *k = (TKey*)f->GetListOfKeys()->At(0);
if (!k) { Error("macro","No keys in file."); }
TCanvas *c = (TCanvas*)k->ReadObj();
if (!c || !c->InheritsFrom(TCanvas::Class())) { Error("macro","First object is not a TCanvas."); }

// Turn off ROOT's automatic title drawing (prevents TPaveText titles from reappearing)
gStyle->SetOptTitle(0);

// c->Draw();
c->cd();

// --- enter first sub-pad if one exists ---
TPad *p = nullptr;
{ TIter it(c->GetListOfPrimitives()); TObject *o;
  while ((o = it())) { if (o->InheritsFrom(TPad::Class())) { p = (TPad*)o; break; } }
}
if (p) p->cd();

gPad->Update(); // ensure frame/axes exist

// --- find the multigraph in the current pad ---
TMultiGraph *mg = nullptr;
{ TIter it(gPad->GetListOfPrimitives()); TObject *o;
  while ((

## Recreated code from after it was accidentally deleted

In [2]:
# Save_Q = True
# TLEGEND_HEIGHT = 0.25
# Unfold_Q = True
# Q2_y_Bin_Set_List = range(1, 18)

# List_of_Files = []

# # Function definition for ROOT
# ROOT_FUNCTION_DEF = r"""
# void DrawLine(const char* name, double x, double yline, double size, int font, const char* txt) {
#   if (TObject *prev = gPad->GetPrimitive(name)) { gPad->GetListOfPrimitives()->Remove(prev); delete prev; }
#   TLatex *lt = new TLatex();
#   lt->SetName(name);
#   lt->SetNDC(true);
#   lt->SetTextAlign(13); // left-top
#   lt->SetTextFont(font);
#   lt->SetTextSize(size);
#   lt->DrawLatex(x, yline, txt);
# }

# void process_one(const char* infile, const char* outfile,
#                  const char* title_top, const char* title_sub,
#                  const char* q2_range, const char* y_range,
#                  double ymin, double ymax, double legend_height) {

#   TFile *f = TFile::Open(infile);
#   if (!f || f->IsZombie()) { Error("macro","Could not open file."); return; }
#   TKey *k = (TKey*)f->GetListOfKeys()->At(0);
#   if (!k) { Error("macro","No keys in file."); return; }
#   TCanvas *c = (TCanvas*)k->ReadObj();
#   if (!c || !c->InheritsFrom(TCanvas::Class())) { Error("macro","First object is not a TCanvas."); return; }

#   c->SetCanvasSize(900, 900);
#   gStyle->SetOptTitle(0);
#   c->cd();

#   TPad *p = nullptr;
#   { TIter it(c->GetListOfPrimitives()); TObject *o; while ((o = it())) { if (o->InheritsFrom(TPad::Class())) { p = (TPad*)o; break; } } }
#   if (p) p->cd();

#   gPad->Update();

#   TMultiGraph *mg = nullptr;
#   { TIter it(gPad->GetListOfPrimitives()); TObject *o; while ((o = it())) { if (o->InheritsFrom(TMultiGraph::Class())) { mg = (TMultiGraph*)o; break; } } }
#   if (!mg) { Warning("macro","No TMultiGraph found in this pad."); return; }

#   if (mg) {
#     gPad->Update();
#     TH1 *frame = mg->GetHistogram();
#     if (frame && frame->GetYaxis()) {
#       frame->GetYaxis()->SetRangeUser(ymin, ymax);
#     } else {
#       mg->SetMinimum(ymin);
#       mg->SetMaximum(ymax);
#     }
#   }

#   // enforce marker styles
#   if (mg && mg->GetListOfGraphs()) {
#     int filledStyles[7] = {20, 21, 22, 23, 29, 33, 43};
#     int idx = 0;
#     TIter gIt(mg->GetListOfGraphs()); TObject *obj = nullptr;
#     while ((obj = gIt())) {
#       if (obj->InheritsFrom(TGraph::Class())) {
#         TGraph *gr = (TGraph*)obj;
#         gr->SetMarkerSize(3.0);
#         gr->SetMarkerStyle(filledStyles[idx]);
#         idx = (idx + 1) % 7;
#         gr->SetLineWidth(1);
#       }
#     }
#   }

#   // adjust legend position and style
#   TLegend *leg = nullptr;
#   { TIter it2(gPad->GetListOfPrimitives()); TObject *o2; while ((o2 = it2())) { if (o2->InheritsFrom(TLegend::Class())) { leg = (TLegend*)o2; break; } } }
#   if (leg) {
#     leg->SetNColumns(3);
#     leg->SetColumnSeparation(0.1);
#     leg->SetMargin(0.2);
#     double x1 = gPad->GetLeftMargin();              // aligns with y-axis
#     double x2 = 1.0 - gPad->GetRightMargin();       // aligns with right border
#     double y1 = 0.125;
#     double y2 = y1 + legend_height;
#     leg->SetX1NDC(x1);
#     leg->SetX2NDC(x2);
#     leg->SetY1NDC(y1);
#     leg->SetY2NDC(y2);
#     leg->SetBorderSize(0);
#     leg->SetFillStyle(0);   // transparent
#     leg->SetFillColor(0);   // no fill

#     // Iterate over legend entries to clean up labels and enlarge text
#     TIter nextEntry(leg->GetListOfPrimitives());
#     TObject *objEntry = nullptr;
#     while ((objEntry = nextEntry())) {
#       if (!objEntry->InheritsFrom(TLegendEntry::Class())) continue;
#       TLegendEntry *entry = (TLegendEntry*)objEntry;
#       if (!entry) continue;
#       TString label = entry->GetLabel();
#       label.ReplaceAll("z Bin:", "");
#       label.ReplaceAll("P_{T} Bin:", "");
#       entry->SetLabel(label.Data());
#       entry->SetTextSize(0.045);
#     }
#   }

#   // clear old TPaveText
#   {
#     if (TPaveText *old = (TPaveText*)gPad->GetPrimitive("title")) { gPad->GetListOfPrimitives()->Remove(old); delete old; }
#     if (TList *plist = gPad->GetListOfPrimitives()) {
#       std::vector<TObject*> toDelete;
#       TIter it3(plist); TObject *o3 = nullptr;
#       while ((o3 = it3())) { if (o3->InheritsFrom(TPaveText::Class())) { TString nm = o3->GetName(); if (nm.BeginsWith("title")) toDelete.push_back(o3); } }
#       for (auto *o : toDelete) { plist->Remove(o); delete o; }
#     }
#   }

#   // redraw the multigraph
#   mg->Draw("A P E1");

#   // redraw legend so it sits on top
#   if (leg) leg->Draw();

#   // now draw text so it sits on top
#   {
#     double Lm = gPad->GetLeftMargin();
#     double Tm = gPad->GetTopMargin();
#     const double x = Lm + 0.02;
#     double y = 1.0 - 0.50*Tm;
#     TH1 *frame = (mg ? mg->GetHistogram() : nullptr);
#     int axisFont = 42;
#     double axisSize = 0.04;
#     if (frame && frame->GetXaxis()) { axisFont = frame->GetXaxis()->GetTitleFont(); axisSize = frame->GetXaxis()->GetTitleSize(); }

#     DrawLine("user_title_l1", x, y, axisSize, axisFont, title_top);
#     y -= 1.35*axisSize;
#     DrawLine("user_title_l2", x, y, axisSize, axisFont, title_sub);
#     y -= 1.60*axisSize;
#     DrawLine("user_title_l3", x, y, axisSize, axisFont, q2_range);
#     y -= 1.25*axisSize;
#     DrawLine("user_title_l4", x, y, axisSize, axisFont, y_range);

#     if (mg) {
#       gPad->Update();
#       TH1 *fr = mg->GetHistogram();
#       if (fr) {
#         TString xtitle = fr->GetXaxis()->GetTitle(); xtitle.ReplaceAll("(Smeared) ", ""); fr->GetXaxis()->SetTitle(xtitle);
#         TString ytitle = fr->GetYaxis()->GetTitle(); if ((!ytitle.Contains("Amplitude")) && (!ytitle.Contains("Multiplicity")) ) { ytitle = "Amplitude of " + ytitle; } fr->GetYaxis()->SetTitle(ytitle);
#       }
#     }
#   }

#   gPad->Modified();
#   gPad->Update();
#   c->SaveAs(outfile);
# }
# """


# Q2_Bin_Ranges = { "1_Q2": "2.00 < Q^{2} < 2.40",
#         		   "1_y":  "0.65 < y < 0.75", 
#         		  "2_Q2": "2.00 < Q^{2} < 2.40",
#         		   "2_y":  "0.55 < y < 0.65", 
#         		  "3_Q2": "2.00 < Q^{2} < 2.40",
#         		   "3_y":  "0.45 < y < 0.55", 
#         		  "4_Q2": "2.00 < Q^{2} < 2.40",
#         		   "4_y":  "0.35 < y < 0.45", 
#         		  "5_Q2": "2.40 < Q^{2} < 2.90",
#         		   "5_y":  "0.65 < y < 0.75", 
#         		  "6_Q2": "2.40 < Q^{2} < 2.90",
#         		   "6_y":  "0.55 < y < 0.65", 
#         		  "7_Q2": "2.40 < Q^{2} < 2.90",
#         		   "7_y":  "0.45 < y < 0.55", 
#         		  "8_Q2": "2.40 < Q^{2} < 2.90",
#         		   "8_y":  "0.35 < y < 0.45", 
#         		  "9_Q2": "2.90 < Q^{2} < 3.70",
#         		   "9_y":  "0.65 < y < 0.75", 
#         		 "10_Q2": "2.90 < Q^{2} < 3.70",
#         		  "10_y":  "0.55 < y < 0.65", 
#         		 "11_Q2": "2.90 < Q^{2} < 3.70",
#         		  "11_y":  "0.45 < y < 0.55", 
#         		 "12_Q2": "2.90 < Q^{2} < 3.70",
#         		  "12_y":  "0.35 < y < 0.45", 
#         		 "13_Q2": "3.70 < Q^{2} < 5.30",
#         		  "13_y":  "0.65 < y < 0.75", 
#         		 "14_Q2": "3.70 < Q^{2} < 5.30",
#         		  "14_y":  "0.55 < y < 0.65", 
#         		 "15_Q2": "3.70 < Q^{2} < 5.30",
#         		  "15_y":  "0.45 < y < 0.55", 
#         		 "16_Q2": "5.30 < Q^{2} < 7.90",
#         		  "16_y":  "0.65 < y < 0.75", 
#         		 "17_Q2": "5.30 < Q^{2} < 7.90",
#         		  "17_y":  "0.55 < y < 0.65", 
#         		 }

# for Q2_y_Bin_Set in Q2_y_Bin_Set_List:
#     if(Unfold_Q):        
#         List_of_Files.append(["Par_B_RC", f"Q2_y_Bin_{Q2_y_Bin_Set}", "VS_Z", -0.8, 0.125, Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_Q2"], Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_y"]])
#         List_of_Files.append(["Par_B",    f"Q2_y_Bin_{Q2_y_Bin_Set}", "VS_Z", -0.8, 0.125, Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_Q2"], Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_y"]])
#         List_of_Files.append(["Par_C_RC", f"Q2_y_Bin_{Q2_y_Bin_Set}", "VS_Z", -0.3, 0.25,  Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_Q2"], Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_y"]])
#         List_of_Files.append(["Par_C",    f"Q2_y_Bin_{Q2_y_Bin_Set}", "VS_Z", -0.3, 0.25,  Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_Q2"], Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_y"]])
#     else:
#         List_of_Files.append(["Par_A_RC", f"Q2_y_Bin_{Q2_y_Bin_Set}", "VS_Z",  0.825, 1.1,  Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_Q2"], Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_y"]])
#         List_of_Files.append(["Par_B_RC", f"Q2_y_Bin_{Q2_y_Bin_Set}", "VS_Z", -0.2,   0.0,  Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_Q2"], Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_y"]])
#         List_of_Files.append(["Par_C_RC", f"Q2_y_Bin_{Q2_y_Bin_Set}", "VS_Z", -0.04, 0.05,  Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_Q2"], Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_y"]])

# # Output
# output_text = ROOT_FUNCTION_DEF + "\n\n"

# for par, Bin, var, yMin, yMax, q2_range, y_range in List_of_Files:
#     if(Unfold_Q):
#         title_top = "#font[22]{CLAS12 Preliminary}"
#         title_sub = "#font[22]{With RC Factors}" if("RC" in par) else "#font[22]{Without RC Factors}"
#         file_name_in  = f"Fit_{par}_Bayesian_Smear_{Bin}_MultiDim_z_pT_Bin_Y_bin_phi_t_{var}.root"
#         file_name_out = file_name_in.replace(".root", ".pdf")
#     elif("RC" in par):
#         title_top = "#font[22]{CLAS12 Preliminary}"
#         title_sub = "#color[4]{EvGen RC Factors}"
#         file_name_in  = f"Fit_{par}_Smear_{Bin}_phi_t_{var}.root"
#         file_name_out = file_name_in.replace(".root", ".pdf")
#     else:
#         continue

#     output_text += f'process_one("{file_name_in}", "{file_name_out}", "{title_top}", "{title_sub}", "{q2_range}", "{y_range}", {yMin}, {yMax}, {TLEGEND_HEIGHT});\n'

# output_text += "\n.q\n"

# print(output_text)


void DrawLine(const char* name, double x, double yline, double size, int font, const char* txt) {
  if (TObject *prev = gPad->GetPrimitive(name)) { gPad->GetListOfPrimitives()->Remove(prev); delete prev; }
  TLatex *lt = new TLatex();
  lt->SetName(name);
  lt->SetNDC(true);
  lt->SetTextAlign(13); // left-top
  lt->SetTextFont(font);
  lt->SetTextSize(size);
  lt->DrawLatex(x, yline, txt);
}

void process_one(const char* infile, const char* outfile,
                 const char* title_top, const char* title_sub,
                 const char* q2_range, const char* y_range,
                 double ymin, double ymax, double legend_height) {

  TFile *f = TFile::Open(infile);
  if (!f || f->IsZombie()) { Error("macro","Could not open file."); return; }
  TKey *k = (TKey*)f->GetListOfKeys()->At(0);
  if (!k) { Error("macro","No keys in file."); return; }
  TCanvas *c = (TCanvas*)k->ReadObj();
  if (!c || !c->InheritsFrom(TCanvas::Class())) { Error("macro","First object is not a TCanvas."); 

In [2]:
# === User-tunable switches and sizes ===
Save_Q = True
TLEGEND_HEIGHT = 0.25
Unfold_Q = not True
Unfold_Q = True
Q2_y_Bin_Set_List = range(1, 18)

# Q2_y_Bin_Set_List = [5, 14]
# Q2_y_Bin_Set_List = [5]


# New options requested:
Draw_Lines_with = True                  # If(True): draw connecting lines and prefix output with "Draw_Lines_with_"
Use_Single_Marker_33_Only = not True    # If(True): force all markers to style 33; else cycle 20,21,22,23,33,47,39
# MARKER_SIZE = 2.5                       # Marker size
MARKER_SIZE = 2.0                       # Marker size
LEGEND_TEXT_SIZE = 0.03                 # Legend text size
AUTO_SCALE_LEGEND_TEXT = False          # If(True): gently scale legend text to fit width; never enlarges beyond base


# === New optional features ===
DRAW_PRELIMINARY_WATERMARK = True   # If True, draw a semi-transparent red "PRELIMINARY" watermark
DRAW_CLAS_LOGO = False              # If True, overlay the CLAS logo image
CLAS_LOGO_FILE = "CLAS_LOGO.png"    # Path to logo (PNG easier to handle than PDF in ROOT)
CLAS_LOGO_X = 0.75                  # NDC X position (0=left, 1=right)
CLAS_LOGO_Y = 0.85                  # NDC Y position (0=bottom, 1=top)
CLAS_LOGO_SIZE = 0.20               # Scale factor (fraction of pad width)
CLAS_LOGO_ALPHA = 0.5               # Transparency (0=transparent, 1=opaque)


DRAW_PRELIMINARY_WATERMARK = Unfold_Q

List_of_Files = []

# Function definition for ROOT
ROOT_FUNCTION_DEF = r"""
void DrawLine(const char* name, double x, double yline, double size, int font, const char* txt) {
  if (TObject *prev = gPad->GetPrimitive(name)) { gPad->GetListOfPrimitives()->Remove(prev); delete prev; }
  TLatex *lt = new TLatex();
  lt->SetName(name);
  lt->SetNDC(true);
  lt->SetTextAlign(13); // left-top
  lt->SetTextFont(font);
  lt->SetTextSize(size);
  lt->DrawLatex(x, yline, txt);
}

void process_one(const char* infile, const char* outfile,
                 const char* title_top, const char* title_sub,
                 const char* q2_range, const char* y_range,
                 double ymin, double ymax, double legend_height,
                 bool draw_lines, double marker_size,
                 bool single_style, int single_style_code,
                 double legend_text_size, bool auto_scale_legend_text,
                 bool draw_prelim, bool draw_logo,
                 const char* logo_file, double logo_x, double logo_y,
                 double logo_size, double logo_alpha) {

    // Ensure TASImage is available
    if (!gROOT->GetClass("TASImage")) gSystem->Load("libASImage");
    
    TFile *f = TFile::Open(infile);
    if (!f || f->IsZombie()) { Error("macro","Could not open file."); return; }
    TKey *k = (TKey*)f->GetListOfKeys()->At(0);
    if (!k) { Error("macro","No keys in file."); return; }
    TCanvas *c = (TCanvas*)k->ReadObj();
    if (!c || !c->InheritsFrom(TCanvas::Class())) { Error("macro","First object is not a TCanvas."); return; }
    
    c->SetCanvasSize(900, 900);
    gStyle->SetOptTitle(0);
    c->cd();
    
    TPad *p = nullptr;
    { TIter it(c->GetListOfPrimitives()); TObject *o; while ((o = it())) { if (o->InheritsFrom(TPad::Class())) { p = (TPad*)o; break; } } }
    if (p) p->cd();
    
    // gPad->SetFrameBorderMode(0);   // no border frame
    // gPad->SetFrameLineWidth(0);    // prevent thin leftover line
    // gPad->SetBorderMode(0);        // no outer pad border
    // // Make sure we’re changing the pad that actually draws the MG
    // TPad *drawPad = dynamic_cast<TPad*>(gPad);
    // if (drawPad) {
    // drawPad->SetTopMargin(0.18);   // try 0.16–0.22 to taste
    // drawPad->Modified();
    // }
    
    gPad->Update();
    
    // Give yourself more headroom for the title
    gPad->SetTopMargin(0.1);   // tweak (0.22–0.32) as needed
    
    // Remove the outer pad border (the second box you see)
    gPad->SetBorderMode(0);
    gPad->SetBorderSize(0);
    
    // Also make sure the canvas isn't adding a border
    c->SetBorderMode(0);
    c->SetBorderSize(0);
    
    // (Keep this — it does NOT affect the axis box)
    gStyle->SetPadBorderMode(0);
    gStyle->SetCanvasBorderMode(0);
    
    
    TMultiGraph *mg = nullptr;
    { TIter it(gPad->GetListOfPrimitives()); TObject *o; while ((o = it())) { if (o->InheritsFrom(TMultiGraph::Class())) { mg = (TMultiGraph*)o; break; } } }
    if (!mg) { Warning("macro","No TMultiGraph found in this pad."); return; }
    
    if (mg) {
    gPad->Update();
    TH1 *frame = mg->GetHistogram();
    if (frame && frame->GetYaxis()) {
      frame->GetYaxis()->SetRangeUser(ymin, ymax);
    } else {
      mg->SetMinimum(ymin);
      mg->SetMaximum(ymax);
    }
    }
    
    // enforce marker styles/sizes
    if (mg && mg->GetListOfGraphs()) {
    int cycleStyles[7] = {20, 21, 22, 23, 33, 47, 39}; // <-- updated order per request
    int idx = 0;
    TIter gIt(mg->GetListOfGraphs()); TObject *obj = nullptr;
    while ((obj = gIt())) {
      if (obj->InheritsFrom(TGraph::Class())) {
        TGraph *gr = (TGraph*)obj;
        gr->SetMarkerSize(marker_size);
        if (single_style) {
          gr->SetMarkerStyle(single_style_code);
        } else {
          gr->SetMarkerStyle(cycleStyles[idx]);
          idx = (idx + 1) % 7;
        }
        gr->SetLineWidth(1); // keeps error bars and optional lines readable
      }
    }
    }
    
    // adjust legend position and style
    TLegend *leg = nullptr;
    { TIter it2(gPad->GetListOfPrimitives()); TObject *o2; while ((o2 = it2())) { if (o2->InheritsFrom(TLegend::Class())) { leg = (TLegend*)o2; break; } } }
    if (leg) {
    leg->SetNColumns(3);
    leg->SetColumnSeparation(0.1);
    leg->SetMargin(0.2);
    double x1 = gPad->GetLeftMargin();              // aligns with y-axis
    double x2 = 1.0 - gPad->GetRightMargin();       // aligns with right border
    double y1 = 0.125;
    double y2 = y1 + legend_height;
    leg->SetX1NDC(x1);
    leg->SetX2NDC(x2);
    leg->SetY1NDC(y1);
    leg->SetY2NDC(y2);
    leg->SetBorderSize(0);
    leg->SetFillStyle(0);   // transparent
    leg->SetFillColor(0);   // no fill
    
    // Clean labels and set base text size
    TIter nextEntry(leg->GetListOfPrimitives());
    TObject *objEntry = nullptr;
    while ((objEntry = nextEntry())) {
      if (!objEntry->InheritsFrom(TLegendEntry::Class())) continue;
      TLegendEntry *entry = (TLegendEntry*)objEntry;
      if (!entry) continue;
      TString label = entry->GetLabel();
      label.ReplaceAll("z Bin:", "");
      label.ReplaceAll("P_{T} Bin:", "");
      entry->SetLabel(label.Data());
      entry->SetTextSize(legend_text_size);
    }
    
    // Optional: auto-scale legend text to fit the width, never enlarge above base
    if (auto_scale_legend_text) {
      double legWidth = leg->GetX2NDC() - leg->GetX1NDC();
      double scale = legWidth / 0.70; // heuristic: ~0.70 NDC supports base size
      double minSize = 0.025;
      double newSize = legend_text_size * ((scale < 1.0) ? scale : 1.0);
      if (newSize < minSize) newSize = minSize;
    
      TIter nextEntry2(leg->GetListOfPrimitives());
      TObject *objEntry2 = nullptr;
      while ((objEntry2 = nextEntry2())) {
        if (!objEntry2->InheritsFrom(TLegendEntry::Class())) continue;
        TLegendEntry *entry2 = (TLegendEntry*)objEntry2;
        if (!entry2) continue;
        entry2->SetTextSize(newSize);
      }
    }
    }
    
    // clear old TPaveText
    {
    if (TPaveText *old = (TPaveText*)gPad->GetPrimitive("title")) { gPad->GetListOfPrimitives()->Remove(old); delete old; }
    if (TList *plist = gPad->GetListOfPrimitives()) {
      std::vector<TObject*> toDelete;
      TIter it3(plist); TObject *o3 = nullptr;
      while ((o3 = it3())) { if (o3->InheritsFrom(TPaveText::Class())) { TString nm = o3->GetName(); if (nm.BeginsWith("title")) toDelete.push_back(o3); } }
      for (auto *o : toDelete) { plist->Remove(o); delete o; }
    }
    }
    
    // redraw the multigraph (choose options based on draw_lines)
    const char* drawOpt = draw_lines ? "A P E1 L" : "A P E1";
    mg->Draw(drawOpt);
    
    // redraw legend so it sits on top
    if (leg) leg->Draw();
    
    // now draw text so it sits on top
    {
    double Lm = gPad->GetLeftMargin();
    double Tm = gPad->GetTopMargin();
    // const double x = Lm + 0.02;
    double x = Lm + 0.02;
    double y = 1.0 - 0.50*Tm;
    TH1 *frame = (mg ? mg->GetHistogram() : nullptr);
    int axisFont = 42;
    double axisSize = 0.04;
    if (frame && frame->GetXaxis()) { axisFont = frame->GetXaxis()->GetTitleFont(); axisSize = frame->GetXaxis()->GetTitleSize(); }
    
    y += 0.5*axisSize;
    y += 0.65*axisSize;
    // y += 0.25*axisSize;
    x -= 0.075;
    DrawLine("user_title_l1", x, y, 1.5*axisSize, axisFont, title_top);
    // y -= 0.25*axisSize;
    y -= 0.35*axisSize;
    y -= 1.0*axisSize;
    x += 0.075;
    DrawLine("user_title_l2", x, y, 1.05*axisSize, axisFont, title_sub);
    y -= 1.25*axisSize;
    // x += 0.05;
    DrawLine("user_title_l3", x, y, axisSize, axisFont, q2_range);
    y -= 1.25*axisSize;
    DrawLine("user_title_l4", x, y, axisSize, axisFont, y_range);
    
    if (mg) {
      gPad->Update();
      TH1 *fr = mg->GetHistogram();
      if (fr) {
        TString xtitle = fr->GetXaxis()->GetTitle(); xtitle.ReplaceAll("(Smeared) ", ""); fr->GetXaxis()->SetTitle(xtitle);
        TString ytitle = fr->GetYaxis()->GetTitle();
        // if ((!ytitle.Contains("Amplitude")) && (!ytitle.Contains("Multiplicity"))) { ytitle = "Amplitude of " + ytitle; }
        if (TString(title_top).Contains("Factors")) {
            if ((!ytitle.Contains("Amplitude")) && (!ytitle.Contains("Multiplicity"))) { ytitle = "#scale[0.9]{" + ytitle + " Moment of RC Factors}"; }
        }
        else {
            if ((!ytitle.Contains("Amplitude")) && (!ytitle.Contains("Multiplicity"))) { ytitle = "#scale[0.9]{" + ytitle + " Moment}"; }
        }
        if (ytitle.Contains("Multiplicity")) { ytitle = "Amplitude"; }
        fr->GetYaxis()->SetTitle(ytitle);
      }
    }
    }
    
    // --- Optionally draw "PRELIMINARY" watermark ---
    if (draw_prelim) {
    TLatex *watermark = new TLatex(0.5, 0.5, "PRELIMINARY");
    watermark->SetNDC(true);
    watermark->SetTextAlign(22);   // center
    watermark->SetTextFont(62);    // bold
    watermark->SetTextSize(0.125);  // adjust size
    // watermark->SetTextSize(0.15);  // adjust size
    // watermark->SetTextColorAlpha(kRed, 0.1); // red with 90% transparency
    watermark->SetTextColorAlpha(kRed, 0.05); // red with 95% transparency
    watermark->SetTextAngle(-30);  // tilt
    watermark->Draw();
    }
    
    // --- Optionally draw CLAS logo ---
    if (draw_logo) {
      if (gSystem->AccessPathName(logo_file)) {
        Warning("macro","Logo file %s not found", logo_file);
      } else {
        // Read image
        TImage *baseImg = TImage::Open(logo_file);
        if (!baseImg) {
          Warning("macro","Failed to open logo image: %s", logo_file);
        } else {
          // We’ll use TASImage features when available
          TASImage *logo = dynamic_cast<TASImage*>(baseImg);
    
          // Compute a rectangle in NDC (centered at logo_x, logo_y)
          // Keep aspect ratio if we know it; otherwise fall back to square
          double w_ndc = logo_size; // fraction of pad width
          double h_ndc = logo_size;
          if (logo && logo->GetWidth() > 0) {
            double ar = (double)logo->GetHeight() / (double)logo->GetWidth();
            h_ndc = w_ndc * ar; // preserve aspect ratio
          }
          double x1 = logo_x - 0.5*w_ndc;
          double x2 = logo_x + 0.5*w_ndc;
          double y1 = logo_y - 0.5*h_ndc;
          double y2 = logo_y + 0.5*h_ndc;
    
          // Create a transparent overlay sub-pad exactly at that rectangle
          static int sLogoPadCounter = 0;
          TString padName = Form("logoPad_%d", ++sLogoPadCounter);
          TPad *logoPad = new TPad(padName, padName, x1, y1, x2, y2);
          // Full transparency, no frame/border
          logoPad->SetFillStyle(4000);       // 4000 = truly transparent
          logoPad->SetFillColor(0);
          logoPad->SetFrameFillStyle(0);
          logoPad->SetFrameBorderMode(0);
          logoPad->SetBorderMode(0);
          logoPad->Draw();
          logoPad->cd();
    
          // Optional global alpha (0..1) for the PNG (manual clamp)
          if (logo) {
            double alpha = logo_alpha;
            if (alpha < 0.0) alpha = 0.0;
            if (alpha > 1.0) alpha = 1.0;
            // NOTE: older ROOT lacks SetConstAlpha. Use PNG's own transparency instead.
            logo->Draw("X");   // "X" = draw with transparency if PNG has it
          } else {
            baseImg->Draw("X");
          }
    
    
          // Back to main canvas
          c->cd();
        }
      }
    }
    
    
    
    gPad->Modified();
    gPad->Update();
    c->SaveAs(outfile);
}
"""


# Q2 and y ranges — note y ranges are ascending (fixed)
Q2_Bin_Ranges = {
    "1_Q2": "2.00 < Q^{2} < 2.40",  "1_y":  "0.65 < y < 0.75",
    "2_Q2": "2.00 < Q^{2} < 2.40",  "2_y":  "0.55 < y < 0.65",
    "3_Q2": "2.00 < Q^{2} < 2.40",  "3_y":  "0.45 < y < 0.55",
    "4_Q2": "2.00 < Q^{2} < 2.40",  "4_y":  "0.35 < y < 0.45",
    "5_Q2": "2.40 < Q^{2} < 2.90",  "5_y":  "0.65 < y < 0.75",
    "6_Q2": "2.40 < Q^{2} < 2.90",  "6_y":  "0.55 < y < 0.65",
    "7_Q2": "2.40 < Q^{2} < 2.90",  "7_y":  "0.45 < y < 0.55",
    "8_Q2": "2.40 < Q^{2} < 2.90",  "8_y":  "0.35 < y < 0.45",
    "9_Q2": "2.90 < Q^{2} < 3.70",  "9_y":  "0.65 < y < 0.75",
    "10_Q2": "2.90 < Q^{2} < 3.70", "10_y": "0.55 < y < 0.65",
    "11_Q2": "2.90 < Q^{2} < 3.70", "11_y": "0.45 < y < 0.55",
    "12_Q2": "2.90 < Q^{2} < 3.70", "12_y": "0.35 < y < 0.45",
    "13_Q2": "3.70 < Q^{2} < 5.30", "13_y": "0.65 < y < 0.75",
    "14_Q2": "3.70 < Q^{2} < 5.30", "14_y": "0.55 < y < 0.65",
    "15_Q2": "3.70 < Q^{2} < 5.30", "15_y": "0.45 < y < 0.55",
    "16_Q2": "5.30 < Q^{2} < 7.90", "16_y": "0.65 < y < 0.75",
    "17_Q2": "5.30 < Q^{2} < 7.90", "17_y": "0.55 < y < 0.65",
}

# Build file list based on your corrected ranges and modes
for Q2_y_Bin_Set in Q2_y_Bin_Set_List:
    if(Unfold_Q):
        List_of_Files.append(["Par_B_RC", f"Q2_y_Bin_{Q2_y_Bin_Set}", "VS_Z", -0.8, 0.125, Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_Q2"], Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_y"]])
        # List_of_Files.append(["Par_B",    f"Q2_y_Bin_{Q2_y_Bin_Set}", "VS_Z", -0.8, 0.125, Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_Q2"], Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_y"]])
        List_of_Files.append(["Par_C_RC", f"Q2_y_Bin_{Q2_y_Bin_Set}", "VS_Z", -0.3, 0.25,  Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_Q2"], Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_y"]])
        # List_of_Files.append(["Par_C",    f"Q2_y_Bin_{Q2_y_Bin_Set}", "VS_Z", -0.3, 0.25,  Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_Q2"], Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_y"]])

        List_of_Files.append(["Par_B_RC", f"Q2_y_Bin_{Q2_y_Bin_Set}", "VS_PT", -0.8, 0.125, Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_Q2"], Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_y"]])
        # List_of_Files.append(["Par_B",    f"Q2_y_Bin_{Q2_y_Bin_Set}", "VS_PT", -0.8, 0.125, Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_Q2"], Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_y"]])
        List_of_Files.append(["Par_C_RC", f"Q2_y_Bin_{Q2_y_Bin_Set}", "VS_PT", -0.3, 0.25,  Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_Q2"], Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_y"]])
        # List_of_Files.append(["Par_C",    f"Q2_y_Bin_{Q2_y_Bin_Set}", "VS_PT", -0.3, 0.25,  Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_Q2"], Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_y"]])
    else:
        # Keep your corrected Unfold_Q = False options exactly
        List_of_Files.append(["Par_A_RC", f"Q2_y_Bin_{Q2_y_Bin_Set}", "VS_Z",  0.775,  1.1,  Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_Q2"], Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_y"]])
        List_of_Files.append(["Par_B_RC", f"Q2_y_Bin_{Q2_y_Bin_Set}", "VS_Z", -0.25,  0.02,  Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_Q2"], Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_y"]])
        List_of_Files.append(["Par_C_RC", f"Q2_y_Bin_{Q2_y_Bin_Set}", "VS_Z", -0.08,  0.05, Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_Q2"], Q2_Bin_Ranges[f"{Q2_y_Bin_Set}_y"]])

# Output macro + calls
output_text = ROOT_FUNCTION_DEF + "\n\n"

# for par, Bin, var, yMin, yMax, q2_range, y_range in List_of_Files:
#     if(Unfold_Q):
#         # title_top = "#font[22]{CLAS12 Preliminary}"
#         title_top = "#font[22]{CLAS12 Preliminary #topbar TITLE}"
#         title_top = title_top.replace("TITLE", f"Amplitude of {'Cos(#phi_{h})' if('Par_B' in str(par)) else 'Cos(2#phi_{h})'}")
#         title_sub = "#font[22]{With RC Factors}" if("RC" in par) else "#font[22]{Without RC Factors}"
#         file_name_in  = f"Fit_{par}_Bayesian_Smear_{Bin}_MultiDim_z_pT_Bin_Y_bin_phi_t_{var}.root"
#     elif("RC" in par):
#         # title_top = "#font[22]{CLAS12 Preliminary}"
#         # title_top = "#font[22]{CLAS12 Preliminary #topbar TITLE}"
#         title_top = "#scale[0.8]{#font[22]{TITLE}}"
#         title_top = title_top.replace("TITLE", f"Amplitude of {'Cos(#phi_{h})' if('Par_B' in str(par)) else 'Cos(2#phi_{h})'} Moments within the RC Factors" if("Par_A" not in str(par)) else "Multiplicity")
#         # title_sub = "#color[4]{EvGen RC Factors = #scale[0.575]{#frac{#sigma_{RAD}}{#sigma_{BORN}}}}"
#         # title_sub = "#color[4]{EvGen RC Factors}"
#         title_sub = ""
#         file_name_in  = f"Fit_{par}_Smear_{Bin}_phi_t_{var}.root"
#     else:
#         continue

for par, Bin, var, yMin, yMax, q2_range, y_range in List_of_Files:
    if(Unfold_Q):
        # title_top = "#font[22]{CLAS12 Preliminary}"
        title_top = "#font[22]{CLAS12 Preliminary #topbar TITLE}"
        # title_top = title_top.replace("TITLE", f"Amplitude of {'Cos(#phi_{h})' if('Par_B' in str(par)) else 'Cos(2#phi_{h})'}")
        title_top = title_top.replace("TITLE", f"{'Cos(#phi_{h})' if('Par_B' in str(par)) else 'Cos(2#phi_{h})'} Moments")
        title_sub = "#font[22]{With RC Factors}" if("RC" in par) else "#font[22]{Without RC Factors}"
        file_name_in  = f"Fit_{par}_Bayesian_Smear_{Bin}_MultiDim_z_pT_Bin_Y_bin_phi_t_{var}.root"
    elif("RC" in par):
        # title_top = "#font[22]{CLAS12 Preliminary}"
        # title_top = "#font[22]{CLAS12 Preliminary #topbar TITLE}"
        title_top = "#scale[0.8]{#font[22]{TITLE}}"
        # title_top = title_top.replace("TITLE", f"Amplitude of {'Cos(#phi_{h})' if('Par_B' in str(par)) else 'Cos(2#phi_{h})'} Moments of the RC Factors" if("Par_A" not in str(par)) else "(General) Amplitude of RC Factors")
        title_top = title_top.replace("TITLE", f"{'Cos(#phi_{h})' if('Par_B' in str(par)) else 'Cos(2#phi_{h})'} Moments of the RC Factors" if("Par_A" not in str(par)) else "(General) Amplitude of RC Factors")
        # title_sub = "#color[4]{EvGen RC Factors = #scale[0.575]{#frac{#sigma_{RAD}}{#sigma_{BORN}}}}"
        # title_sub = "#color[4]{EvGen RC Factors}"
        title_sub = ""
        file_name_in  = f"Fit_{par}_Smear_{Bin}_phi_t_{var}.root"
    else:
        continue

    file_name_out = file_name_in.replace(".root", ".pdf")
    if(Draw_Lines_with and Unfold_Q):
        # Prefix requested string to the saved filename
        file_name_out = f"Draw_Lines_with_{file_name_out}"

    # file_name_out = f"LOGO_Test_{file_name_out}"

    # output_text += (
    #     f'process_one("{file_name_in}", "{file_name_out}", '
    #     f'"{title_top}", "{title_sub}", "{q2_range}", "{y_range}", '
    #     f'{yMin}, {yMax}, {TLEGEND_HEIGHT}, '
    #     f'{"true" if Draw_Lines_with else "false"}, '
    #     f'{MARKER_SIZE}, '
    #     f'{"true" if Use_Single_Marker_33_Only else "false"}, '
    #     f'33, {LEGEND_TEXT_SIZE}, '
    #     f'{"true" if AUTO_SCALE_LEGEND_TEXT else "false"}, '
    #     f'{"true" if DRAW_PRELIMINARY_WATERMARK else "false"}, '
    #     f'{"true" if DRAW_CLAS_LOGO else "false"}, '
    #     f'"{CLAS_LOGO_FILE}", {CLAS_LOGO_X}, {CLAS_LOGO_Y}, '
    #     f'{CLAS_LOGO_SIZE}, {CLAS_LOGO_ALPHA});\n'
    # )
    output_text += (
        f'process_one("{file_name_in}", "{file_name_out}", '
        f'"{title_top}", "{title_sub}", "{q2_range}", "{y_range}", '
        f'{yMin}, {yMax}, {TLEGEND_HEIGHT}, '
        f'{"true" if Draw_Lines_with else "false"}, '
        f'{MARKER_SIZE}, '
        f'{"true" if Use_Single_Marker_33_Only else "false"}, '
        f'33, {LEGEND_TEXT_SIZE}, '
        f'{"true" if AUTO_SCALE_LEGEND_TEXT else "false"}, '
        f'{"true" if DRAW_PRELIMINARY_WATERMARK else "false"}, '
        f'{"true" if DRAW_CLAS_LOGO else "false"}, '
        f'"{CLAS_LOGO_FILE}", {CLAS_LOGO_X}, {CLAS_LOGO_Y}, '
        f'{CLAS_LOGO_SIZE}, {CLAS_LOGO_ALPHA});\n'
    )



if(Save_Q):
    output_text += "\n.q\n"

print(output_text)



void DrawLine(const char* name, double x, double yline, double size, int font, const char* txt) {
  if (TObject *prev = gPad->GetPrimitive(name)) { gPad->GetListOfPrimitives()->Remove(prev); delete prev; }
  TLatex *lt = new TLatex();
  lt->SetName(name);
  lt->SetNDC(true);
  lt->SetTextAlign(13); // left-top
  lt->SetTextFont(font);
  lt->SetTextSize(size);
  lt->DrawLatex(x, yline, txt);
}

void process_one(const char* infile, const char* outfile,
                 const char* title_top, const char* title_sub,
                 const char* q2_range, const char* y_range,
                 double ymin, double ymax, double legend_height,
                 bool draw_lines, double marker_size,
                 bool single_style, int single_style_code,
                 double legend_text_size, bool auto_scale_legend_text,
                 bool draw_prelim, bool draw_logo,
                 const char* logo_file, double logo_x, double logo_y,
                 double logo_size, double logo_alph

### Extra Lines

In [44]:
test = "Q2_Bin_Ranges = {"
Q2 = [2, 2.4, 2.9, 3.7, 5.3, 7.9]
y_col = 0
Q2row = 0
for ii in range(1, 18):
    test = f"{test}'{ii:>2.0f}_Q2': '{Q2[Q2row]:1.2f} < Q^{{2}} < {Q2[Q2row+1]:1.2f}',\n\t\t  '{ii:>2.0f}_y':  '{0.75-(y_col*0.1):1.2f} < y < {0.65-(y_col*0.1):1.2f}', \n\t\t "
    y_col += 1
    if(ii in [4, 8, 12, 15, 17]):
        y_col  = 0
        Q2row += 1
    
test = test.replace("'", '"')
test = test.replace('" ', ' "')
print(f"\n{test}}}\n")


Q2_Bin_Ranges = { "1_Q2": "2.00 < Q^{2} < 2.40",
		   "1_y":  "0.75 < y < 0.65", 
		  "2_Q2": "2.00 < Q^{2} < 2.40",
		   "2_y":  "0.65 < y < 0.55", 
		  "3_Q2": "2.00 < Q^{2} < 2.40",
		   "3_y":  "0.55 < y < 0.45", 
		  "4_Q2": "2.00 < Q^{2} < 2.40",
		   "4_y":  "0.45 < y < 0.35", 
		  "5_Q2": "2.40 < Q^{2} < 2.90",
		   "5_y":  "0.75 < y < 0.65", 
		  "6_Q2": "2.40 < Q^{2} < 2.90",
		   "6_y":  "0.65 < y < 0.55", 
		  "7_Q2": "2.40 < Q^{2} < 2.90",
		   "7_y":  "0.55 < y < 0.45", 
		  "8_Q2": "2.40 < Q^{2} < 2.90",
		   "8_y":  "0.45 < y < 0.35", 
		  "9_Q2": "2.90 < Q^{2} < 3.70",
		   "9_y":  "0.75 < y < 0.65", 
		 "10_Q2": "2.90 < Q^{2} < 3.70",
		  "10_y":  "0.65 < y < 0.55", 
		 "11_Q2": "2.90 < Q^{2} < 3.70",
		  "11_y":  "0.55 < y < 0.45", 
		 "12_Q2": "2.90 < Q^{2} < 3.70",
		  "12_y":  "0.45 < y < 0.35", 
		 "13_Q2": "3.70 < Q^{2} < 5.30",
		  "13_y":  "0.75 < y < 0.65", 
		 "14_Q2": "3.70 < Q^{2} < 5.30",
		  "14_y":  "0.65 < y < 0.55", 
		 "15_Q2": "3.70 < Q^{2} < 5.30"

In [5]:
tdf = "not N/A"
out_print_main_tdf = "None"
if((tdf not in ["N/A"]) and out_print_main_tdf):
    print("Pass")
else:
    print("False")

Pass


In [1]:
test = """\\begin{figure}[!h]
    \centering
    \\begin{subfigure}{0.3\\textwidth}
        \includegraphics[width=\\textwidth, height=\\textwidth]{All_Images/Analysis_Procedure_Section/Radiative_Corrections/Fit_Par_A_RC_Smear_Q2_y_Bin_#_phi_t_VS_Z.pdf}
        \caption{RC Fit Parameter A}
    \end{subfigure}%
    \\begin{subfigure}{0.3\\textwidth}
        \includegraphics[width=\\textwidth, height=\\textwidth]{All_Images/Analysis_Procedure_Section/Radiative_Corrections/Fit_Par_B_RC_Smear_Q2_y_Bin_#_phi_t_VS_Z.pdf}
        \caption{RC Fit Parameter B}
    \end{subfigure}%
    \\begin{subfigure}{0.3\\textwidth}
        \includegraphics[width=\\textwidth, height=\\textwidth]{All_Images/Analysis_Procedure_Section/Radiative_Corrections/Fit_Par_C_RC_Smear_Q2_y_Bin_#_phi_t_VS_Z.pdf}
        \caption{RC Fit Parameter C}
    \end{subfigure}
    \caption{RC Factor Parameters as functions of $z$ in $Q^2$-$y$ Bin #. See \Cref{FitForMomentsFunction} for definitions.}
    \label{fig:RC_Fit_Factors_for_Q2_y_Bin_#}
\end{figure}

"""

test2 = "\n\n"
for ii in range(1, 18):
    test2 = f"{test2}{test.replace('#', str(ii))}"


print(test2)



\begin{figure}[!h]
    \centering
    \begin{subfigure}{0.3\textwidth}
        \includegraphics[width=\textwidth, height=\textwidth]{All_Images/Analysis_Procedure_Section/Radiative_Corrections/Fit_Par_A_RC_Smear_Q2_y_Bin_1_phi_t_VS_Z.pdf}
        \caption{RC Fit Parameter A}
    \end{subfigure}%
    \begin{subfigure}{0.3\textwidth}
        \includegraphics[width=\textwidth, height=\textwidth]{All_Images/Analysis_Procedure_Section/Radiative_Corrections/Fit_Par_B_RC_Smear_Q2_y_Bin_1_phi_t_VS_Z.pdf}
        \caption{RC Fit Parameter B}
    \end{subfigure}%
    \begin{subfigure}{0.3\textwidth}
        \includegraphics[width=\textwidth, height=\textwidth]{All_Images/Analysis_Procedure_Section/Radiative_Corrections/Fit_Par_C_RC_Smear_Q2_y_Bin_1_phi_t_VS_Z.pdf}
        \caption{RC Fit Parameter C}
    \end{subfigure}
    \caption{RC Factor Parameters as functions of $z$ in $Q^2$-$y$ Bin 1. See \Cref{FitForMomentsFunction} for definitions.}
    \label{fig:RC_Fit_Factors_for_Q2_y_Bin_1}
\end{

In [3]:
test = """\\begin{figure}[h!]
    \centering
    \includegraphics[width=0.95\\textwidth, height=0.4\\textheight]{All_Images/Data_Collection_Section/MC_Simulation_Sections/Pass_2_clasdis_preunfold/Multi_3D_Unfolded_Histos_Q2_y_Bin_#_Data_Smeared.png}
    \caption{\label{fig:Pre_Unfolded_Phi_h_Q2_y_Bin_#}$\phi_{h}$ Comparisons between Data and the \\textsc{clasdis} MC for all events in $Q^2$-$y$ Bin #. See \Cref{fig:Pre_Unfolded_Phi_h_Q2_y_Bin_1} for more details}
\end{figure}

"""

test2 = "\n\n"
for ii in range(1, 18):
    test2 = f"{test2}{test.replace('#', str(ii))}"


print(test2)



\begin{figure}[h!]
    \centering
    \includegraphics[width=0.95\textwidth, height=0.4\textheight]{All_Images/Data_Collection_Section/MC_Simulation_Sections/Pass_2_clasdis_preunfold/Multi_3D_Unfolded_Histos_Q2_y_Bin_1_Data_Smeared.png}
    \caption{\label{fig:Pre_Unfolded_Phi_h_Q2_y_Bin_1}$\phi_{h}$ Comparisons between Data and the \textsc{clasdis} MC for all events in $Q^2$-$y$ Bin 1. See \Cref{fig:Pre_Unfolded_Phi_h_Q2_y_Bin_1} for more details}
\end{figure}

\begin{figure}[h!]
    \centering
    \includegraphics[width=0.95\textwidth, height=0.4\textheight]{All_Images/Data_Collection_Section/MC_Simulation_Sections/Pass_2_clasdis_preunfold/Multi_3D_Unfolded_Histos_Q2_y_Bin_2_Data_Smeared.png}
    \caption{\label{fig:Pre_Unfolded_Phi_h_Q2_y_Bin_2}$\phi_{h}$ Comparisons between Data and the \textsc{clasdis} MC for all events in $Q^2$-$y$ Bin 2. See \Cref{fig:Pre_Unfolded_Phi_h_Q2_y_Bin_1} for more details}
\end{figure}

\begin{figure}[h!]
    \centering
    \includegraphics[width=0.9

In [11]:
len([["y", "$y$"], ["z", "$z$"], ["pT", "$P_T$"], ["xB", "$x_B$"], ["phi_t", "$\phi_h$"], ["W", "$W$"], ["MM", "Missing Mass"], ["el", "Electron Momentum"], ["elth", "$\\theta_{Electron}$"], ["elPhi", "$\phi_{Electron}$"],  ["pip", "$\pi^+$ Pion Momentum"], ["pipth", "$\\theta_{\pi^{+}\\text{ Pion}}$"], ["pipPhi", "$\phi_{\pi^{+}\\text{ Pion}}$"]])

13

In [31]:

# test = """\\begin{figure}[h!]
#     \centering
#     \includegraphics[width=0.85\\textwidth, height=0.275\\textheight, trim=0 400 25 0,clip]{All_Images/Data_Collection_Section/MC_Simulation_Sections/Pass_2_Kinematic_Comparisons_with_EvGen/Kinematic_Comparison_of_VAR_High_Mx_Norm_Full_EvGen_Weighed.png}
#     \caption{\label{fig:Kinematic_Compare_VAR}Comparisons between Data and both MCs of the TEXT_var distributions. See \Cref{fig:Kinematic_Compare_Q2} for more details}
# \end{figure}

# """

test = """\\begin{figure}[h!]
    \centering
    \includegraphics[width=1\\textwidth, trim=0 400 25 0,clip]{All_Images/Data_Collection_Section/MC_Simulation_Sections/Pass_2_Kinematic_Comparisons_with_EvGen/Kinematic_Comparison_of_VAR_High_Mx_Norm_Full_EvGen_Weighed.png}
    \caption{\label{fig:Kinematic_Compare_VAR}Comparisons between Data and both MCs of the TEXT_var distributions. See \Cref{fig:Kinematic_Compare_Q2} for more details}
\end{figure}

"""

test2 = "\n\n"
for ii, jj in [["y", "$y$"], ["z", "$z$"], ["pT", "$P_T$"], ["xB", "$x_B$"], ["phi_t", "$\phi_h$"], ["W", "$W$"], ["MM", "Missing Mass"], ["el", "Electron Momentum"], ["elth", "$\\theta_{Electron}$"], ["elPhi", "$\phi_{Electron}$"],  ["pip", "$\pi^+$ Pion Momentum"], ["pipth", "$\\theta_{\pi^{+}\\text{ Pion}}$"], ["pipPhi", "$\phi_{\pi^{+}\\text{ Pion}}$"]]:
    test2 = f"{test2}{test.replace('VAR', str(ii))}"
    test2 = test2.replace("TEXT_var", str(jj))


print(test2)



\begin{figure}[h!]
    \centering
    \includegraphics[width=1\textwidth, trim=0 400 25 0,clip]{All_Images/Data_Collection_Section/MC_Simulation_Sections/Pass_2_Kinematic_Comparisons_with_EvGen/Kinematic_Comparison_of_y_High_Mx_Norm_Full_EvGen_Weighed.png}
    \caption{\label{fig:Kinematic_Compare_y}Comparisons between Data and both MCs of the $y$ distributions. See \Cref{fig:Kinematic_Compare_Q2} for more details}
\end{figure}

\begin{figure}[h!]
    \centering
    \includegraphics[width=1\textwidth, trim=0 400 25 0,clip]{All_Images/Data_Collection_Section/MC_Simulation_Sections/Pass_2_Kinematic_Comparisons_with_EvGen/Kinematic_Comparison_of_z_High_Mx_Norm_Full_EvGen_Weighed.png}
    \caption{\label{fig:Kinematic_Compare_z}Comparisons between Data and both MCs of the $z$ distributions. See \Cref{fig:Kinematic_Compare_Q2} for more details}
\end{figure}

\begin{figure}[h!]
    \centering
    \includegraphics[width=1\textwidth, trim=0 400 25 0,clip]{All_Images/Data_Collection_Section/M

In [7]:
test = r"""\begin{figure}[h!]
    \centering
    \begin{subfigure}{0.49\textwidth}\includegraphics[width=0.85\textwidth]{All_Images/Data_Collection_Section/Distributions_and_Binning/RDF_Kinematic_Variable_Bins_for_Q2_y_Bin_NUM_1.png}
        \caption{$Q^2$-$y$ Bin NUM_1}
    \end{subfigure}%
    \begin{subfigure}{0.49\textwidth}\includegraphics[width=0.85\textwidth]{All_Images/Data_Collection_Section/Distributions_and_Binning/RDF_Kinematic_Variable_Bins_for_Q2_y_Bin_NUM_2.png}
        \caption{$Q^2$-$y$ Bin NUM_2}
    \end{subfigure}
    \caption{\label{fig:z_pT_for_Q2_y_NUM_1_NUM_2}2D Kinematic Binning Schemes for $Q^2$-$y$ Bin NUM_1 and NUM_2.}
\end{figure}

"""
print("")
for ii in range(1, 15, 2):
    temp_test =      test.replace("NUM_1", str(ii))
    temp_test = temp_test.replace("NUM_2", str(ii+1))
    print(temp_test)
print("")


\begin{figure}[h!]
    \centering
    \begin{subfigure}{0.49\textwidth}\includegraphics[width=0.85\textwidth]{All_Images/Data_Collection_Section/Distributions_and_Binning/RDF_Kinematic_Variable_Bins_for_Q2_y_Bin_1.png}
        \caption{$Q^2$-$y$ Bin 1}
    \end{subfigure}%
    \begin{subfigure}{0.49\textwidth}\includegraphics[width=0.85\textwidth]{All_Images/Data_Collection_Section/Distributions_and_Binning/RDF_Kinematic_Variable_Bins_for_Q2_y_Bin_2.png}
        \caption{$Q^2$-$y$ Bin 2}
    \end{subfigure}
    \caption{\label{fig:z_pT_for_Q2_y_1_2}2D Kinematic Binning Schemes for $Q^2$-$y$ Bin 1 and 2.}
\end{figure}


\begin{figure}[h!]
    \centering
    \begin{subfigure}{0.49\textwidth}\includegraphics[width=0.85\textwidth]{All_Images/Data_Collection_Section/Distributions_and_Binning/RDF_Kinematic_Variable_Bins_for_Q2_y_Bin_3.png}
        \caption{$Q^2$-$y$ Bin 3}
    \end{subfigure}%
    \begin{subfigure}{0.49\textwidth}\includegraphics[width=0.85\textwidth]{All_Images/Data_Collec

In [ ]:
TFile *f = TFile::Open("Unfolded_Histos_From_Just_RooUnfold_SIDIS_richcap_Lower_Acceptance_Cut.root");

// Bin ranges (index 0 unused so index = bin number)
const char* Q2_ranges[18] = {
    "", "2.00 < Q^{2} < 2.40", "2.00 < Q^{2} < 2.40", "2.00 < Q^{2} < 2.40", "2.00 < Q^{2} < 2.40",
    "2.40 < Q^{2} < 2.90", "2.40 < Q^{2} < 2.90", "2.40 < Q^{2} < 2.90", "2.40 < Q^{2} < 2.90",
    "2.90 < Q^{2} < 3.70", "2.90 < Q^{2} < 3.70", "2.90 < Q^{2} < 3.70", "2.90 < Q^{2} < 3.70",
    "3.70 < Q^{2} < 5.30", "3.70 < Q^{2} < 5.30", "3.70 < Q^{2} < 5.30",
    "5.30 < Q^{2} < 7.90", "5.30 < Q^{2} < 7.90"
};

const char* y_ranges[18] = {
    "", "0.75 < y < 0.65", "0.65 < y < 0.55", "0.55 < y < 0.45", "0.45 < y < 0.35",
    "0.75 < y < 0.65", "0.65 < y < 0.55", "0.55 < y < 0.45", "0.45 < y < 0.35",
    "0.75 < y < 0.65", "0.65 < y < 0.55", "0.55 < y < 0.45", "0.45 < y < 0.35",
    "0.75 < y < 0.65", "0.65 < y < 0.55", "0.55 < y < 0.45",
    "0.75 < y < 0.65", "0.65 < y < 0.55"
};

// Loop over the 17 histograms
for (int i = 1; i <= 17; i++) {
    TString hname = Form("(Response_Matrix)_(mdf)_(SMEAR=Smear)_(Q2_y_Bin_%d)_(z_pT_Bin_All)_(MultiDim_z_pT_Bin_Y_bin_phi_t)", i);
    TH2D *h = (TH2D*)f->Get(hname);
    if (!h) { printf("Missing: %s\n", hname.Data()); continue; }

    TCanvas *c = new TCanvas(Form("c%d", i), "", 1600, 1400);
    gStyle->SetOptStat(0);
    c->SetLogz();
    h->Draw("colz");

    // Now build the fancier title
    TString title = Form(
        "#splitline{#splitline{Response Matrix of 3D Kinematic Bins (z+P_{T}+#phi_{h})}"
        "{Q^{2}-y Bin: %d #topbar #splitline{%s}{%s}}}{#color[4]{Pass 2}}",
        i, Q2_ranges[i], y_ranges[i]
    );

    h->SetTitle(title);
    h->GetXaxis()->SetTitle("3D Kinematic Bins (z+P_{T}+#phi_{h} - GEN)");
    h->GetYaxis()->SetTitle("3D Kinematic Bins (z+P_{T}+#phi_{h} - REC)");

    TString outname = Form("3D_Response_Matrix_for_Q2_y_Bin_%d_Pass_2_Smeared.pdf", i);
    c->SaveAs(outname);
    delete c;
}
f->Close();


In [8]:
root_color.Blue

4

In [9]:
TFile *f = TFile::Open("Unfolded_Histos_From_Just_RooUnfold_SIDIS_richcap_Lower_Acceptance_Cut.root");
TH2D *h = (TH2D*)f->Get("(Response_Matrix)_(mdf)_(SMEAR=Smear)_(Q2_y_Bin_All)_(z_pT_Bin_All)_(phi_t)");
if (!h) {
    printf("Histogram not found!\n");
} else {
    TCanvas *c = new TCanvas("c_all", "", 800, 700);
    gStyle->SetOptStat(0);
    c->SetLogz();
    h->Draw("colz");
    h->SetTitle("#splitline{Response Matrix of #phi_{h}}{Pass 2 #topbar All Bins}");
    h->GetXaxis()->SetTitle("#phi_{h} (GEN)");
    h->GetYaxis()->SetTitle("#phi_{h} (REC)");
    c->SaveAs("1D_Response_Matrix_for_Q2_y_Bin_All_Pass_2_Smeared.pdf");
    delete c;
}
f->Close();


'#topbar'

In [8]:
len("""-rw-r--r--. 1 richcap clas12-grp  33G Sep 24 15:26 nb-clasdis-Q2_1.5-9688_0.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 24 17:14 nb-clasdis-Q2_1.5-9688_1.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 24 18:26 nb-clasdis-Q2_1.5-9688_2.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 24 19:10 nb-clasdis-Q2_1.5-9688_3.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 24 19:53 nb-clasdis-Q2_1.5-9688_4.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 24 20:35 nb-clasdis-Q2_1.5-9688_5.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 24 21:16 nb-clasdis-Q2_1.5-9688_6.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 24 21:58 nb-clasdis-Q2_1.5-9688_7.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 24 22:43 nb-clasdis-Q2_1.5-9688_8.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 24 23:28 nb-clasdis-Q2_1.5-9688_9.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 25 00:11 nb-clasdis-Q2_1.5-9690_0.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 25 00:55 nb-clasdis-Q2_1.5-9690_1.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 25 01:40 nb-clasdis-Q2_1.5-9690_2.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 25 02:23 nb-clasdis-Q2_1.5-9690_3.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 25 03:05 nb-clasdis-Q2_1.5-9690_4.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 25 03:50 nb-clasdis-Q2_1.5-9690_5.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 25 04:34 nb-clasdis-Q2_1.5-9690_6.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 25 05:16 nb-clasdis-Q2_1.5-9690_7.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 25 06:04 nb-clasdis-Q2_1.5-9690_8.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 25 06:45 nb-clasdis-Q2_1.5-9690_9.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 25 07:27 nb-clasdis-Q2_1.5-9656_0.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 25 08:08 nb-clasdis-Q2_1.5-9656_1.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 25 08:54 nb-clasdis-Q2_1.5-9656_2.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 25 09:41 nb-clasdis-Q2_1.5-9656_3.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 25 10:23 nb-clasdis-Q2_1.5-9656_4.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 25 11:13 nb-clasdis-Q2_1.5-9656_5.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 25 11:57 nb-clasdis-Q2_1.5-9656_6.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 25 12:42 nb-clasdis-Q2_1.5-9656_7.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 25 13:27 nb-clasdis-Q2_1.5-9656_8.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 25 14:13 nb-clasdis-Q2_1.5-9656_9.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 25 15:02 nb-clasdis-Q2_1.5-9662_0.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 25 15:52 nb-clasdis-Q2_1.5-9662_1.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 25 16:39 nb-clasdis-Q2_1.5-9662_2.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 25 17:26 nb-clasdis-Q2_1.5-9662_3.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 25 18:17 nb-clasdis-Q2_1.5-9662_4.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 25 19:01 nb-clasdis-Q2_1.5-9662_5.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 25 19:43 nb-clasdis-Q2_1.5-9662_6.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 25 20:24 nb-clasdis-Q2_1.5-9662_7.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 25 21:05 nb-clasdis-Q2_1.5-9662_8.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 25 21:55 nb-clasdis-Q2_1.5-9662_9.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 25 22:37 nb-clasdis-Q2_1.5-9678_0.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 25 23:20 nb-clasdis-Q2_1.5-9678_1.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 26 00:01 nb-clasdis-Q2_1.5-9678_2.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 26 00:42 nb-clasdis-Q2_1.5-9678_3.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 26 01:25 nb-clasdis-Q2_1.5-9678_4.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 26 02:06 nb-clasdis-Q2_1.5-9678_5.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 26 02:49 nb-clasdis-Q2_1.5-9678_6.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 26 03:29 nb-clasdis-Q2_1.5-9678_7.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 26 04:09 nb-clasdis-Q2_1.5-9678_8.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 26 04:50 nb-clasdis-Q2_1.5-9678_9.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 27 21:21 nb-clasdis-Q2_1.5-9689_0.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 27 22:00 nb-clasdis-Q2_1.5-9689_1.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 27 22:44 nb-clasdis-Q2_1.5-9689_2.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 27 23:27 nb-clasdis-Q2_1.5-9689_3.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 28 00:10 nb-clasdis-Q2_1.5-9689_4.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 28 00:51 nb-clasdis-Q2_1.5-9689_5.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 28 01:31 nb-clasdis-Q2_1.5-9689_6.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 28 02:11 nb-clasdis-Q2_1.5-9689_7.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 28 02:50 nb-clasdis-Q2_1.5-9689_8.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 28 03:30 nb-clasdis-Q2_1.5-9689_9.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 28 04:07 nb-clasdis-Q2_1.5-9691_0.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 28 04:47 nb-clasdis-Q2_1.5-9691_1.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 28 05:27 nb-clasdis-Q2_1.5-9691_2.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 28 06:06 nb-clasdis-Q2_1.5-9691_3.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 28 06:47 nb-clasdis-Q2_1.5-9691_4.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 28 07:28 nb-clasdis-Q2_1.5-9691_5.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 28 08:08 nb-clasdis-Q2_1.5-9691_6.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 28 08:48 nb-clasdis-Q2_1.5-9691_7.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 28 09:29 nb-clasdis-Q2_1.5-9691_8.hipo
-rw-r--r--. 1 richcap clas12-grp  33G Sep 28 10:10 nb-clasdis-Q2_1.5-9691_9.hipo""".split("\n"))

70

In [5]:
List_of_All_Histos_For_Unfolding = {}
Histo_New, Output_Fit_Function, List_of_Chi_Squared, List_of_Fit_Par_A, List_of_Fit_Par_B, List_of_Fit_Par_C = Fitting_Phi_Function(Histo_To_Fit=Histo_Original, Method=..., Special=[Q2_Y_Bin, Z_PT_Bin])
List_of_All_Histos_For_Unfolding[str(HISTO_NAME_Binned)]                                       = Histo_New.Clone(str(HISTO_NAME_Binned))
List_of_All_Histos_For_Unfolding[str(HISTO_NAME_Binned).replace(entry_type, "Fit_Function")]   = Output_Fit_Function.Clone(str(HISTO_NAME_Binned).replace(entry_type, "Fit_Function"))
List_of_All_Histos_For_Unfolding[str(HISTO_NAME_Binned).replace(entry_type, "Chi_Squared")]    = List_of_Chi_Squared
List_of_All_Histos_For_Unfolding[str(HISTO_NAME_Binned).replace(entry_type, "Fit_Par_A")]      = List_of_Fit_Par_A
List_of_All_Histos_For_Unfolding[str(HISTO_NAME_Binned).replace(entry_type, "Fit_Par_B")]      = List_of_Fit_Par_B
List_of_All_Histos_For_Unfolding[str(HISTO_NAME_Binned).replace(entry_type, "Fit_Par_C")]      = List_of_Fit_Par_C

NameError: name 'Fitting_Phi_Function' is not defined

In [ ]:
from functools import partial

def func_fit(params, x, y):
    A, B, C = params
    y_pred = [A*(1 + B*(ROOT.cos(xi)) + C*(ROOT.cos(2*xi))) for xi in x]
    return sum((y_pred[i] - y[i])**2 for i in range(len(x)))

def nelder_mead(func, x0, args=(), max_iter=1000, tol=1e-6):
    N = len(x0)
    simplex = [x0]
    for i in range(N):
        point = list(x0)
        point[i] = x0[i] + 1.0
        simplex.append(point)
    for _ in range(max_iter):
        simplex.sort(key=lambda point: func(point, *args))
        if(abs(func(simplex[0], *args) - func(simplex[-1], *args)) < tol):
            break
        centroid = [sum(simplex[i][j] for i in range(N)) / N for j in range(N)]
        reflected = [centroid[j] + (centroid[j] - simplex[-1][j]) for j in range(N)]
        if(func(simplex[0], *args) <= func(reflected, *args) < func(simplex[-2], *args)):
            simplex[-1] = reflected
            continue
        if(func(reflected, *args) < func(simplex[0], *args)):
            expanded = [centroid[j] + 2.0 * (centroid[j] - simplex[-1][j]) for j in range(N)]
            if(func(expanded, *args) < func(reflected, *args)):
                simplex[-1] = expanded
            else:
                simplex[-1] = reflected
            continue
        contracted = [centroid[j] + 0.5 * (simplex[-1][j] - centroid[j]) for j in range(N)]
        if(func(contracted, *args) < func(simplex[-1], *args)):
            simplex[-1] = contracted
            continue
        for i in range(1, N+1):
            simplex[i] = [simplex[0][j] + 0.5 * (simplex[i][j] - simplex[0][j]) for j in range(N)]
    return simplex[0]

def Full_Calc_Fit(Histo):
    # Helping the closure tests with known values of B and C
    # if(Closure_Test):
    #     B_opt, C_opt = -0.500, 0.025
    #     Histo_max_bin     = Histo.GetMaximumBin()
    #     Histo_max_bin_phi = (3.1415926/180)*Histo.GetBinCenter(Histo_max_bin)
    #     Histo_max_bin_num = Histo.GetBinContent(Histo_max_bin)
    #     A_opt    = (Histo_max_bin_num)/((1 + B_opt*ROOT.cos(Histo_max_bin_phi) + C_opt*ROOT.cos(2*Histo_max_bin_phi)))
    # elif(Sim_Test):
    #     B_opt, C_opt = 0, 0
    #     Histo_max_bin     = Histo.GetMaximumBin()
    #     Histo_max_bin_phi = (3.1415926/180)*Histo.GetBinCenter(Histo_max_bin)
    #     Histo_max_bin_num = Histo.GetBinContent(Histo_max_bin)
    #     A_opt    = (Histo_max_bin_num)/((1 + B_opt*ROOT.cos(Histo_max_bin_phi) + C_opt*ROOT.cos(2*Histo_max_bin_phi)))
    # else:
    x_data, y_data = [], []
    try:
        for ii in range(0, Histo.GetNbinsX(), 1):
            x_data.append(Histo.GetBinCenter(ii))
            y_data.append(Histo.GetBinContent(ii))

        # Perform optimization using the Nelder-Mead method
        initial_guess = [1e6, 1, 1]  # Initial guess for A, B, C
        optim_params = nelder_mead(partial(func_fit, x=x_data, y=y_data), initial_guess)

        # Extract the optimized parameters
        A_opt, B_opt, C_opt = optim_params
    except:
        print(f"{color.Error}Full_Calc_Fit(...) ERROR:\n{color.END}{traceback.format_exc()}\n\n{color.Error}ERROR is with 'Histo' = {Histo}\n{color.END}")
        A_opt, B_opt, C_opt = "Error", "Error", "Error"
        
    return [A_opt, B_opt, C_opt]
    

from Phi_h_Fit_Parameters_Initialize import special_fit_parameters_set
def Fitting_Phi_Function(Histo_To_Fit, Method="FIT", Special="Normal", Overwrite_Fit_Test=Fit_Test):
    if((Method in ["RC"]) and Overwrite_Fit_Test):
        try:
            Q2_y_Bin_Special, z_pT_Bin_Special = str(Special[0]), str(Special[1])
            fit_function = "[A]*(1 + [B]*cos(x*(3.1415926/180)) + [C]*cos(2*x*(3.1415926/180)))"
            Fitting_Function = ROOT.TF1(f"Fitting_Function_of_{Histo_To_Fit.GetName()}_{Method}", fit_function, 0, 360)
            RC_Par_A, RC_Err_A, RC_Par_B, RC_Err_B, RC_Par_C, RC_Err_C = Find_RC_Fit_Params(Q2_y_bin=Q2_y_Bin_Special, z_pT_bin=z_pT_Bin_Special, root_in="/w/hallb-scshelf2102/clas12/richcap/Radiative_MC/SIDIS_RC_EvGen_richcap/Running_EvGen_richcap/RC_Cross_Section_Scan_Outputs_Final.root", cache_in=None, cache_out=None, quiet=True)
            Fitting_Function.SetParameter(0, RC_Par_A)
            Fitting_Function.SetParLimits(0, RC_Par_A - 2*abs(RC_Err_A), RC_Par_A + 2*abs(RC_Err_A))
            Fitting_Function.SetParameter(1, RC_Par_B)
            Fitting_Function.SetParLimits(1, RC_Par_B - 2*abs(RC_Err_B), RC_Par_B + 2*abs(RC_Err_B))
            Fitting_Function.SetParameter(2, RC_Par_C)
            Fitting_Function.SetParLimits(2, RC_Par_C - 2*abs(RC_Err_C), RC_Par_C + 2*abs(RC_Err_C))
            Fitting_Function.SetParName(0, "A")
            Fitting_Function.SetParName(1, "B")
            Fitting_Function.SetParName(2, "C")
            Histo_To_Fit.Fit(Fitting_Function, "QRB")
            A_Unfold       = Fitting_Function.GetParameter(0)
            B_Unfold       = Fitting_Function.GetParameter(1)
            C_Unfold       = Fitting_Function.GetParameter(2)
            A_Unfold_Error = Fitting_Function.GetParError(0)
            B_Unfold_Error = Fitting_Function.GetParError(1)
            C_Unfold_Error = Fitting_Function.GetParError(2)
            try:
                Fit_Chisquared = Fitting_Function.GetChisquare()
                Fit_ndf        = Fitting_Function.GetNDF()
            except:
                Fit_Chisquared = "Fit_Chisquared"
                Fit_ndf        = "Fit_ndf"
            Out_Put = [Histo_To_Fit,  Fitting_Function,   [Fit_Chisquared,    Fit_ndf],  [A_Unfold,    A_Unfold_Error],  [B_Unfold,    B_Unfold_Error],  [C_Unfold,    C_Unfold_Error]]
        except:
            print(f"{color.Error}ERROR IN FITTING:\n{color.END}{str(traceback.format_exc())}\n")
            Out_Put = [Histo_To_Fit, "Fitting_Function",  ["Fit_Chisquared", "Fit_ndf"], ["A_Unfold", "A_Unfold_Error"], ["B_Unfold", "B_Unfold_Error"], ["C_Unfold", "C_Unfold_Error"]]
        return Out_Put
    elif((Method in ["gdf", "gen", "MC GEN", "bbb", "Bin", "Bin-by-Bin", "Bin-by-bin", "bay", "bayes", "bayesian", "Bayesian", "FIT", "SVD", "tdf", "true", "RC_Bin", "RC_Bayesian"]) and Overwrite_Fit_Test):
        A_Unfold, B_Unfold, C_Unfold = Full_Calc_Fit(Histo_To_Fit)
        fit_function = "[A]*(1 + [B]*cos(x*(3.1415926/180)) + [C]*cos(2*x*(3.1415926/180)))"
        
        Fitting_Function = ROOT.TF1(f"Fitting_Function_of_{Histo_To_Fit.GetName()}_{str(Method).replace(' ', '_')}", str(fit_function), 0, 360)
        # Fitting_Function.SetParName(0, "Parameter A")
        # Fitting_Function.SetParName(1, "Parameter B")
        # Fitting_Function.SetParName(2, "Parameter C")

        fit_range_lower = 0
        fit_range_upper = 360
        # Number of bins in the histogram
        n_bins = Histo_To_Fit.GetNbinsX()
        
        # Find the lower fit range (first non-empty bin)
        for bin_lower in range(1, n_bins // 2 + 1):  # Search from the start to the center
            if(Histo_To_Fit.GetBinContent(bin_lower) != 0):
                fit_range_lower = Histo_To_Fit.GetXaxis().GetBinLowEdge(bin_lower)
                break  # Stop the loop once the first non-empty bin is found

        # Find the upper fit range (last non-empty bin)
        for bin_upper in range(n_bins, n_bins // 2, -1):  # Search from the end towards the center
            if(Histo_To_Fit.GetBinContent(bin_upper) != 0):
                fit_range_upper = Histo_To_Fit.GetXaxis().GetBinUpEdge(bin_upper)
                break  # Stop the loop once the last non-empty bin is found
        
        
        Fitting_Function.SetRange(fit_range_lower, fit_range_upper)
        
        Fitting_Function.SetLineColor(2)
        if(Special in ["Normal"]):
            if(Method in ["rdf", "Experimental"]):
                Fitting_Function.SetLineColor(root_color.Blue)
            if(Method in ["mdf", "MC REC"]):
                Fitting_Function.SetLineColor(root_color.Red)
            if(Method in ["gdf", "gen", "MC GEN"]):
                Fitting_Function.SetLineColor(root_color.Green)
            if(Method in ["tdf", "true"]):
                Fitting_Function.SetLineColor(root_color.Cyan)
            if(Method in ["bbb", "Bin", "Bin-by-Bin", "Bin-by-bin"]):
                Fitting_Function.SetLineColor(root_color.Brown)
            if(Method in ["bayes", "bayesian", "Bayesian", "bay"]):
                Fitting_Function.SetLineColor(root_color.Teal)
            if(Method in ["RC_Bin"]):
                Fitting_Function.SetLineColor(ROOT.kOrange + 4)
            if(Method in ["RC_Bayesian"]):
                Fitting_Function.SetLineColor(ROOT.kViolet - 8)
            if(Method in ["SVD"]):
                Fitting_Function.SetLineColor(root_color.Pink)
        
        Allow_Multiple_Fits   = True
        Allow_Multiple_Fits_C = True

        try:
            if("Error" not in [A_Unfold, B_Unfold, C_Unfold]):
                # This is the constant scaling factor - A (should basically always be positive)
                Fitting_Function.SetParameter(0,      abs(A_Unfold))
                Fitting_Function.SetParLimits(0, 0.05*abs(A_Unfold), 5.5*abs(A_Unfold))

                # Cos(phi) Moment - B
                Fitting_Function.SetParameter(1, B_Unfold)
                Fitting_Function.SetParLimits(1, B_Unfold - 5.5*abs(B_Unfold), B_Unfold + 5.5*abs(B_Unfold))

                # Cos(2*phi) Moment - C
                Fitting_Function.SetParameter(2, C_Unfold)
                Fitting_Function.SetParLimits(2, C_Unfold - 5.5*abs(C_Unfold), C_Unfold + 5.5*abs(C_Unfold))

                if(((Special not in ["Normal"]) and isinstance(Special, list)) and (not args.closure)):
                    try:
                        Q2_y_Bin_Special, z_pT_Bin_Special = str(Special[0]), str(Special[1])
                        # print(f"Fitting_Phi_Function Special: Q2_y_Bin_Special, z_pT_Bin_Special = {Q2_y_Bin_Special}, {z_pT_Bin_Special}")
                        if(len(Special) > 2):
                            Sector_Special = str(Special[2])
                        else:
                            Sector_Special = "N/A"
                        if(str(z_pT_Bin_Special) in ["All", "0"]):
                            print(f"\n\n{color.BBLUE}z_pT_Bin_Special = {z_pT_Bin_Special}{color.END}\n\n\n")
                        if(str(z_pT_Bin_Special) in ["Integrated", "All", "-1", "0"]):
                            if(Sector_Special not in ["N/A"]):
                                if((Q2_y_Bin_Special, z_pT_Bin_Special, "Sectors", "Trusted") in special_fit_parameters_set):
                                    bin_ranges = special_fit_parameters_set[(Q2_y_Bin_Special, z_pT_Bin_Special, "Sectors", "Trusted")]
                                    print(f"\n{color.Error}Using Sector Fit Ranges\n{color.END}")
                                    fit_range_lower = bin_ranges.get("fit_range_lower")
                                    fit_range_upper = bin_ranges.get("fit_range_upper")
                                elif((Q2_y_Bin_Special, "All", "Sectors", "Trusted") in special_fit_parameters_set):
                                    bin_ranges = special_fit_parameters_set[(Q2_y_Bin_Special, "All", "Sectors", "Trusted")]
                                    print(f"\n{color.Error}Using Sector Fit Ranges\n{color.END}")
                                    fit_range_lower = bin_ranges.get("fit_range_lower")
                                    fit_range_upper = bin_ranges.get("fit_range_upper")
                            elif((Q2_y_Bin_Special, z_pT_Bin_Special, "Trusted") in special_fit_parameters_set):
                                bin_ranges = special_fit_parameters_set[(Q2_y_Bin_Special, z_pT_Bin_Special, "Trusted")]
                                fit_range_lower = bin_ranges.get("fit_range_lower")
                                fit_range_upper = bin_ranges.get("fit_range_upper")
                            elif((Q2_y_Bin_Special, "All", "Trusted") in special_fit_parameters_set):
                                bin_ranges = special_fit_parameters_set[(Q2_y_Bin_Special, "All", "Trusted")]
                                fit_range_lower = bin_ranges.get("fit_range_lower")
                                fit_range_upper = bin_ranges.get("fit_range_upper")
                            else:
                                fit_range_lower =  45 if(str(Q2_y_Bin_Special) in ["1", "5"]) else  30 if(str(Q2_y_Bin_Special) in ["2", "3", "6", "9", "10", "13", "16"]) else  15
                                fit_range_upper = 315 if(str(Q2_y_Bin_Special) in ["1", "5"]) else 330 if(str(Q2_y_Bin_Special) in ["2", "3", "6", "9", "10", "13", "16"]) else 345
                            Fitting_Function.SetRange(fit_range_lower, fit_range_upper)
                        if((z_pT_Bin_Special in ["Integrated", "-1"]) and not ((Q2_y_Bin_Special, z_pT_Bin_Special) in special_fit_parameters_set)):
                            if((Q2_y_Bin_Special, "All") in special_fit_parameters_set):
                                bin_settings = special_fit_parameters_set[(Q2_y_Bin_Special, "All")]
                                if(bin_settings.get("B_initial") is not None):
                                    Fitting_Function.SetParameter(1, bin_settings["B_initial"])
                                    if(bin_settings.get("B_limits")):
                                        Fitting_Function.SetParLimits(1, *sorted(bin_settings["B_limits"]))
                                if(bin_settings.get("C_initial") is not None):
                                    Fitting_Function.SetParameter(2, bin_settings["C_initial"])
                                    if(bin_settings.get("C_limits")):
                                        Fitting_Function.SetParLimits(2, *sorted(bin_settings["C_limits"]))
                                Allow_Multiple_Fits   = bin_settings.get("Allow_Multiple_Fits",   True)
                                Allow_Multiple_Fits_C = bin_settings.get("Allow_Multiple_Fits_C", True)
                            else:
                                Par_initial_B = -0.05 if(str(Q2_y_Bin_Special) in ["12"]) else -0.1   if(str(Q2_y_Bin_Special) in ["8", "15"]) else -0.12 if(str(Q2_y_Bin_Special) in ["4", "11", "17"]) else -0.14  if(str(Q2_y_Bin_Special) in ["7", "16"]) else -0.155 if(str(Q2_y_Bin_Special) in ["3",  "6", "13", "14"]) else -0.1625 if(str(Q2_y_Bin_Special) in ["12"]) else -0.174
                                Par_initial_C = -0.05 if(str(Q2_y_Bin_Special) in ["8"])  else -0.075 if(str(Q2_y_Bin_Special) in ["12"])      else -0.04 if(str(Q2_y_Bin_Special) in ["4", "15"])       else -0.029 if(str(Q2_y_Bin_Special) in ["11"])      else -0.021 if(str(Q2_y_Bin_Special) in ["7", "14", "16", "17"]) else -0.01   if(str(Q2_y_Bin_Special) in ["10"]) else -0.006 if(str(Q2_y_Bin_Special) in ["3", "6", "13"]) else 0.0045 if(str(Q2_y_Bin_Special) in ["2"]) else 0.0067 if(str(Q2_y_Bin_Special) in ["9"]) else 0.009
                                Par__range__B = [0.5*Par_initial_B, 1.5*Par_initial_B]
                                Par__range__C = [0.5*Par_initial_C, 1.5*Par_initial_C]
                                # Cos(phi) Moment - B
                                Fitting_Function.SetParameter(1, Par_initial_B)
                                Fitting_Function.SetParLimits(1, min(Par__range__B), max(Par__range__B))
                                # Cos(2*phi) Moment - C
                                Fitting_Function.SetParameter(2, Par_initial_C)
                                Fitting_Function.SetParLimits(2, min(Par__range__C), max(Par__range__C))
                        elif((Q2_y_Bin_Special, z_pT_Bin_Special) in special_fit_parameters_set):
                                if(("RC_" in Method) and ((Q2_y_Bin_Special, z_pT_Bin_Special, "RC") in special_fit_parameters_set)):
                                    print(f"\n{color.BCYAN}Using RC Initial Fit Configs for Bin ({Q2_y_Bin_Special}-{z_pT_Bin_Special}){color.END}\n")
                                    bin_settings = special_fit_parameters_set[(Q2_y_Bin_Special, z_pT_Bin_Special, "RC")]
                                else:
                                    bin_settings = special_fit_parameters_set[(Q2_y_Bin_Special, z_pT_Bin_Special)]
                                if(bin_settings.get("B_initial") is not None):
                                    Fitting_Function.SetParameter(1, bin_settings["B_initial"])
                                    if(bin_settings.get("B_limits")):
                                        Fitting_Function.SetParLimits(1, *sorted(bin_settings["B_limits"]))
                                if(bin_settings.get("C_initial") is not None):
                                    Fitting_Function.SetParameter(2, bin_settings["C_initial"])
                                    if(bin_settings.get("C_limits")):
                                        Fitting_Function.SetParLimits(2, *sorted(bin_settings["C_limits"]))
                                Allow_Multiple_Fits   = bin_settings.get("Allow_Multiple_Fits",   True)
                                Allow_Multiple_Fits_C = bin_settings.get("Allow_Multiple_Fits_C", True)                        
                    except:
                        print(f"{color.Error}\nERROR in Fitting_Phi_Function() for 'Special' arguement...{color.END_B}\nTraceback:\n{str(traceback.format_exc())}{color.END}\n")
                else:
                    print(f"\n\n\n{color.RED}Fitting_Phi_Function Not Special{color.END}\n\n\n")
                Histo_To_Fit.Fit(Fitting_Function, "QRB")
            else:
                # print(f"{color.RED}Error in A_Unfold, B_Unfold, C_Unfold = {A_Unfold}, {B_Unfold}, {C_Unfold}{color.END}")
                Histo_To_Fit.Fit(Fitting_Function, "QR")

            A_Unfold = Fitting_Function.GetParameter(0)
            B_Unfold = Fitting_Function.GetParameter(1)
            C_Unfold = Fitting_Function.GetParameter(2)
            
            # Re-fitting with the new parameters
            # The constant scaling factor - A
            Fitting_Function.SetParameter(0,     abs(A_Unfold))
            Fitting_Function.SetParLimits(0, 0.5*abs(A_Unfold), 1.5*abs(A_Unfold))
            
            if(Allow_Multiple_Fits): # Cos(phi) Moment - B
                Fitting_Function.SetParameter(1, B_Unfold)
                Fitting_Function.SetParLimits(1, B_Unfold - 0.5*abs(B_Unfold), B_Unfold + 0.5*abs(B_Unfold))
            else:                    # Cos(phi) Moment - B
                Fitting_Function.SetParameter(1, B_Unfold)
                Fitting_Function.SetParLimits(1, B_Unfold - Fitting_Function.GetParError(1), B_Unfold + Fitting_Function.GetParError(1))
                
            if(Allow_Multiple_Fits_C): # Cos(2*phi) Moment - C
                Fitting_Function.SetParameter(2, C_Unfold)
                Fitting_Function.SetParLimits(2, C_Unfold - 0.5*abs(C_Unfold), C_Unfold + 0.5*abs(C_Unfold))
            else:                      # Cos(2*phi) Moment - C
                Fitting_Function.SetParameter(2, C_Unfold)
                Fitting_Function.SetParLimits(2, C_Unfold - Fitting_Function.GetParError(2), C_Unfold + Fitting_Function.GetParError(2))

            # Re-Fitting the plots
            Histo_To_Fit.Fit(Fitting_Function, "QRB")

            A_Unfold       = Fitting_Function.GetParameter(0)
            B_Unfold       = Fitting_Function.GetParameter(1)
            C_Unfold       = Fitting_Function.GetParameter(2)

            A_Unfold_Error = Fitting_Function.GetParError(0)
            B_Unfold_Error = Fitting_Function.GetParError(1)
            C_Unfold_Error = Fitting_Function.GetParError(2)
            
            try:
                Fit_Chisquared = Fitting_Function.GetChisquare()
                Fit_ndf        = Fitting_Function.GetNDF()
            except:
                Fit_Chisquared = "Fit_Chisquared"
                Fit_ndf        = "Fit_ndf"

            Out_Put = [Histo_To_Fit,  Fitting_Function,   [Fit_Chisquared,    Fit_ndf],  [A_Unfold,    A_Unfold_Error],  [B_Unfold,    B_Unfold_Error],  [C_Unfold,    C_Unfold_Error]]
        except:
            print(f"{color.Error}ERROR IN FITTING:\n{color.END}{str(traceback.format_exc())}\n")
            Out_Put = [Histo_To_Fit, "Fitting_Function",  ["Fit_Chisquared", "Fit_ndf"], ["A_Unfold", "A_Unfold_Error"], ["B_Unfold", "B_Unfold_Error"], ["C_Unfold", "C_Unfold_Error"]]
        return Out_Put
    else:
        print(f"\n\n\n{color.Error}ERROR WITH Fitting_Phi_Function()\n\t'Method' is not selected for proper output...\n\n\n{color.END}")
        return "ERROR"
    
################################################################################################################################################################################################################################################
##==========##==========##     Fitting Function For Phi Plots                     ##==========##==========##==========##==========##==========##==========##==========##==========##==========##==========##==========##==========##==========##
################################################################################################################################################################################################################################################

def Save_Fit_Outputs_To_ROOT(Histo_Name, Chi_List, ParA_List, ParB_List, ParC_List, Histo_New=None, Fit_Function=None):
    # If no output file requested, do nothing
    if(args.no_fit or (args.fit_root in [None, ""])):
        return

    # Decide whether to UPDATE or RECREATE
    tfile = ROOT.TFile.Open(args.fit_root, "UPDATE" if(os.path.exists(fit_path)) else "RECREATE")
    if((not tfile) or tfile.IsZombie()):
        print(f"""{color.Error}ERROR:{color.END} Could not open fit-output file '{args.fit_root}' in mode '{"UPDATE" if(os.path.exists(args.fit_root)) else "RECREATE"}'.""")
        return

    # Determine the entry_type prefix
    Histo_Name_str = str(Histo_Name)
    entry_type = "(1D)_" if(Histo_Name_str.startswith("(1D)_")) else "(MultiDim_3D_Histo)_" if(Histo_Name_str.startswith("(MultiDim_3D_Histo)_")) else "(MultiDim_5D_Histo)_"
    
    Fit_Function_Name    = Histo_Name_str.replace(str(entry_type), "(Fit_Function)_")
    Chi_Squared_Name     = Histo_Name_str.replace(str(entry_type), "(Chi_Squared)_")
    Fit_Par_A_List_Name  = Histo_Name_str.replace(str(entry_type), "(Fit_Par_A)_")
    Fit_Par_B_List_Name  = Histo_Name_str.replace(str(entry_type), "(Fit_Par_B)_")
    Fit_Par_C_List_Name  = Histo_Name_str.replace(str(entry_type), "(Fit_Par_C)_")

    # 1) Overwrite the fitted histogram itself
    if(Histo_New):
        histo_out = Histo_New.Clone(Histo_Name_str)
        histo_out.SetName(Histo_Name_str)
        safe_write(histo_out, tfile)

    # 2) Fit function
    if(Fit_Function):
        fit_out = Fit_Function.Clone(Fit_Function_Name)
        fit_out.SetName(Fit_Function_Name)
        safe_write(fit_out, tfile)

    # Helper: Python list -> TVectorD
    def write_vec(name, py_list):
        if(py_list is None):
            return
        vec = ROOT.TVectorD(len(py_list))
        for i, val in enumerate(py_list):
            vec[i] = float(val)
        vec.SetName(f"TVectorD_{name}")
        safe_write(vec, tfile)

    # 3) Lists (as TVectorD) with the requested names
    write_vec(Chi_Squared_Name,    Chi_List)
    write_vec(Fit_Par_A_List_Name, ParA_List)
    write_vec(Fit_Par_B_List_Name, ParB_List)
    write_vec(Fit_Par_C_List_Name, ParC_List)

    tfile.Close()


def Compare_TH1D_Histograms(ROOT_In_1, HISTO_NAME_1, ROOT_In_2, HISTO_NAME_2, legend_labels=("Histogram 1", "Histogram 2"), output_prefix="Compare_", SAVE=args.save_name, Format=args.file_format, TITLE=Standard_Histogram_Title_Addition, Q2y_str="1", zPT_str="1", Unfolding_Diff_Data_In=Unfolding_Diff_Data, Return_Histos=False):
    # Retrieve histograms
    if(args.remake and (args.mod or args.sim)):
        output_prefix = f"Resummed_Weights_{output_prefix}"
        if(args.mod):
            histo1, histo2 = Make_New_BbB_Mod(ROOT_IN_NORMAL=ROOT_In_1, ROOT_IN_MODULATED=ROOT_In_2, HISTO_NAME_INPUT=HISTO_NAME_1)
        elif(args.sim):
            histo1, histo2 = Make_New_BbB_Sim(ROOT_IN_MODULATED=ROOT_In_1, HISTO_NAME_INPUT=HISTO_NAME_1)
        else:
            print(f"{color.Error}ERROR:{color.END} Could not remake the histograms for the test requested.\n\t{color.BOLD}Returning default outputs")
            histo1 = ROOT_In_1.Get(HISTO_NAME_1)
            histo2 = ROOT_In_2.Get(HISTO_NAME_2)
    else:
        histo1 = ROOT_In_1.Get(HISTO_NAME_1)
        histo2 = ROOT_In_2.Get(HISTO_NAME_2)
    
    histo1.SetStats(0)
    histo2.SetStats(0)
    # Ensure both exist
    if((not histo1) or (not histo2)):
        print(f"{color.Error}ERROR:{color.END} Could not retrieve one or both histograms.")
        if(not Return_Histos):
            return False, Unfolding_Diff_Data_In
        else:
            return histo1, histo2, None, Unfolding_Diff_Data_In
    # Ensure both are TH1D
    if((not histo1.InheritsFrom("TH1D")) or (not histo2.InheritsFrom("TH1D"))):
        print(f"{color.Error}ERROR:{color.END} Both histograms must be TH1D.")
        if(not Return_Histos):
            return False, Unfolding_Diff_Data_In
        else:
            return histo1, histo2, None, None, Unfolding_Diff_Data_In
    
    normalization_to_histo1, normalization_to_histo2 = 1.0, 1.0
    if(args.normalize):
        histo1, histo2, _, normalization_to_histo1, normalization_to_histo2 = Normalize_To_Shared_Bins(histo1, histo2, threshold=0.0, include_under_over=False, name_suffix="_NormShared")
    
    if(("Pass 2" in histo1.GetTitle()) and ((TITLE not in histo1.GetTitle()) and (TITLE not in histo2.GetTitle()))):
        histo1.SetTitle(str(histo1.GetTitle()).replace("Pass 2", TITLE))
    elif((TITLE not in histo1.GetTitle()) and (TITLE not in histo2.GetTitle())):
        histo1.SetTitle(f"#splitline{{{histo1.GetTitle()}}}{{{TITLE}}}")

    if(histo1.GetLineColor() == histo2.GetLineColor()):
        histo2.SetLineColor((histo2.GetLineColor() + 2) if(histo2.GetLineColor() != 28) else 26)
    histo1.SetLineWidth(3)
    histo2.SetLineWidth(2)
    
    g_asym = None
    if(args.use_errors):
        g_asym = ROOT.TGraphAsymmErrors(histo1)
        g_asym.SetName(f"{histo1.GetName()}_AsymErr")
        g_asym.SetLineColor(histo1.GetLineColor())
        g_asym.SetMarkerColor(histo1.GetMarkerColor())
        g_asym.SetLineWidth(3)

    # Clone one histogram to create the difference histogram
    h_diff = histo1.Clone(f"{HISTO_NAME_1}_vs_{HISTO_NAME_2}_absdiff")
    h_diff.Reset("ICES")  # clear contents, keep errors and structure
    h_diff.SetStats(0)

    h_uncertainty = h_diff.Clone(f"{HISTO_NAME_1}_Modeled_Uncertainty")
    h_uncertainty.Reset("ICES")  # clear contents, keep errors and structure
    h_uncertainty.SetStats(0)

    histo_key = f"{Q2y_str}_{zPT_str}"
    # Initialize the dictionary entry if it doesn't exist
    if(histo_key not in Unfolding_Diff_Data_In):
        Unfolding_Diff_Data_In[histo_key] = []
    max_content = 0
    max_cd_1    = 0
    max_M_uncer = 0
    # Fill it with |bin1 - bin2|
    for bin_idx in range(1, histo1.GetNbinsX() + 1):
        val1        = histo1.GetBinContent(bin_idx)
        val2        = histo2.GetBinContent(bin_idx)
        err1        = histo1.GetBinError(bin_idx)
        err2        = histo2.GetBinError(bin_idx)
        max_cd_1    = max([max_cd_1, val1 + err1, val2 + err2])
        diff        = abs(val1 - val2)
        # err         = math.sqrt(err1**2 + err2**2)
        err         = math.sqrt(max([err1**2 - err2**2, 0]))
        # M_uncer     = math.sqrt(max([0, (val2 - val1)**2 - err**2]))
        M_uncer     = diff
        max_content = max([max_content, diff])
        max_M_uncer = max([max_M_uncer, M_uncer])
        # max_content = max([max_content, diff + err])
        h_diff.SetBinContent(bin_idx,           diff)
        h_diff.SetBinError(bin_idx,              err)
        h_uncertainty.SetBinContent(bin_idx, M_uncer)
        h_uncertainty.SetBinError(bin_idx,       0.0)
        Unfolding_Diff_Data_In[histo_key].append({"phi_bin": bin_idx, f"{legend_labels[0]} — Value": val1, f"{legend_labels[0]} — Error": err1, f"{legend_labels[1]} — Value": val2, f"{legend_labels[1]} — Error": err2, "diff": diff, "err": err, "uncertainty": M_uncer, "scale_to_nominal": normalization_to_histo1, "scale_to_variation": normalization_to_histo2})
        # if(args.use_errors and args.mod):
        #     histo1.SetBinError(bin_idx, math.sqrt(err1**2 + (diff + err)**2))
        # if(args.use_errors):
        #     histo1.SetBinError(bin_idx, math.sqrt(err1**2 + diff**2))
        # if(args.use_errors):
        #     histo1.SetBinError(bin_idx, math.sqrt(err1**2 + M_uncer**2))
        if(args.use_errors):
            # Default: purely statistical
            low_err  = diff if((diff > err1) and (val1 > val2)) else err1
            high_err = diff if((diff > err1) and (val1 < val2)) else err1
            histo1.SetBinError(bin_idx, max([low_err, high_err]))
            if(g_asym):
                g_asym.SetPointEYlow(bin_idx  - 1,  low_err)
                g_asym.SetPointEYhigh(bin_idx - 1, high_err)
    if(args.use_errors and g_asym):
        histo1.asym_errors = g_asym

    h_diff.GetYaxis().SetRangeUser(0,        1.2*max_content)
    h_uncertainty.GetYaxis().SetRangeUser(0, 1.2*max_M_uncer)
    histo1.GetYaxis().SetRangeUser(0,        1.2*max_cd_1)
    histo2.GetYaxis().SetRangeUser(0,        1.2*max_cd_1)
    
    h_diff.SetLineColor(ROOT.kBlack)
    h_diff.SetLineWidth(2)
    h_diff.SetTitle(f"#splitline{{Absolute Bin Content Differences between}}{{{root_color.Bold}{{{legend_labels[0]}}} and {root_color.Bold}{{{legend_labels[1]}}}}}")
    h_diff.GetYaxis().SetTitle("#Delta Bin Contents")

    h_uncertainty.SetLineColor(ROOT.kOrange + 2)
    h_uncertainty.SetLineWidth(2)
    h_uncertainty.SetTitle(f"#splitline{{Modeled Uncertainty between}}{{{root_color.Bold}{{{legend_labels[0]}}} and {root_color.Bold}{{{legend_labels[1]}}}}}")
    if(args.use_errors):
        h_uncertainty.SetTitle(f"#splitline{{{h_uncertainty.GetTitle()}}}{{#scale[1.25]{{#splitline{{These uncertainties have been propagated to}}{{{root_color.Bold}{{{legend_labels[0]}}}}}}}}}")
    h_uncertainty.GetYaxis().SetTitle("Model Uncertainty")

    if(Return_Histos):
        return histo1, histo2, h_diff, h_uncertainty, Unfolding_Diff_Data_In
    else:
        # Create canvas
        canvas_name = f"c_{output_prefix}{HISTO_NAME_1}_vs_{HISTO_NAME_2}"
        c = ROOT.TCanvas(canvas_name, canvas_name, 2000, 700)
        c.Divide(3, 1)
    
        # Pad 1: Overlay the two histograms
        c.cd(1)
        if(hasattr(histo1, "asym_errors")):
            histo1.Draw("H P")
            histo1.asym_errors.Draw("P E SAME")
        else:
            histo1.Draw("H P E0")

        histo2.Draw("H P E0 SAME")
    
        # Add legend
        legend = ROOT.TLegend(0.45, 0.15, 0.7, 0.35)
        legend.SetBorderSize(1)
        legend.SetFillStyle(0)
        legend.AddEntry(histo1, f"#scale[1.75]{{{legend_labels[0]}}}", "APL E")
        legend.AddEntry(histo2, f"#scale[1.75]{{{legend_labels[1]}}}", "APL E")
        legend.Draw()

        # Pad 2: Draw the Model Uncertainties
        c.cd(2)
        h_uncertainty.Draw("H P E0")
        
        # Pad 3: Draw the absolute difference
        c.cd(3)
        h_diff.Draw("H P E0")
    
        # Save
        Save_Name = f"{output_prefix}{HISTO_NAME_1}{SAVE}{Format}"
        for replace in ["(", ")", "'", '"', "'"]:
            Save_Name = Save_Name.replace(replace, "")
        Save_Name = Save_Name.replace("__", "_")
        Save_Name = Save_Name.replace("SMEAR=Smear", "Smeared")
        Save_Name = Save_Name.replace("SMEAR=_", "")
        Save_Name = Save_Name.replace("__", "_")
        Save_Name = Save_Name.replace("_.", ".")
        c.SaveAs(Save_Name)
        print(f"Saved comparison as: {Save_Name}")
        return True, Unfolding_Diff_Data_In









def z_pT_Images_Together_For_Comparisons(ROOT_Input_In=None, ROOT_Mod_In=None, Unfolding_Diff_Data_Input={}, Q2_Y_Bin=1, Plot_Orientation="z_pT"):
    HISTO_NAME = f"\n{color.ERROR}ERROR{color.END}\n"
    if(args.dimensions   in ["1D"]):
        HISTO_NAME  = f"(1D)_({args.unfold})_(SMEAR={args.smearing_option})_(Q2_y_Bin_{Q2_Y_Bin})_(z_pT_Bin_ALL)_(phi_t)"
    elif(args.dimensions in ["3D", "MultiDim_3D_Histo"]):
        HISTO_NAME  = f"(MultiDim_3D_Histo)_({args.unfold})_(SMEAR={args.smearing_option})_(Q2_y_Bin_{Q2_Y_Bin})_(z_pT_Bin_ALL)_(MultiDim_z_pT_Bin_Y_bin_phi_t)"

    Canvases_to_Make = [HISTO_NAME]
    HISTO_True = "ERROR"
    Legend_Labels = [f"Plots of {args.dimensions} {args.unfold}"]
    if(args.mod):
        Canvases_to_Make = [f"Mod_Test_{HISTO_NAME}", f"Mod_Test_DIFF_{HISTO_NAME}", f"Mod_Test_UNCERTAINTY_{HISTO_NAME}"]
        Legend_Labels    = ["Unfolded with Normal MC", "Unfolded with Modulated MC"]
    elif(args.sim or args.closure):
        Canvases_to_Make = [f"Sim_Test_{HISTO_NAME}", f"Sim_Test_DIFF_{HISTO_NAME}", f"Sim_Test_UNCERTAINTY_{HISTO_NAME}"] if(not args.closure) else [f"Closure_Test_{HISTO_NAME}", f"Closure_Test_DIFF_{HISTO_NAME}", f"Closure_Test_UNCERTAINTY_{HISTO_NAME}"]
        Legend_Labels    = ["Unfolded Synthetic (MC) Data", "True Distribution of Synthetic Data"]
        if(args.dimensions   in ["1D"]):
            HISTO_True   = f"(1D)_(tdf)_(SMEAR=Smear)_(Q2_y_Bin_{Q2_y_BIN_NUM})_(z_pT_Bin_ALL)_(phi_t)"
        elif(args.dimensions in ["3D", "MultiDim_3D_Histo"]):
            HISTO_True   = f"(MultiDim_3D_Histo)_(tdf)_(SMEAR='')_(Q2_y_Bin_{Q2_y_BIN_NUM})_(z_pT_Bin_ALL)_(MultiDim_z_pT_Bin_Y_bin_phi_t)"
    if(args.single_file):
        HISTO_NAME = f"{HISTO_NAME}{'_(Closure_Test)' if(args.closure) else '_(Sim_Test)' if(args.sim) else ''}"
        # HISTO_NAME = f"{HISTO_NAME}{'_(Mod_Test)' if(args.mod) else '_(Closure_Test)' if(args.closure) else '_(Sim_Test)' if(args.sim) else ''}"
        HISTO_True = f"{HISTO_True}{'_(Mod_Test)' if(args.mod) else '_(Closure_Test)' if(args.closure) else '_(Sim_Test)' if(args.sim) else ''}"
        HISTO_True = HISTO_True.replace("(tdf)", "(gdf)")
    #######################################################################################################################################################################################################
    ####  Histogram Creations     #########################################################################################################################################################################
    Saved_Histos   = {}
    z_pT_Bin_Range = range(1, Get_Num_of_z_pT_Bins_w_Migrations(Q2_y_Bin_Num_In=Q2_Y_Bin)[1] + 1)
    for z_PT_BIN_NUM  in z_pT_Bin_Range:
        if(skip_condition_z_pT_bins(Q2_Y_BIN=Q2_Y_Bin, Z_PT_BIN=z_PT_BIN_NUM, BINNING_METHOD=Binning_Method, Common_z_pT_Range_Q=False)):
            continue
        HISTO_NAME_Binned = HISTO_NAME.replace("(z_pT_Bin_ALL)", f"(z_pT_Bin_{z_PT_BIN_NUM})")
        HISTO_True_Binned = HISTO_True.replace("(z_pT_Bin_ALL)", f"(z_pT_Bin_{z_PT_BIN_NUM})")
        if(args.data):
            Data_Legend_Titles = ("Experimental Data", "Reconstructed Monte Carlo" if(not (args.mod or args.closure)) else "Reconstructed MC (with weights)")
            Legend_Labels = ["Experimental Data", "Reconstructed Monte Carlo" if(not (args.mod or args.closure)) else "Reconstructed MC (with weights)"]
            if("rdf" in HISTO_NAME_Binned):
                HISTO_NAME_Binned_1 = HISTO_NAME_Binned.replace("SMEAR=Smear", "SMEAR=''")
                for clear_name in ["_(Mod_Test)", "_(Closure_Test)", "_(Sim_Test)"]:
                    HISTO_NAME_Binned_1 = HISTO_NAME_Binned_1.replace(clear_name, "")
                HISTO_NAME_Binned_2 = HISTO_NAME_Binned.replace("rdf", "mdf")
                HISTO_NAME_Binned_2 = HISTO_NAME_Binned_2.replace("SMEAR=''", f"SMEAR={args.smearing_option}")
                if(args.single_file):
                    for string, condition in [['_(Mod_Test)', args.mod], ['_(Closure_Test)', args.closure], ['_(Sim_Test)', args.sim]]:
                        if((string not in str(HISTO_NAME_Binned_2)) and condition):
                            HISTO_NAME_Binned_2 = f"{HISTO_NAME_Binned_2}{string}"
                            break
            else: # mdf in 'HISTO_NAME_Binned'
                HISTO_NAME_Binned_1 = HISTO_NAME_Binned.replace("SMEAR=''", f"SMEAR={args.smearing_option}")
                if(args.single_file):
                    for string, condition in [['_(Mod_Test)', args.mod], ['_(Closure_Test)', args.closure], ['_(Sim_Test)', args.sim]]:
                        if((string not in str(HISTO_NAME_Binned_1)) and condition):
                            HISTO_NAME_Binned_1 = f"{HISTO_NAME_Binned_1}{string}"
                            break
                HISTO_NAME_Binned_2 = HISTO_NAME_Binned.replace("mdf", "rdf")
                HISTO_NAME_Binned_2 = HISTO_NAME_Binned_2.replace("SMEAR=Smear", "SMEAR=''")
                for clear_name in ["_(Mod_Test)", "_(Closure_Test)", "_(Sim_Test)"]:
                    HISTO_NAME_Binned_2 = HISTO_NAME_Binned_2.replace(clear_name, "")
                Data_Legend_Titles = ("Reconstructed Monte Carlo" if(not (args.mod or args.closure)) else "Reconstructed MC (with weights)", "Experimental Data")
                Legend_Labels = ["Reconstructed Monte Carlo" if(not (args.mod or args.closure)) else "Reconstructed MC (with weights)", "Experimental Data"]
            try:
                if(not ROOT_Mod_In):
                    ROOT_Mod_In = ROOT_Input_In
            except:
                ROOT_Mod_In = ROOT_Input_In
            if((HISTO_NAME_Binned_1 in ROOT_Input_In.GetListOfKeys()) and (HISTO_NAME_Binned_2 in ROOT_Mod_In.GetListOfKeys())):
                if(args.verbose):
                    print(f"{color.BGREEN}Found: {color.END_b}{HISTO_NAME_Binned_1}{color.BGREEN} and {color.END_b}{HISTO_NAME_Binned_2}{color.END}")
                Saved_Histos[f"histo1_{z_PT_BIN_NUM}"], Saved_Histos[f"histo2_{z_PT_BIN_NUM}"], Saved_Histos[f"h_diff_{z_PT_BIN_NUM}"], Saved_Histos[f"h_uncertainty_{z_PT_BIN_NUM}"], Unfolding_Diff_Data_Input =  Compare_TH1D_Histograms(ROOT_In_1=ROOT_Input_In, HISTO_NAME_1=HISTO_NAME_Binned_1, ROOT_In_2=ROOT_Mod_In,   HISTO_NAME_2=HISTO_NAME_Binned_2,                                                                                                                                                 legend_labels=Data_Legend_Titles,                                                      output_prefix="Mod_Test_" if(args.mod) else "Closure_Test_" if(args.closure) else "Sim_Test_" if(args.sim) else "", SAVE=args.save_name, Format=args.file_format, TITLE=Standard_Histogram_Title_Addition, Q2y_str=Q2_Y_Bin, zPT_str=z_PT_BIN_NUM, Unfolding_Diff_Data_In=Unfolding_Diff_Data_Input, Return_Histos=True)
        elif(args.mod):
            if(args.single_file):
                try:
                    if(not ROOT_Mod_In):
                        ROOT_Mod_In = ROOT_Input_In
                except:
                    ROOT_Mod_In = ROOT_Input_In
            if((HISTO_NAME_Binned in ROOT_Input_In.GetListOfKeys()) and (f"{HISTO_NAME_Binned}{'' if(not args.single_file) else '_(Mod_Test)'}" in ROOT_Mod_In.GetListOfKeys())):
                if(args.verbose):
                    print(f"{color.BGREEN}Found: {color.END_b}{HISTO_NAME_Binned}{color.END}")
                    if(args.single_file):
                        print(f"\t{color.BGREEN}AND {color.END_b}{HISTO_NAME_Binned}_(Mod_Test){color.END}")
                Saved_Histos[f"histo1_{z_PT_BIN_NUM}"], Saved_Histos[f"histo2_{z_PT_BIN_NUM}"], Saved_Histos[f"h_diff_{z_PT_BIN_NUM}"], Saved_Histos[f"h_uncertainty_{z_PT_BIN_NUM}"], Unfolding_Diff_Data_Input =  Compare_TH1D_Histograms(ROOT_In_1=ROOT_Input_In, HISTO_NAME_1=HISTO_NAME_Binned,   ROOT_In_2=ROOT_Mod_In,   HISTO_NAME_2=f"{HISTO_NAME_Binned}{'' if(not args.single_file) else '_(Mod_Test)' if(args.mod) else '_(Closure_Test)' if(args.closure) else '_(Sim_Test)' if(args.sim) else ''}", legend_labels=("Unfolded with Normal MC", "Unfolded with Modulated MC"),               output_prefix="Mod_Test_",                                                                                          SAVE=args.save_name, Format=args.file_format, TITLE=Standard_Histogram_Title_Addition, Q2y_str=Q2_Y_Bin, zPT_str=z_PT_BIN_NUM, Unfolding_Diff_Data_In=Unfolding_Diff_Data_Input, Return_Histos=True)
        elif(args.sim or args.closure):
            if((HISTO_NAME_Binned in ROOT_Input_In.GetListOfKeys()) and (HISTO_True_Binned in ROOT_Input_In.GetListOfKeys())):
                if(args.verbose):
                    print(f"{color.BGREEN}Found: {color.END_b}{HISTO_NAME_Binned}{color.BGREEN} and {color.END_b}{HISTO_True_Binned}{color.END}")
                Saved_Histos[f"histo1_{z_PT_BIN_NUM}"], Saved_Histos[f"histo2_{z_PT_BIN_NUM}"], Saved_Histos[f"h_diff_{z_PT_BIN_NUM}"], Saved_Histos[f"h_uncertainty_{z_PT_BIN_NUM}"], Unfolding_Diff_Data_Input =  Compare_TH1D_Histograms(ROOT_In_1=ROOT_Input_In, HISTO_NAME_1=HISTO_NAME_Binned,   ROOT_In_2=ROOT_Input_In, HISTO_NAME_2=HISTO_True_Binned,                                                                                                                                                   legend_labels=("Unfolded Synthetic (MC) Data", "True Distribution of Synthetic Data"), output_prefix="Sim_Test_",                                                                                          SAVE=args.save_name, Format=args.file_format, TITLE=Standard_Histogram_Title_Addition, Q2y_str=Q2_Y_Bin, zPT_str=z_PT_BIN_NUM, Unfolding_Diff_Data_In=Unfolding_Diff_Data_Input, Return_Histos=True)
            else:
                print(f"{color.Error}Missing one of the following:\n{color.END_B} HISTO_NAME_Binned = {HISTO_NAME_Binned}\n HISTO_True_Binned = {HISTO_True_Binned}\n{color.END}")
        elif(HISTO_NAME in ROOT_Input_In.GetListOfKeys()):
            if(args.verbose):
                print(f"{color.BGREEN}Found: {color.END_b}{HISTO_NAME_Binned}{color.END}")
            Saved_Histos[str(z_PT_BIN_NUM)] = Save_Histograms_As_Images(ROOT_In=ROOT_Input_In, HISTO_NAME_In=HISTO_NAME_Binned, Format=args.file_format, SAVE=args.save_name, SAVE_prefix="Sim_Test_" if(args.sim) else "Mod_Test_" if(args.mod) else "", TITLE=Standard_Histogram_Title_Addition, Return_Histos=True)
        else:
            print(f"\n{color.Error}MISSING: {HISTO_NAME_Binned}{color.END}\n")

        if(Fit_Test and (args.unfold in ["Bayesian", "Bin", "gdf", "tdf"])):
            if(args.mod or args.sim or args.closure):
                Saved_Histos[f"histo1_{z_PT_BIN_NUM}_fitted"],     Saved_Histos[f"Fit_Function_1_{z_PT_BIN_NUM}"], Saved_Histos[f"Chi_Squared_1_{z_PT_BIN_NUM}"], Saved_Histos[f"Fit_Par_A_1_{z_PT_BIN_NUM}"], Saved_Histos[f"Fit_Par_B_1_{z_PT_BIN_NUM}"], Saved_Histos[f"Fit_Par_C_1_{z_PT_BIN_NUM}"] = Fitting_Phi_Function(Histo_To_Fit=Saved_Histos[f"histo1_{z_PT_BIN_NUM}"].asym_errors if(hasattr(Saved_Histos[f"histo1_{z_PT_BIN_NUM}"], "asym_errors")) else Saved_Histos[f"histo1_{z_PT_BIN_NUM}"], Method=args.unfold, Special=[Q2_Y_Bin, z_PT_BIN_NUM])
                if(hasattr(Saved_Histos[f"histo1_{z_PT_BIN_NUM}"], "asym_errors")):
                    Saved_Histos[f"histo1_{z_PT_BIN_NUM}"].asym_errors = Saved_Histos[f"histo1_{z_PT_BIN_NUM}_fitted"]
                else:
                    Saved_Histos[f"histo1_{z_PT_BIN_NUM}"]             = Saved_Histos[f"histo1_{z_PT_BIN_NUM}_fitted"]
                if(args.sim or args.closure):
                    Saved_Histos[f"histo2_{z_PT_BIN_NUM}_fitted"], Saved_Histos[f"Fit_Function_2_{z_PT_BIN_NUM}"], Saved_Histos[f"Chi_Squared_2_{z_PT_BIN_NUM}"], Saved_Histos[f"Fit_Par_A_2_{z_PT_BIN_NUM}"], Saved_Histos[f"Fit_Par_B_2_{z_PT_BIN_NUM}"], Saved_Histos[f"Fit_Par_C_2_{z_PT_BIN_NUM}"] = Fitting_Phi_Function(Histo_To_Fit=Saved_Histos[f"histo2_{z_PT_BIN_NUM}"].asym_errors if(hasattr(Saved_Histos[f"histo2_{z_PT_BIN_NUM}"], "asym_errors")) else Saved_Histos[f"histo2_{z_PT_BIN_NUM}"], Method='tdf' if('(tdf)' in str(HISTO_True)) else 'gdf' if('(gdf)' in str(HISTO_True)) else args.unfold, Special=[Q2_Y_Bin, z_PT_BIN_NUM])
                else:
                    Saved_Histos[f"histo2_{z_PT_BIN_NUM}_fitted"], Saved_Histos[f"Fit_Function_2_{z_PT_BIN_NUM}"], Saved_Histos[f"Chi_Squared_2_{z_PT_BIN_NUM}"], Saved_Histos[f"Fit_Par_A_2_{z_PT_BIN_NUM}"], Saved_Histos[f"Fit_Par_B_2_{z_PT_BIN_NUM}"], Saved_Histos[f"Fit_Par_C_2_{z_PT_BIN_NUM}"] = Fitting_Phi_Function(Histo_To_Fit=Saved_Histos[f"histo2_{z_PT_BIN_NUM}"].asym_errors if(hasattr(Saved_Histos[f"histo2_{z_PT_BIN_NUM}"], "asym_errors")) else Saved_Histos[f"histo2_{z_PT_BIN_NUM}"], Method=args.unfold, Special=[Q2_Y_Bin, z_PT_BIN_NUM])
                if(hasattr(Saved_Histos[f"histo2_{z_PT_BIN_NUM}"], "asym_errors")):
                    Saved_Histos[f"histo2_{z_PT_BIN_NUM}"].asym_errors = Saved_Histos[f"histo2_{z_PT_BIN_NUM}_fitted"]
                else:
                    Saved_Histos[f"histo2_{z_PT_BIN_NUM}"]             = Saved_Histos[f"histo2_{z_PT_BIN_NUM}_fitted"]
                Save_Fit_Outputs_To_ROOT(Histo_Name=HISTO_NAME_Binned,                                                     Chi_List=Saved_Histos[f"Chi_Squared_1_{z_PT_BIN_NUM}"], ParA_List=Saved_Histos[f"Fit_Par_A_1_{z_PT_BIN_NUM}"], ParB_List=Saved_Histos[f"Fit_Par_B_1_{z_PT_BIN_NUM}"], ParC_List=Saved_Histos[f"Fit_Par_C_1_{z_PT_BIN_NUM}"], Histo_New=Saved_Histos[f"histo1_{z_PT_BIN_NUM}"], Fit_Function=Saved_Histos[f"Fit_Function_1_{z_PT_BIN_NUM}"])
                Save_Fit_Outputs_To_ROOT(Histo_Name=HISTO_True_Binned if(args.sim or args.closure) else HISTO_NAME_Binned, Chi_List=Saved_Histos[f"Chi_Squared_2_{z_PT_BIN_NUM}"], ParA_List=Saved_Histos[f"Fit_Par_A_2_{z_PT_BIN_NUM}"], ParB_List=Saved_Histos[f"Fit_Par_B_2_{z_PT_BIN_NUM}"], ParC_List=Saved_Histos[f"Fit_Par_C_2_{z_PT_BIN_NUM}"], Histo_New=Saved_Histos[f"histo2_{z_PT_BIN_NUM}"], Fit_Function=Saved_Histos[f"Fit_Function_2_{z_PT_BIN_NUM}"])
            else:
                Saved_Histos[f"fitted_{z_PT_BIN_NUM}"],            Saved_Histos[f"Fit_Function_1_{z_PT_BIN_NUM}"], Saved_Histos[f"Chi_Squared_1_{z_PT_BIN_NUM}"], Saved_Histos[f"Fit_Par_A_1_{z_PT_BIN_NUM}"], Saved_Histos[f"Fit_Par_B_1_{z_PT_BIN_NUM}"], Saved_Histos[f"Fit_Par_C_1_{z_PT_BIN_NUM}"] = Fitting_Phi_Function(Histo_To_Fit=Saved_Histos[str(z_PT_BIN_NUM)].asym_errors        if(hasattr(Saved_Histos[str(z_PT_BIN_NUM)],        "asym_errors")) else Saved_Histos[str(z_PT_BIN_NUM)],        Method=args.unfold, Special=[Q2_Y_Bin, z_PT_BIN_NUM])
                if(hasattr(Saved_Histos[str(z_PT_BIN_NUM)],        "asym_errors")):
                    Saved_Histos[str(z_PT_BIN_NUM)].asym_errors        = Saved_Histos[f"fitted_{z_PT_BIN_NUM}"]
                else:
                    Saved_Histos[str(z_PT_BIN_NUM)]                    = Saved_Histos[f"fitted_{z_PT_BIN_NUM}"]
                Save_Fit_Outputs_To_ROOT(Histo_Name=HISTO_NAME_Binned, Chi_List=Saved_Histos[f"Chi_Squared_1_{z_PT_BIN_NUM}"], ParA_List=Saved_Histos[f"Fit_Par_A_1_{z_PT_BIN_NUM}"], ParB_List=Saved_Histos[f"Fit_Par_B_1_{z_PT_BIN_NUM}"], ParC_List=Saved_Histos[f"Fit_Par_C_1_{z_PT_BIN_NUM}"], Histo_New=Saved_Histos[str(z_PT_BIN_NUM)], Fit_Function=Saved_Histos[f"Fit_Function_1_{z_PT_BIN_NUM}"])
            
    ####  Histogram Creations     #########################################################################################################################################################################
    #######################################################################################################################################################################################################
    ####  Canvas (Main) Creation  #########################################################################################################################################################################
    All_z_pT_Canvas, All_z_pT_Canvas_cd_1, All_z_pT_Canvas_cd_1_Upper, All_z_pT_Canvas_cd_1_Lower, All_z_pT_Canvas_cd_2, All_z_pT_Canvas_cd_2_cols, legend = {}, {}, {}, {}, {}, {}, {}
    for canvas_num, Canvas_Name in enumerate(Canvases_to_Make):
        All_z_pT_Canvas[Canvas_Name] = Canvas_Create(Name=Canvas_Name, Num_Columns=2, Num_Rows=1, Size_X=3900, Size_Y=2175, cd_Space=0.01)
        All_z_pT_Canvas[Canvas_Name].SetFillColor(root_color.LGrey)
        All_z_pT_Canvas_cd_1[Canvas_Name]       = All_z_pT_Canvas[Canvas_Name].cd(1)
        All_z_pT_Canvas_cd_1[Canvas_Name].SetFillColor(root_color.LGrey)
        All_z_pT_Canvas_cd_1[Canvas_Name].SetPad(xlow=0.005, ylow=0.015, xup=0.27, yup=0.985)
        All_z_pT_Canvas_cd_1[Canvas_Name].Divide(1, 2, 0, 0)
        All_z_pT_Canvas_cd_1_Upper[Canvas_Name] = All_z_pT_Canvas_cd_1[Canvas_Name].cd(1)
        All_z_pT_Canvas_cd_1_Upper[Canvas_Name].SetPad(xlow=0, ylow=0.425, xup=1, yup=1)
        All_z_pT_Canvas_cd_1_Upper[Canvas_Name].Divide(1, 1, 0, 0)
        All_z_pT_Canvas_cd_1_Lower[Canvas_Name] = All_z_pT_Canvas_cd_1[Canvas_Name].cd(2)
        All_z_pT_Canvas_cd_1_Lower[Canvas_Name].SetPad(xlow=0, ylow=0, xup=1, yup=0.42)
        All_z_pT_Canvas_cd_1_Lower[Canvas_Name].Divide(1, 1, 0, 0)
        All_z_pT_Canvas_cd_1_Lower[Canvas_Name].cd(1).SetPad(xlow=0.035, ylow=0.025, xup=0.95, yup=0.975)
        All_z_pT_Canvas_cd_2[Canvas_Name]       = All_z_pT_Canvas[Canvas_Name].cd(2)
        All_z_pT_Canvas_cd_2[Canvas_Name].SetPad(xlow=0.28, ylow=0.015, xup=0.995, yup=0.9975)
        All_z_pT_Canvas_cd_2[Canvas_Name].SetFillColor(root_color.LGrey)
        if(Plot_Orientation in ["z_pT"]):
            number_of_rows, number_of_cols = Get_Num_of_z_pT_Rows_and_Columns(Q2_Y_Bin_Input=Q2_Y_Bin)
            All_z_pT_Canvas_cd_2[Canvas_Name].Divide(number_of_cols, number_of_rows, 0.0001, 0.0001)
        else:
            number_of_rows, number_of_cols = Get_Num_of_z_pT_Rows_and_Columns(Q2_Y_Bin_Input=Q2_Y_Bin)
            All_z_pT_Canvas_cd_2[Canvas_Name].Divide(1, number_of_cols, 0.0001, 0.0001)
            for ii in range(1, number_of_cols + 1, 1):
                All_z_pT_Canvas_cd_2_cols[Canvas_Name] = All_z_pT_Canvas_cd_2[Canvas_Name].cd(ii)
                All_z_pT_Canvas_cd_2_cols[Canvas_Name].Divide(number_of_rows, 1, 0.0001, 0.0001)
    ####  Canvas (Main) Creation End ######################################################################################################################################################################
    #######################################################################################################################################################################################################
        legend[Canvas_Name] = ROOT.TLegend(0.01, 0.01, 0.99, 0.99)
        Legend_Header = f"#splitline{{#scale[2]{{Q^{{2}}-y Bin {Q2_Y_Bin}}}}}{{#scale[1.5]{{Plots Shown}}}}"
        if(args.normalize):
            Legend_Header = f"#splitline{{{Legend_Header}}}{{Plots were normalized}}"
        if(args.data):
            Legend_Header = f"#splitline{{#scale[1.5]{{Comparing Data and MC}}}}{{{Legend_Header}}}"
        elif(args.closure):
            Legend_Header = f"#splitline{{#splitline{{#scale[1.5]{{Closure Test}}}}{{Corrected the MC with itself}}}}{{{Legend_Header}}}"
        legend[Canvas_Name].SetHeader(Legend_Header, "C") # option "C" allows to center the header
        if(args.mod or args.sim or args.closure or args.data):
            if(canvas_num   == 0):
                for ii, label in enumerate(Legend_Labels):
                    legend[Canvas_Name].AddEntry(Saved_Histos[f"histo{ii+1}_1"], label, "lep")
            elif(canvas_num == 1):
                legend[Canvas_Name].AddEntry(Saved_Histos[f"h_diff_1"],        f"#scale[0.85]{{#splitline{{Differences Between}}{{#splitline{{'{Legend_Labels[0]}' and}}{{'{Legend_Labels[1]}'}}}}}}",         "lep")
            else:
                legend[Canvas_Name].AddEntry(Saved_Histos[f"h_uncertainty_1"], f"#scale[0.85]{{#splitline{{Modeled Uncertainty between}}{{#splitline{{'{Legend_Labels[0]}' and}}{{'{Legend_Labels[1]}'}}}}}}", "lep")
        else:
            legend[Canvas_Name].AddEntry(Saved_Histos["1"], Legend_Labels[0], "lep")
        Draw_Canvas(All_z_pT_Canvas_cd_1_Upper[Canvas_Name], 1, 0.15)
        Blank = Saved_Histos[f"h_diff_1" if(args.mod or args.sim or args.closure or args.data) else "1"].Clone("EMPTY")
        Blank.SetTitle("")
        Blank.Draw("H P E0")
        legend[Canvas_Name].DrawClone()
        ROOT.gPad.Update()
        All_z_pT_Canvas[Canvas_Name].Update()
        for z_pT in range(1, Get_Num_of_z_pT_Bins_w_Migrations(Q2_y_Bin_Num_In=int(Q2_Y_Bin))[1]+1):
            if(skip_condition_z_pT_bins(Q2_Y_BIN=Q2_Y_Bin, Z_PT_BIN=z_pT, BINNING_METHOD=Binning_Method)):
                continue
            cd_number_of_z_pT_all_together = z_pT
            if(Plot_Orientation in ["z_pT"]):
                All_z_pT_Canvas_cd_2_z_pT_Bin = All_z_pT_Canvas_cd_2[Canvas_Name].cd(cd_number_of_z_pT_all_together)
                All_z_pT_Canvas_cd_2_z_pT_Bin.SetFillColor(root_color.LGrey)
                All_z_pT_Canvas_cd_2_z_pT_Bin.Divide(1, 1, 0, 0)
            else:
                cd_row = int(cd_number_of_z_pT_all_together/number_of_cols) + 1
                if(0  ==    (cd_number_of_z_pT_all_together%number_of_cols)):
                    cd_row += -1
                cd_col =     cd_number_of_z_pT_all_together - ((cd_row - 1)*number_of_cols)
                All_z_pT_Canvas_cd_2_z_pT_Bin_Row = All_z_pT_Canvas_cd_2[Canvas_Name].cd((number_of_cols - cd_col) + 1)
                All_z_pT_Canvas_cd_2_z_pT_Bin     = All_z_pT_Canvas_cd_2_z_pT_Bin_Row.cd((number_of_rows + 1) - cd_row)
                All_z_pT_Canvas_cd_2_z_pT_Bin.SetFillColor(root_color.LGrey)
                All_z_pT_Canvas_cd_2_z_pT_Bin.Divide(1, 1, 0, 0)

            Draw_Canvas(All_z_pT_Canvas_cd_2_z_pT_Bin, 1, 0.15)
            if(args.mod or args.sim or args.closure or args.data):
                if(canvas_num == 0):
                    if(hasattr(Saved_Histos[f"histo1_{z_pT}"], "asym_errors")):
                        Saved_Histos[f"histo1_{z_pT}"].Draw("H P SAME")
                        Saved_Histos[f"histo1_{z_pT}"].asym_errors.Draw("P E SAME")
                    else:
                        Saved_Histos[f"histo1_{z_pT}"].Draw("H P E0 SAME")
                    Saved_Histos[f"histo2_{z_pT}"].Draw("H P E0 SAME")
                elif(canvas_num == 1):
                    Saved_Histos[f"h_diff_{z_pT}"].Draw("H P E0")
                else:
                    Saved_Histos[f"h_uncertainty_{z_pT}"].Draw("H P E0")
            else:
                if(hasattr(Saved_Histos[str(z_pT)], "asym_errors")):
                    Saved_Histos[str(z_pT)].Draw("H P")
                    Saved_Histos[str(z_pT)].asym_errors.Draw("P E SAME")
                else:
                    Saved_Histos[str(z_pT)].Draw("H P E0")
            ROOT.gPad.Update()
            All_z_pT_Canvas[Canvas_Name].Update()
                
        ##################################################################### ################################################################ ################################################################
        #####==========#####        Saving Canvas        #####==========##### ################################################################ ################################################################
        ##################################################################### ################################################################ ################################################################
        Save_Name = f"{Canvas_Name}_{args.save_name}{args.file_format}"
        if(Plot_Orientation != "z_pT"):
            Save_Name = Save_Name.replace(f"{args.save_name}{args.file_format}", f"{args.save_name}_Flipped{args.file_format}")
        for replace in ["(", ")", "'", '"', "'"]:
            Save_Name = Save_Name.replace(replace, "")
        if(args.remake):
            Save_Name = f"Remade_with_normalization_weights_{Save_Name}"
        Save_Name = Save_Name.replace("__", "_")
        Save_Name = Save_Name.replace("SMEAR=Smear", "Smeared")
        Save_Name = Save_Name.replace("SMEAR=_", "")
        Save_Name = Save_Name.replace("__", "_")
        Save_Name = Save_Name.replace("_.", ".")
        All_z_pT_Canvas[Canvas_Name].SaveAs(Save_Name)
        print(f"Saved Image: {Save_Name}")
        ##################################################################### ################################################################ ################################################################
        #####==========#####        Saving Canvas        #####==========##### ################################################################ ################################################################
        ##################################################################### ################################################################ ################################################################
    return Unfolding_Diff_Data_Input

################################################################################################################################################
##### Large/Combined Image Function ############################################################################################################
################################################################################################################################################

In [9]:
def decode_Q2_y_and_z_pT(Q2_y_z_pT_4D_Bins):
    # Inverse mapping: from 4D bin index back to (Q2_y_Bin, z_pT_Bin)
    # Returns (0, 0) for invalid / underflow codes.
    if(Q2_y_z_pT_4D_Bins <= 0):
        return 0, 0

    # Maximum possible Q2_y_Bin (adjust if your analysis uses a different range)
    max_Q2_y_Bin = 17

    # Scan from highest Q2_y_Bin downwards so we grab the "largest offset < bin"
    for Q2_y_Bin in range(max_Q2_y_Bin, 0, -1):
        offset = 0

        if(Q2_y_Bin >  1):
            offset += 35
        if(Q2_y_Bin >  2):
            offset += 36
        if(Q2_y_Bin >  3):
            offset += 30
        if(Q2_y_Bin >  4):
            offset += 36
        if(Q2_y_Bin >  5):
            offset += 36
        if(Q2_y_Bin >  6):
            offset += 30
        if(Q2_y_Bin >  7):
            offset += 36
        if(Q2_y_Bin >  8):
            offset += 35
        if(Q2_y_Bin >  9):
            offset += 35
        if(Q2_y_Bin > 10):
            offset += 36
        if(Q2_y_Bin > 11):
            offset += 25
        if(Q2_y_Bin > 12):
            offset += 25
        if(Q2_y_Bin > 13):
            offset += 30
        if(Q2_y_Bin > 14):
            offset += 36
        if(Q2_y_Bin > 15):
            offset += 25
        if(Q2_y_Bin > 16):
            offset += 30

        if(Q2_y_z_pT_4D_Bins > offset):
            z_pT_Bin = Q2_y_z_pT_4D_Bins - offset
            return Q2_y_Bin, z_pT_Bin

    # If we somehow get here, the code was outside all valid ranges
    return 0, 0
    
Q2_y_z_pT_4D_Bins = 0
Q2_y_Bin_Find_In, z_pT_Bin_Find_In = decode_Q2_y_and_z_pT(Q2_y_z_pT_4D_Bins)
Q2_y_bins, z_pT_bins = Find_Q2_y_z_pT_Bin_Stats(Q2_y_Bin_Find=Q2_y_Bin_Find_In, z_pT_Bin_Find=z_pT_Bin_Find_In, List_Of_Histos_For_Stats_Search="Use_Center", Smearing_Q="''", DataType="bbb", Binning_Method_Input=Binning_Method)
Q2_bins, y_bins = Q2_y_bins
z_bins, pT_bins = z_pT_bins
Q2_Center, y_Center, z_Center, pT_Center = Q2_bins[1], y_bins[1], z_bins[1], pT_bins[1]

Q2_y_z_pT_4D_Bins
print(f"Q2_y_z_pT_4D_Bins = {Q2_y_z_pT_4D_Bins}")
print(f"\t2D Bins: ({Q2_y_Bin_Find_In}-{z_pT_Bin_Find_In})")

print(f"Q2_Center = {Q2_Center}")
print(f"y_Center  = {y_Center}")
print(f"z_Center  = {z_Center}")
print(f"pT_Center = {pT_Center}")

# [[[Q2_bin_min, Q2_Center, Q2_bin_max], [y_bin_min, y_Center, y_bin_max]], [[z_bin_min, z_Center, z_bin_max], [pT_bin_min, pT_Center, pT_bin_max]]]

ERROR
Q2_y_z_pT_4D_Bins = 0
	2D Bins: (0-0)
Q2_Center = 2.2
y_Center  = 0.7
z_Center  = 0.425
pT_Center = 0.525


In [11]:
pT_Ranges_per_Q2_y_Bin = {}
Q2_y_Bin_Num_In = 1
for Q2_y_Bin_Num_In in range(1, 18):
    pT_Ranges_per_Q2_y_Bin[Q2_y_Bin_Num_In] = []
    Total_Number_of_Bins, _, _ = Get_Num_of_z_pT_Bins_w_Migrations(Q2_y_Bin_Num_In=Q2_y_Bin_Num_In)
    for z_pT in range(1, Total_Number_of_Bins + 1, 1):
        _, _, pT_max, pT_min = Get_z_pT_Bin_Corners(z_pT_Bin_Num=z_pT, Q2_y_Bin_Num=Q2_y_Bin_Num_In)
        # pT_range = [pT_max, pT_min]
        pT_range = [pT_min, pT_max]
        if(pT_range not in pT_Ranges_per_Q2_y_Bin[Q2_y_Bin_Num_In]):
            pT_Ranges_per_Q2_y_Bin[Q2_y_Bin_Num_In].append(pT_range)
print("\n\nDone\n\n")

for Q2_y_Bin_Num_In in range(1, 18):
    print(f"pT_Ranges_per_Q2_y_Bin[{Q2_y_Bin_Num_In}] = {pT_Ranges_per_Q2_y_Bin[Q2_y_Bin_Num_In]}")

print("\n\n# pT_Ranges_per_Q2_y_Bin['key'] = [[pT_min, pT_max], [pT_min, pT_max], ...] # Each list of '[pT_min, pT_max]' corresponds to a different range of individual pT bins for the given Q2_y_Bin (given as the integer key)")
print(f"pT_Ranges_per_Q2_y_Bin = {pT_Ranges_per_Q2_y_Bin}")


Removing ' - REMOVE - MIGRATION BIN' to be able to return the z-pT borders from `Get_z_pT_Bin_Corners()`.
Will return:
	[z_max, z_min, pT_max, pT_min] = [0.33, 0.26, 0.61, 0.48]

Removing ' - REMOVE - MIGRATION BIN' to be able to return the z-pT borders from `Get_z_pT_Bin_Corners()`.
Will return:
	[z_max, z_min, pT_max, pT_min] = [0.33, 0.27, 0.6, 0.46]

Removing ' - REMOVE - MIGRATION BIN' to be able to return the z-pT borders from `Get_z_pT_Bin_Corners()`.
Will return:
	[z_max, z_min, pT_max, pT_min] = [0.33, 0.27, 0.6, 0.45]


Done


pT_Ranges_per_Q2_y_Bin[1] = [[0.05, 0.22], [0.22, 0.32], [0.32, 0.42], [0.42, 0.52], [0.52, 0.63], [0.63, 0.75], [0.75, 0.99]]
pT_Ranges_per_Q2_y_Bin[2] = [[0.05, 0.25], [0.25, 0.35], [0.35, 0.45], [0.45, 0.54], [0.54, 0.67], [0.67, 0.93]]
pT_Ranges_per_Q2_y_Bin[3] = [[0.05, 0.2], [0.2, 0.3], [0.3, 0.39], [0.39, 0.49], [0.49, 0.59], [0.59, 0.76]]
pT_Ranges_per_Q2_y_Bin[4] = [[0.05, 0.2], [0.2, 0.29], [0.29, 0.38], [0.38, 0.48], [0.48, 0.61], [0.61, 0.8

In [25]:
Q2_Borders = [2.0, 2.4, 2.9, 3.7, 5.3, 7.9]
y_Borders  = [0.35, 0.45, 0.55, 0.65, 0.75]
Q2_Centers = []
last = 0
# print("\nQ2:")
for num, ii in enumerate(Q2_Borders):
    if(num == 0):
        last = ii
        continue
    else:
        # print(f"""0.5*({ii} + {last}) = {round((ii + last)/2, 2)}""")
        Q2_Centers.append(f"{round((ii + last)/2, 2):0.2f}")
        last = ii

# print("\ny:")
y_Centers = []
last = 0
for num, ii in enumerate(y_Borders):
    if(num == 0):
        last = ii
        continue
    else:
        # print(f"""0.5*({ii} + {last}) = {round((ii + last)/2, 2)}""")
        y_Centers.append(f"{round((ii + last)/2, 2):0.2f}")
        last = ii

print(f"""Q2_Centers = {str(Q2_Centers).replace("'", "")}""")
print(f"""y_Centers  = {str(y_Centers).replace("'", "")}""")


Q2_Centers = [2.20, 2.65, 3.30, 4.50, 6.60]
y_Centers  = [0.40, 0.50, 0.60, 0.70]


In [9]:
function = """
(a2*pT_val^2 + b2*pT_val + c2)*z_val^2 + (a1*pT_val^2 + b1*pT_val + c1)*z_val + (a0*pT_val^2 + b0*pT_val + c0)
"""
list_of_Q2_pars = []
for pT_par in ["a2", "b2", "c2", "a1", "b1", "c1", "a0", "b0", "c0"]:
    function = function.replace(pT_par, f"(a{pT_par}*Q2_val^2 + b{pT_par}*Q2_val + c{pT_par})")
    if(f"a{pT_par}" not in list_of_Q2_pars):
        list_of_Q2_pars.append(f"a{pT_par}")
    if(f"b{pT_par}" not in list_of_Q2_pars):
        list_of_Q2_pars.append(f"b{pT_par}")
    if(f"c{pT_par}" not in list_of_Q2_pars):
        list_of_Q2_pars.append(f"c{pT_par}")

print("\n")
print(f"function = {function}\n")
print(f"list_of_Q2_pars = {list_of_Q2_pars} (len = {len(list_of_Q2_pars)})\n")

list_of_y_pars = []
for Q2_par in list_of_Q2_pars:
    function = function.replace(Q2_par, f"(A{Q2_par}*y_val^2 + B{Q2_par}*y_val + C{Q2_par})")
    if(f"A{Q2_par}" not in list_of_y_pars):
        list_of_y_pars.append(f"A{Q2_par}")
    if(f"B{Q2_par}" not in list_of_y_pars):
        list_of_y_pars.append(f"B{Q2_par}")
    if(f"C{Q2_par}" not in list_of_y_pars):
        list_of_y_pars.append(f"C{Q2_par}")

print("\n")
print(f"function = {function}\n")
print(f"list_of_Q2_pars = {list_of_y_pars} (len = {len(list_of_y_pars)})\n")




function = 
((aa2*Q2_val^2 + ba2*Q2_val + ca2)*pT_val^2 + (ab2*Q2_val^2 + bb2*Q2_val + cb2)*pT_val + (ac2*Q2_val^2 + bc2*Q2_val + cc2))*z_val^2 + ((aa1*Q2_val^2 + ba1*Q2_val + ca1)*pT_val^2 + (ab1*Q2_val^2 + bb1*Q2_val + cb1)*pT_val + (ac1*Q2_val^2 + bc1*Q2_val + cc1))*z_val + ((aa0*Q2_val^2 + ba0*Q2_val + ca0)*pT_val^2 + (ab0*Q2_val^2 + bb0*Q2_val + cb0)*pT_val + (ac0*Q2_val^2 + bc0*Q2_val + cc0))


list_of_Q2_pars = ['aa2', 'ba2', 'ca2', 'ab2', 'bb2', 'cb2', 'ac2', 'bc2', 'cc2', 'aa1', 'ba1', 'ca1', 'ab1', 'bb1', 'cb1', 'ac1', 'bc1', 'cc1', 'aa0', 'ba0', 'ca0', 'ab0', 'bb0', 'cb0', 'ac0', 'bc0', 'cc0'] (len = 27)



function = 
(((Aaa2*y_val^2 + Baa2*y_val + Caa2)*Q2_val^2 + (Aba2*y_val^2 + Bba2*y_val + Cba2)*Q2_val + (Aca2*y_val^2 + Bca2*y_val + Cca2))*pT_val^2 + ((Aab2*y_val^2 + Bab2*y_val + Cab2)*Q2_val^2 + (Abb2*y_val^2 + Bbb2*y_val + Cbb2)*Q2_val + (Acb2*y_val^2 + Bcb2*y_val + Ccb2))*pT_val + ((Aac2*y_val^2 + Bac2*y_val + Cac2)*Q2_val^2 + (Abc2*y_val^2 + Bbc2*y_val + Cbc2)*Q2_

In [9]:
Q2_y_z_pT_4D_Bin_event_val_last = 0
for Q2_y_Bin_event_val in range(1, 18):
    Q2_y_z_pT_4D_Bin_event_val = 1
    # if(Q2_y_Bin_event_val > 1):
    #     Q2_y_z_pT_4D_Bin_event_val += 35
    # if(Q2_y_Bin_event_val > 2):
    #     Q2_y_z_pT_4D_Bin_event_val += 42
    # if(Q2_y_Bin_event_val > 3):
    #     Q2_y_z_pT_4D_Bin_event_val += 35
    # if(Q2_y_Bin_event_val > 4):
    #     Q2_y_z_pT_4D_Bin_event_val += 36
    # if(Q2_y_Bin_event_val > 5):
    #     Q2_y_z_pT_4D_Bin_event_val += 36
    # if(Q2_y_Bin_event_val > 6):
    #     Q2_y_z_pT_4D_Bin_event_val += 30
    # if(Q2_y_Bin_event_val > 7):
    #     Q2_y_z_pT_4D_Bin_event_val += 30
    # if(Q2_y_Bin_event_val > 8):
    #     Q2_y_z_pT_4D_Bin_event_val += 30
    # if(Q2_y_Bin_event_val > 9):
    #     Q2_y_z_pT_4D_Bin_event_val += 36
    # if(Q2_y_Bin_event_val > 10):
    #     Q2_y_z_pT_4D_Bin_event_val += 36
    # if(Q2_y_Bin_event_val > 11):
    #     Q2_y_z_pT_4D_Bin_event_val += 30
    # if(Q2_y_Bin_event_val > 12):
    #     Q2_y_z_pT_4D_Bin_event_val += 20
    # if(Q2_y_Bin_event_val > 13):
    #     Q2_y_z_pT_4D_Bin_event_val += 25
    # if(Q2_y_Bin_event_val > 14):
    #     Q2_y_z_pT_4D_Bin_event_val += 25
    # if(Q2_y_Bin_event_val > 15):
    #     Q2_y_z_pT_4D_Bin_event_val += 20
    # if(Q2_y_Bin_event_val > 16):
    #     Q2_y_z_pT_4D_Bin_event_val += 20

    if(Q2_y_Bin_event_val >  1):
        Q2_y_z_pT_4D_Bin_event_val += 35
    if(Q2_y_Bin_event_val >  2):
        Q2_y_z_pT_4D_Bin_event_val += 36
    if(Q2_y_Bin_event_val >  3):
        Q2_y_z_pT_4D_Bin_event_val += 30
    if(Q2_y_Bin_event_val >  4):
        Q2_y_z_pT_4D_Bin_event_val += 36
    if(Q2_y_Bin_event_val >  5):
        Q2_y_z_pT_4D_Bin_event_val += 36
    if(Q2_y_Bin_event_val >  6):
        Q2_y_z_pT_4D_Bin_event_val += 30
    if(Q2_y_Bin_event_val >  7):
        Q2_y_z_pT_4D_Bin_event_val += 36
    if(Q2_y_Bin_event_val >  8):
        Q2_y_z_pT_4D_Bin_event_val += 35
    if(Q2_y_Bin_event_val >  9):
        Q2_y_z_pT_4D_Bin_event_val += 35
    if(Q2_y_Bin_event_val > 10):
        Q2_y_z_pT_4D_Bin_event_val += 36
    if(Q2_y_Bin_event_val > 11):
        Q2_y_z_pT_4D_Bin_event_val += 25
    if(Q2_y_Bin_event_val > 12):
        Q2_y_z_pT_4D_Bin_event_val += 25
    if(Q2_y_Bin_event_val > 13):
        Q2_y_z_pT_4D_Bin_event_val += 30
    if(Q2_y_Bin_event_val > 14):
        Q2_y_z_pT_4D_Bin_event_val += 36
    if(Q2_y_Bin_event_val > 15):
        Q2_y_z_pT_4D_Bin_event_val += 25
    if(Q2_y_Bin_event_val > 16):
        Q2_y_z_pT_4D_Bin_event_val += 30

    if(Q2_y_Bin_event_val != 1):
        print(f"Q^{{2}}-y Bin {Q2_y_Bin_event_val-1:>2.0f}: 4D Bins {Q2_y_z_pT_4D_Bin_event_val_last:>3.0f} to {Q2_y_z_pT_4D_Bin_event_val-1:>3.0f}")
    if(Q2_y_Bin_event_val > 16):
        print(f"Q^{{2}}-y Bin {Q2_y_Bin_event_val:>2.0f}: 4D Bins {Q2_y_z_pT_4D_Bin_event_val:>3.0f}+")
    Q2_y_z_pT_4D_Bin_event_val_last = Q2_y_z_pT_4D_Bin_event_val

Q^{2}-y Bin  1: 4D Bins   1 to  35
Q^{2}-y Bin  2: 4D Bins  36 to  71
Q^{2}-y Bin  3: 4D Bins  72 to 101
Q^{2}-y Bin  4: 4D Bins 102 to 137
Q^{2}-y Bin  5: 4D Bins 138 to 173
Q^{2}-y Bin  6: 4D Bins 174 to 203
Q^{2}-y Bin  7: 4D Bins 204 to 239
Q^{2}-y Bin  8: 4D Bins 240 to 274
Q^{2}-y Bin  9: 4D Bins 275 to 309
Q^{2}-y Bin 10: 4D Bins 310 to 345
Q^{2}-y Bin 11: 4D Bins 346 to 370
Q^{2}-y Bin 12: 4D Bins 371 to 395
Q^{2}-y Bin 13: 4D Bins 396 to 425
Q^{2}-y Bin 14: 4D Bins 426 to 461
Q^{2}-y Bin 15: 4D Bins 462 to 486
Q^{2}-y Bin 16: 4D Bins 487 to 516
Q^{2}-y Bin 17: 4D Bins 517+
